# **PROYECTO SCHUTZ EN PYTHON PARA CREACIÓN DE DASHBOARD 1 - PARTE 1**

## 1.- IMPORTACIÓN DE LIBRERÍAS PARA EL PROYECTO

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
from itertools import zip_longest
import io
import requests
from io import BytesIO
import pandas as pd
import chardet
import re
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from gspread_dataframe import set_with_dataframe
from urllib.request import FancyURLopener
from urllib.request import urlopen
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import datetime
import ssl
import warnings
from datetime import datetime, timedelta


In [2]:
pd.options.display.max_rows = None
pd.options.display.float_format = '{:.2f}'.format

warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

## 2.- LECTURA DE DATAFRAMES

### 2.1 LECTURA "LIBRO MAYOR"

In [3]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Libro%20Mayor%202023.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_libro_mayor = pd.DataFrame(data)

# Imprimir el dataframe
df_libro_mayor.sample(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia
2696,4101010-000,2023,3,2023-03-07T00:00:00-03:00,CIN,172,"Disco corte 4 1/2"" metal",P-1688,I17440838,3040,0,49739
4106,4101010-000,2023,4,2023-04-13T00:00:00-03:00,CIN,469,"Perno U 3/8"" x 2 1/2"" con tuercas y goli",P-1726,I15410491,18400,0,50411
919,4101010-000,2023,1,2023-01-04T00:00:00-03:00,CIN,79,Camara Bala Hikvision DS-2CD1623G0-IZ(2.,P-1682,S09222817,77608,0,48728
5617,4101010-000,2023,5,2023-05-17T00:00:00-04:00,CIN,957,"Union copla red 2 1/2"" x 2"" RAN RAN",P-1712,P08200234,19272,0,51247
4849,4101010-000,2023,5,2023-05-03T00:00:00-03:00,CIN,47,Derivador tap 8 sal 20dB 5-1200MHz alamb,P-1707,C06111967,64140,0,50825


In [4]:
df_libro_mayor.to_csv('archivos_respaldo/df_libro_mayor_1.csv', index = False)

### 2.2 LECTURA "DESPACHOS REALIZADOS"

In [5]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Despachos%20Realizados.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_desp_realizados = pd.DataFrame(data)

# Imprimir el dataframe
df_desp_realizados.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
4256,P-Proyectos,VCO,2023-02-23T00:00:00-03:00,49555,None,H21571172,1,Guante Multiflex Negro,UN,788,788
6341,NaN,STT,2022-05-12T00:00:00-03:00,45401,None,S09240328,1,Video balun pasivo (par),UN,1808,1808
13690,P-1688,VCO,2023-01-23T00:00:00-03:00,49056,1053,C04091042,1,Unidad de Citofonia Lef 10,UN,102041,102041
14992,P-1694,VCO,2023-05-11T00:00:00-03:00,51078,1102,C08712745,1,Derivador tap 4 sal 20dB 5-1200MHz alamb,UN,4349,4349
9667,P-1712,VCO,2022-08-04T00:00:00-04:00,46600,1114,I15412322,15,"Teflon 3/4"" X 10M",UN,175,2625


In [6]:
df_desp_realizados.to_csv('archivos_respaldo/df_desp_realizados_1.csv', index = False)

### 2.3 LECTURA "CODIGOS DE PROYECTO"

In [7]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\codigos%20de%20proyectos.txt'

# Descargar el archivo XML desde la URL
with urllib.request.urlopen(url) as response:
    xml_data = response.read()

# Parsear el archivo XML
root = ET.fromstring(xml_data)

# Crear una lista de diccionarios con los datos
data = []
for child in root:
    row = {}
    for subchild in child:
        row[subchild.tag] = subchild.text
    data.append(row)

# Convertir la lista de diccionarios en un dataframe de Pandas
df_cod_proyecto = pd.DataFrame(data)

# Imprimir el dataframe
df_cod_proyecto.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE
166,P-1517,Estacionamientos Pedro de Valdivia,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO
177,P-1535,Amapolas,Empresa Constructora DLP S.A.
142,P-1515,Pio X,CONSTRUCTORA ARMAS LTDA
162,P-1685,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.
46,P-1565,Duble,CANTAUCO CONSTRUCCION LTDA.


In [8]:
df_cod_proyecto.to_csv('archivos_respaldo/df_cod_proyecto_1.csv', index = False)

### 2.4 LECTURA "PROYECTADOS"

In [9]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso
import datetime

# Obtener el mes actual de acuerdo a la fecha actual, y el resultado se convierte en el nombre del MES para luego buscar aquel nombre en el archivo.
mes_actual = datetime.date.today().month

print(mes_actual)

if mes_actual == 1:
    mes_actual2 = "ENERO"
elif mes_actual == 2:
    mes_actual2 = "FEBRERO"
elif mes_actual == 3:
    mes_actual2 = "MARZO"
elif mes_actual == 4:
    mes_actual2 = "ABRIL"
elif mes_actual == 5:
    mes_actual2 = "MAYO"
elif mes_actual == 6:
    mes_actual2 = "JUNIO"
elif mes_actual == 7:
    mes_actual2 = "JULIO"
elif mes_actual == 8:
    mes_actual2 = "AGOSTO"
elif mes_actual == 9:
    mes_actual2 = "SEPTIEMBRE"
elif mes_actual == 10:
    mes_actual2 = "OCTUBRE"
elif mes_actual == 11:
    mes_actual2 = "NOVIEMBRE"
else:
    mes_actual2 = "DICIEMBRE"

print(mes_actual2)

6
JUNIO


In [10]:
ssl._create_default_https_context = ssl._create_unverified_context

# Lectura de presupuestos desde dropbox
df_proyectado = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_actual2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4212000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.4-2 LECTURA "PROYECTADOS 2"

In [11]:
# Get the current date
import datetime

fecha_actual = datetime.datetime.now()

# Calculate the previous month
mes_anterior2 = (fecha_actual - timedelta(days=30)).month
print(mes_anterior2)

if mes_anterior2 == 1:
    mes_anterior2 = "ENERO"
elif mes_anterior2 == 2:
    mes_anterior2 = "FEBRERO"
elif mes_anterior2 == 3:
    mes_anterior2 = "MARZO"
elif mes_anterior2 == 4:
    mes_anterior2 = "ABRIL"
elif mes_anterior2 == 5:
    mes_anterior2 = "MAYO"
elif mes_anterior2 == 6:
    mes_anterior2 = "JUNIO"
elif mes_anterior2 == 7:
    mes_anterior2 = "JULIO"
elif mes_anterior2 == 8:
    mes_anterior2 = "AGOSTO"
elif mes_anterior2 == 9:
    mes_anterior2 = "SEPTIEMBRE"
elif mes_anterior2 == 10:
    mes_anterior2 = "OCTUBRE"
elif mes_anterior2 == 11:
    mes_anterior2 = "NOVIEMBRE"
else:
    mes_anterior2 = "DICIEMBRE"

print(mes_anterior2)

# Lectura de presupuestos desde dropbox
df_proyectado_1M = pd.read_excel("https://www.dropbox.com/s/m96bbvnctgmaxrz/Proyecci%C3%B3n%20mensual%20%282023%29.xlsx?dl=1",sheet_name=mes_anterior2 ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_proyectado_1M.head(20)

5
MAYO


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.5 LECTURA "LIBRO MAYOR AÑOS ANTERIORES"

In [12]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         'keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

In [13]:
spr = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Fx1nerTu10PZPOwknLcQuri_I6ARat5h7zuq4P3vypg/edit#gid=1097918290')
recipients = spr.sheet1.get_all_values()

# Se el string creado en un dataframe
df_libro_mayor_old = pd.DataFrame(recipients)

# se reasignan los headers en el dataframe.
new_header = df_libro_mayor_old.iloc[0] #grab the first row for the header
df_libro_mayor_old = df_libro_mayor_old[1:] #take the data less the header row
df_libro_mayor_old.columns = new_header #set the header row as the df header

# Se conservan solo las columnas de interés
#df_seguridad_v3 = df_seguridad_v3[["Nº Cédula","Nombres y apellidos","APRUEBA FASE DE CALIFICACIÓN ?","PUNTOS VIGENTES","APRUEBA FASE DE EVALUACIÓN ?","RESULTADO FINAL"]]

print("Las dimensiones del dataframe son :",df_libro_mayor_old.shape)
df_libro_mayor_old.sample(5)

Las dimensiones del dataframe son : (17368, 13)


,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
10896,4101010-000,2022,5,5/27/22 0:00,CIN,510,"Gabinete Incendio P/Carrete 1"" 900x900x",P-1693,P08180247,45580,336000,0,
5841,4101010-000,2021,10,10/20/21 0:00,CIN,677,Cable paralelo 2x20 color,P-1652,T14380054,43484,71600,0,
1205,4101010-000,2021,1,1/4/21 0:00,CIN,709,Instalacion cerco electrico La Florida,P-1629,O19501068,40113,168000,0,
12146,4101010-000,2022,7,7/29/22 0:00,CIN,1243,Telefono Loft Basic Duox Plus Fermax F03451,P-1692,C04082154,46512,150854,0,
16022,4150002-000,2022,7,7/31/22 0:00,AJC,1507,REMUNERACIONES DIRECTAS PROYECTO,P-1685,13285624,,1008434,0,


In [14]:
df_libro_mayor_old.to_csv('archivos_respaldo/df_libro_mayor_old.csv', mode='a')

### 2.6 LECTURA "PLANILLA RESUMEN - PRESPUESTOS"

In [15]:
# Lectura de presupuestos desde dropbox
df_presupuesto = pd.read_excel("https://www.dropbox.com/s/miow7p9wl7t3jsr/PLANILLA%20RESUMEN.xlsb.xlsx?dl=1",sheet_name= "PROYECTOS" ,engine="openpyxl")
# OJO !! El link original tiene un 0 para ser visto desde la web y un 1 para descargarlo.....
df_presupuesto.head(5)

,PROYECTO,OBRA,RUT,CONSTRUCTORA,FECHA INICIO,FECHA TERMINO,ULTIMO EP PRESENTADO,AVANCE EN OBRA,MONTO EN $,$ POR FACTURAR,...,217465.036046312,217465.036046312.1,217465.036046312.2,217465.036046312.3,0,0.1,0.2,0.3,0.4,0.5
0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,8111745112.45,3273128441.17,...,1,2.00,3.00,4.00,5.00,6.00,7.00,8.00,9.00,10.00
1,P-1632,Chorrillos Clorox,76.108.978-1,SuKsa S.A,2019-02-04 00:00:00,2019-05-31,NaN,0.99,138107706.03,915642.26,...,217465.04,217465.04,217465.04,217465.04,0.00,0.00,0.00,0.00,0.00,0.00
2,P-1638,Colon,78134800-7,Grevia,2019-08-13 00:00:00,2020-03-27,NaN,1.00,43206282.00,2177592.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P-1645,Los 3 Antonios,77.710.830-1,EDICASA,2019-08-31 00:00:00,NaT,NaN,0.98,18150000.00,208732.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P-1647,Bodega corrosivos TW,76.564.630-8,Inmobiliaria y Rentas Logisticas S.A,NaN,NaT,NaN,0.58,29482131.00,12437344.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_presupuesto.to_csv('archivos_respaldo/df_presupuesto.csv', mode='a')

### 2.7 LECTURA "TABLA- PRESPUESTOS"

In [17]:
url = 'http://asp.maximise.cl/query.ashx?alias=gsea&user=excel_bd&password=excel_bd&filename=queries\Presupuesto Proyectos.txt'
response = requests.get(url)
query_result = response.text
soup = BeautifulSoup(response.content, 'xml')
rows = soup.find_all('Table')


data = []
for row in rows:
    values = [cell.text for cell in row.find_all()]
    data.append(values)

df_tabla_prespuesto = pd.DataFrame(data, columns=['CODIGO_PROYECTO','PARTIDA_PRESUPUESTARIA','PRODUCTO','UNIDAD','CANTIDAD','COSTO_UNITARIO','COSTO_TOTAL'])

# Exportación a excel
df_tabla_prespuesto.to_csv('df_tabla_prespuesto.csv', index = False)

df_tabla_prespuesto.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490,428490
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866,1022434
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846,95384
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608,138224
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200,353600


## 3.- TRATAMIENTO DE DATAFRAMES

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR

#### 3.1.1 - Exploración inicial de los datos

In [18]:
df_libro_mayor.shape

(7948, 12)

In [19]:
df_libro_mayor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7948 entries, 0 to 7947
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Cta_Contable     7948 non-null   object
 1   Año              7948 non-null   object
 2   Mes              7948 non-null   object
 3   Fecha            7923 non-null   object
 4   Tipo_Comp        7948 non-null   object
 5   Num_Comp         7948 non-null   object
 6   Glosa            7948 non-null   object
 7   Primer_Analisis  7945 non-null   object
 8   Referencia       7948 non-null   object
 9   Debito           7948 non-null   object
 10  Credito          7948 non-null   object
 11  Num_Referencia   6358 non-null   object
dtypes: object(12)
memory usage: 745.2+ KB


#### 3.1.2 - Estandarización campo "Debito" y Crédito

In [20]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor['Debito_2'] = pd.to_numeric(df_libro_mayor['Debito'], errors='coerce')
df_libro_mayor['Credito_2'] = pd.to_numeric(df_libro_mayor['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor['Credito_2'] = df_libro_mayor['Credito_2'].fillna(0)

df_libro_mayor.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.3 - Creación campo Codigo_proyecto

In [21]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Primer_Analisis']

cols = df_libro_mayor.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('Codigo_proyecto'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor = df_libro_mayor.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor['Codigo_proyecto'] = df_libro_mayor['Codigo_proyecto'].str.upper()

df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0


#### 3.1.4 - Creación de campo total

In [22]:
df_libro_mayor['TOTAL'] = df_libro_mayor['Credito_2'] - df_libro_mayor['Debito_2']
df_libro_mayor.head(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL
0,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,173,VENTAS DE PROYECTO,P-1677,P-1677,4081983,0,NaN,4081983,0,-4081983
1,P-1677,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,174,VENTAS DE PROYECTO,P-1677,P-1677,6464848,0,NaN,6464848,0,-6464848
2,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,824,VENTAS DE PROYECTO,P-1700,P-1700,4673685,0,10604,4673685,0,-4673685
3,P-1700,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,825,VENTAS DE PROYECTO,P-1700,P-1700,1570055,0,10603,1570055,0,-1570055
4,P-1704,3101030-000,2023,1,2023-01-01T00:00:00-03:00,AJC,827,VENTAS DE PROYECTO,P-1704,P-1704,3613652,0,10602,3613652,0,-3613652


#### 3.1.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [23]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return 'INGRESO'
    elif cta_contable == "4101010-000":
        return 'COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return 'COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return 'COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor['Tipo_cuenta'] = df_libro_mayor['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
4569,P-1728,4101010-000,2023,4,2023-04-21T00:00:00-03:00,CIN,1024,Modulo Monitoreo simple EST CT1,P-1728,P07120833,15022,0,50665,15022,0,-15022,COSTO MATERIALES
6171,P-1706,4101010-000,2023,6,2023-06-05T00:00:00-04:00,CIN,95,Telefono Veo Duox Plus Fermax F03444,P-1706,C04082152,621280,0,51523,621280,0,-621280,COSTO MATERIALES
2283,P-1696,4101010-000,2023,2,2023-02-16T00:00:00-03:00,CCO,937,COSTOS DE PROYECTOS,P-1696,27212485-K,206897,0,30563,206897,0,-206897,COSTO MATERIALES
408,P-1697,3101030-000,2023,4,2023-04-28T00:00:00-03:00,CVT,1318,Obra: 18000 Estocolmo\nEp (9) CCDD neto UF 192...,P-1697,96897350-9,0,0,13709,0,0,0,INGRESO
7821,P-OFICINA,4150002-003,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES DIRECTAS PROYECTO POR DISTRIBUIR,P-Oficina,10659869,56717,0,NaN,56717,0,-56717,OTRA


In [24]:
# EXPLORACIÓN DE DATOS

df_libro_mayor_piv = pd.pivot_table(df_libro_mayor, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_piv = pd.DataFrame(df_libro_mayor_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
1,COSTO MANO DE OBRA,2023,2,7,82645544,-82645537
2,COSTO MANO DE OBRA,2023,3,0,96802066,-96802066
14,COSTO SUPERVISIÓN,2023,4,0,5096121,-5096121
3,COSTO MANO DE OBRA,2023,4,0,111885468,-111885468
17,INGRESO,2023,2,231086456,0,231086456
16,INGRESO,2023,1,449490833,191247845,258242988
23,OTRA,2023,2,0,22347286,-22347286
11,COSTO SUPERVISIÓN,2023,1,0,4336206,-4336206
4,COSTO MANO DE OBRA,2023,5,0,119945281,-119945281
0,COSTO MANO DE OBRA,2023,1,0,72542885,-72542885


#### 3.1.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [25]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor2 = df_libro_mayor.copy()

In [26]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-PROYECTOS' 'P-1669'
 'P-1667' 'P-1668' 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658'
 'P-1660' 'P-1644' 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653'
 'P-1673' 'P-1683' 'P-1661' 'P-1612' 'P-1728' 'P-SERVICIOS' 'P-1639'
 'P-1674' 'P1718' 'P-1620' 'P-1603' 'P-1626' 'P-1649' 'P-1685' 'P-1632'
 'P-1643' 'P-1666' 'P-1656' '15314868-6' 'P-1717' 'P-1663' '78540190-5'
 '5555555-9' 'P-1729' 'P-1636' 'P-OFICINA' 'P-ANALISIS PROYE' 'P-1557'
 'P-1591' '18800250-1' '13056802-5' '16411600-K' nan]
Dimensiones del dataframe antes de 

##### 3.1.6.1 - Se Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [27]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor2_2 = df_libro_mayor2.copy()

# Filtrar el DataFrame
df_libro_mayor2_2 = df_libro_mayor2_2[(df_libro_mayor2_2['Codigo_proyecto'] == 'P-OFICINA') & (df_libro_mayor2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor2_2

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta
7631,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2153838,0,NaN,2153838,0,-2153838,COSTO SUPERVISIÓN
7632,P-OFICINA,4150002-002,2023,1,2023-01-31T00:00:00-03:00,AJC,1606,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7633,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN
7634,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7635,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292504,0,NaN,2292504,0,-2292504,COSTO SUPERVISIÓN
7636,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN
7637,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2533436,0,NaN,2533436,0,-2533436,COSTO SUPERVISIÓN
7638,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN
7639,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,2292357,0,NaN,2292357,0,-2292357,COSTO SUPERVISIÓN
7640,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2350904,0,NaN,2350904,0,-2350904,COSTO SUPERVISIÓN


In [28]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor2 = df_libro_mayor2.dropna(subset=['Codigo_proyecto'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask1 = df_libro_mayor2['Codigo_proyecto'].isin(valores_a_eliminar)

# Crea una máscara booleana que identifica las filas que comienzan con un número
mask2 = df_libro_mayor2['Codigo_proyecto'].str[0].str.isnumeric()

# Une las dos máscaras con el operador OR
mask = mask1 | mask2

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor2 = df_libro_mayor2[~mask]


# Verifica si el valor 'P1718' o valore similares están presentes en la columna Codigo_proyecto
if 'P1718' in df_libro_mayor2['Codigo_proyecto'].values:
    # Reemplaza el valor 'P1718' por 'P-1718' en la columna Codigo_proyecto
    df_libro_mayor2['Codigo_proyecto'] = df_libro_mayor2['Codigo_proyecto'].replace('P1718', 'P-1718')


In [29]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor2["Codigo_proyecto"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor2.shape)

Proyectos disponibles antes de la reducción : 
['P-1677' 'P-1700' 'P-1704' 'P-1716' 'P-1709' 'P-1687' 'P-1690' 'P-1686'
 'P-1688' 'P-1682' 'P-1694' 'P-1695' 'P-1703' 'P-1680' 'P-1697' 'P-1714'
 'P-1696' 'P-1722' 'P-1692' 'P-1718' 'P-1721' 'P-1715' 'P-1645' 'P-1699'
 'P-1701' 'P-1675' 'P-1684' 'P-1727' 'P-1723' 'P-1725' 'P-1672' 'P-1713'
 'P-1706' 'P-1726' 'P-1707' 'P-1724' 'P-1712' 'P-1705' 'P-1670' 'P-1691'
 'P-1720' 'P-1719' 'P-1681' 'P-1698' 'P-1708' 'P-1669' 'P-1667' 'P-1668'
 'P-1641' 'P-1693' 'P-1676' 'P-1689' 'P-1664' 'P-1658' 'P-1660' 'P-1644'
 'P-1592' 'P-1648' 'P-1611' 'P-1679' 'P-1642' 'P-1653' 'P-1673' 'P-1683'
 'P-1661' 'P-1612' 'P-1728' 'P-1639' 'P-1674' 'P-1620' 'P-1603' 'P-1626'
 'P-1649' 'P-1685' 'P-1632' 'P-1643' 'P-1666' 'P-1656' 'P-1717' 'P-1663'
 'P-1729' 'P-1636' 'P-1557' 'P-1591']
Dimensiones del dataframe antes de la reducción : 
(7431, 17)


#### 3.1.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [30]:
df_libro_mayor2['Fecha2'] = pd.to_datetime(df_libro_mayor2['Año'].astype(str) + '-' + df_libro_mayor2['Mes'].astype(str) + '-1')
df_libro_mayor2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
5216,P-1727,4101010-000,2023,5,2023-05-08T00:00:00-03:00,CIN,200,Sirena con estrobo direccionable Simplex,P-1727,P07152141,1075000,0,50956,1075000,0,-1075000,COSTO MATERIALES,2023-05-01
2857,P-1699,4101010-000,2023,3,2023-03-09T00:00:00-03:00,CIN,285,Cable UTP CAT5E unifilar interior Trimer,P-1699,T14380059,45750,0,49800,45750,0,-45750,COSTO MATERIALES,2023-03-01
818,P-1709,4101010-000,2023,1,2023-01-03T00:00:00-03:00,CIN,42,Teclado Bosch B915 Basic Keypads,P-1709,P07130977,70074,0,48691,70074,0,-70074,COSTO MATERIALES,2023-01-01
2031,P-1690,4101010-000,2023,2,2023-02-09T00:00:00-03:00,CIN,228,"Disco corte 4 1/2"" metal",P-1690,I17440838,3024,0,49379,3024,0,-3024,COSTO MATERIALES,2023-02-01
591,P-1706,3401002-000,2023,3,NaN,AJC,1656,Ajuste por anticipos y retenciones,P-1706,P-1706 California/79877450-6 CONST,0,2507470,1,0,2507470,2507470,INGRESO,2023-03-01


##### 3.1.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [31]:
df_libro_mayor2_2['Fecha2'] = pd.to_datetime(df_libro_mayor2_2['Año'].astype(str) + '-' + df_libro_mayor2_2['Mes'].astype(str) + '-1')
df_libro_mayor2_2.sample(5)

,Codigo_proyecto,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Debito,Credito,Num_Referencia,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
7634,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-02-01
7636,P-OFICINA,4150002-002,2023,3,2023-03-31T00:00:00-03:00,AJC,1694,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2182368,0,NaN,2182368,0,-2182368,COSTO SUPERVISIÓN,2023-03-01
7638,P-OFICINA,4150002-002,2023,4,2023-04-30T00:00:00-03:00,AJC,1681,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2562685,0,NaN,2562685,0,-2562685,COSTO SUPERVISIÓN,2023-04-01
7633,P-OFICINA,4150002-002,2023,2,2023-02-28T00:00:00-03:00,AJC,1173,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,1833838,0,NaN,1833838,0,-1833838,COSTO SUPERVISIÓN,2023-02-01
7640,P-OFICINA,4150002-002,2023,5,2023-05-31T00:00:00-04:00,AJC,1844,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,2350904,0,NaN,2350904,0,-2350904,COSTO SUPERVISIÓN,2023-05-01


#### 3.1.8 - Se realiza pivote final del dataframe

In [32]:
# Se exploran las columnas:
df_libro_mayor2.columns

Index(['Codigo_proyecto', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Debito',
       'Credito', 'Num_Referencia', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object')

In [33]:
# crear pivot table sin margenes

df_libro_mayor2_piv = pd.pivot_table(df_libro_mayor2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_piv = pd.DataFrame(df_libro_mayor2_piv.to_records())

df_libro_mayor2_piv.sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
233,P-1695,INGRESO,2023-04-01 00:00:00,2023,4,6929341,0,6929341
182,P-1691,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,16200,990736,-974536
513,P-1724,INGRESO,2023-03-01 00:00:00,2023,3,15294733,0,15294733
331,P-1704,INGRESO,2023-05-01 00:00:00,2023,5,132309997,53250000,79059997
255,P-1697,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,494100,7128724,-6634624


In [34]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor2_piv.shape

(552, 8)

##### 3.1.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [35]:
# crear pivot table sin margenes

df_libro_mayor2_2_piv = pd.pivot_table(df_libro_mayor2_2, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor2_2_piv = pd.DataFrame(df_libro_mayor2_2_piv.to_records())

df_libro_mayor2_2_piv = df_libro_mayor2_2_piv[df_libro_mayor2_2_piv['Codigo_proyecto'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor2_2_piv['Codigo_proyecto'] = df_libro_mayor2_2_piv['Codigo_proyecto'].replace('P-OFICINA', 'TOTAL')


df_libro_mayor2_2_piv

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023,1,0,4336206,-4336206
1,TOTAL,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023,2,0,4016206,-4016206
2,TOTAL,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023,3,0,4474872,-4474872
3,TOTAL,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023,4,0,5096121,-5096121
4,TOTAL,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023,5,0,4643261,-4643261


#### 3.1.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta

In [36]:
df_libro_mayor_piv2 = pd.pivot_table(df_libro_mayor2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_piv2 = pd.DataFrame(df_libro_mayor_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_piv2.insert(loc=0, column='Codigo_proyecto', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_piv2 = df_libro_mayor_piv2[df_libro_mayor_piv2['Tipo_cuenta'] != '']

df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
2,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
3,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
4,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
5,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,114373715,-114373715
6,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
7,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
8,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88079320,-85314908
9,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,150098697,-146851453
10,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,178313193,-169779486


##### 3.1.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [37]:
df_libro_mayor_piv2 = pd.concat([df_libro_mayor_piv2, df_libro_mayor2_2_piv], ignore_index=True)
df_libro_mayor_piv2

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023,2,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023,3,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023,4,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023,5,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023,1,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023,2,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,2764412,88079320,-85314908
8,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023,4,3247244,150098697,-146851453
9,TOTAL,COSTO MATERIALES,2023-05-01 00:00:00,2023,5,8533707,178313193,-169779486


#### 3.1.10 - Se concatenan los 2 dataframes resultantes

In [38]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor2_piv.shape)

Dimensiones del dataframe antes de la concatenación : 
(552, 8)


In [39]:
df_libro_mayor3 = pd.concat([df_libro_mayor2_piv, df_libro_mayor_piv2])
df_libro_mayor3.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
248,P-1697,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023,1,0,192133,-192133
326,P-1704,COSTO MATERIALES,2023-06-01 00:00:00,2023,6,0,16121702,-16121702
291,P-1700,INGRESO,2023-01-01 00:00:00,2023,1,7013074,6243740,769334
192,P-1692,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,361088,4526604,-4165516
113,P-1683,COSTO MATERIALES,2023-03-01 00:00:00,2023,3,31328,0,31328


In [40]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe después de la concatenación : 
(574, 8)


#### 3.1.11 - Se crean las filas que permiten tener siempre todas las opciones de Tipo_Cuenta por cada Codigo_proyecto

In [41]:
combinaciones = []
for proyecto in df_libro_mayor3['Codigo_proyecto'].unique():
    for fecha in df_libro_mayor3['Fecha2'].unique():
        mes = pd.to_datetime(fecha).month
        año = pd.to_datetime(fecha).year
        for cuenta in ['INGRESO', 'COSTO MATERIALES', 'COSTO MANO DE OBRA', 'COSTO SUPERVISIÓN']:
            combinaciones.append((proyecto, fecha, mes, año, cuenta))

# crear un nuevo DataFrame con las combinaciones obtenidas
combinaciones_df = pd.DataFrame(combinaciones, columns=['Codigo_proyecto', 'Fecha2', 'Mes', 'Año', 'Tipo_cuenta'])

# agregar las demás columnas con valores nulos
combinaciones_df['Credito_2'] = 0
combinaciones_df['Debito_2'] = 0
combinaciones_df['TOTAL'] = 0

# Se eliminan filas con fecha en blanco.
combinaciones_df = combinaciones_df.dropna(subset=['Mes'])

combinaciones_df.sample(10)

,Codigo_proyecto,Fecha2,Mes,Año,Tipo_cuenta,Credito_2,Debito_2,TOTAL
507,P-1653,2023-04-01 00:00:00,4.00,2023.00,COSTO SUPERVISIÓN,0,0,0
632,P-1661,2023-03-01 00:00:00,3.00,2023.00,INGRESO,0,0,0
930,P-1675,2023-05-01 00:00:00,5.00,2023.00,COSTO MANO DE OBRA,0,0,0
635,P-1661,2023-03-01 00:00:00,3.00,2023.00,COSTO SUPERVISIÓN,0,0,0
1922,P-1714,2023-03-01 00:00:00,3.00,2023.00,COSTO MANO DE OBRA,0,0,0
1546,P-1698,2023-05-01 00:00:00,5.00,2023.00,COSTO MANO DE OBRA,0,0,0
2135,P-1722,2023-05-01 00:00:00,5.00,2023.00,COSTO SUPERVISIÓN,0,0,0
1133,P-1683,2023-02-01 00:00:00,2.00,2023.00,COSTO MATERIALES,0,0,0
2260,P-1726,2023-06-01 00:00:00,6.00,2023.00,INGRESO,0,0,0
254,P-1636,2023-04-01 00:00:00,4.00,2023.00,COSTO MANO DE OBRA,0,0,0


#### 3.1.12 - Se concatenan los 2 dataframes resultantes

In [42]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor3.shape)

Dimensiones del dataframe antes de la concatenación : 
(574, 8)


In [43]:
df_libro_mayor4 = pd.concat([df_libro_mayor3, combinaciones_df])
df_libro_mayor4.sort_values(by=['Codigo_proyecto']).sample(5)

,Codigo_proyecto,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1082,P-1681,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,0,0
1502,P-1696,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,0,0
2019,P-1718,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0
218,P-1694,INGRESO,2023-02-01 00:00:00,2023,2,7211400,0,7211400
1448,P-1694,INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0


In [44]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor4.shape)

Dimensiones del dataframe después de la concatenación : 
(2614, 8)


#### 3.1.13 - Estandarización de campos 

In [45]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor4 = df_libro_mayor4.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor4['AÑO'] = pd.to_numeric(df_libro_mayor4['AÑO'], downcast='integer')
df_libro_mayor4['MES'] = pd.to_numeric(df_libro_mayor4['MES'], downcast='integer')

df_libro_mayor4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
2123,P-1721,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0
769,P-1668,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0
33,P-1591,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0
629,P-1661,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0
275,P-1636,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0
2027,P-1718,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0
57,P-1670,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,6945,0,6945
2315,P-1728,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0
861,P-1672,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0
1137,P-1683,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0


In [46]:
#df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor4[df_libro_mayor4["CODIGO_PROYECTO"]=="TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
551,TOTAL,,,NaN,NaN,2281381657,1909051349,372330308
0,TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994
1,TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544
2,TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244
3,TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468
4,TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,114373715,-114373715
5,TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040
6,TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234
7,TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88079320,-85314908
8,TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,150098697,-146851453


#### 3.1.14 Creación de columna MARGEN

In [47]:
df_libro_mayor4_2 = df_libro_mayor4.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_libro_mayor4_2.groupby(['CODIGO_PROYECTO','MES']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

    
df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(5)

,CODIGO_PROYECTO,MES,MARGEN
65,P-1639,6.00,0.00
505,TOTAL,2.00,0.00
240,P-1683,1.00,0.00
40,P-1620,5.00,0.00
3,P-1557,4.00,0.00


In [48]:
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  : ", df_libro_mayor4.shape)
print("=======================================================================")
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          : ", df_margen.shape)
print(df_margen.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor4_2  :  (2614, 8)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_margen          :  (510, 3)
(510, 3)


In [49]:
# CONCATENACIÓN DE DATAFRAMES

df_libro_mayor5 = pd.merge(df_libro_mayor4, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor5['CODIGO_PROYECTO'] = df_libro_mayor5['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  : ", df_libro_mayor5.shape)
print(df_libro_mayor5.shape)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5  :  (2614, 9)
(2614, 9)


In [50]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
551,1. TOTAL,,,NaN,NaN,2281381657,1909051349,372330308,NaN
552,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,0.00
553,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,0.00
554,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,0.00
555,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,0.00
556,1. TOTAL,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,114373715,-114373715,0.00
557,1. TOTAL,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,2565889,125972929,-123407040,0.00
558,1. TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,3348963,83658197,-80309234,0.00
559,1. TOTAL,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,2764412,88079320,-85314908,0.00
560,1. TOTAL,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,3247244,150098697,-146851453,0.00


In [51]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
316,P-1704,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,7375493,-7375493,0.00
317,P-1704,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,10751091,-10751091,0.00
318,P-1704,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,12332472,-12332472,0.00
319,P-1704,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,14242535,-14242535,0.00
320,P-1704,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,12890883,-12890883,0.00
321,P-1704,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,20073319,-20073319,0.00
322,P-1704,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,139976,3388760,-3248784,0.00
323,P-1704,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,14768626,-14768626,0.00
324,P-1704,COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,20031042,-20031042,0.00
325,P-1704,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,13967551,-13967551,0.00


In [52]:
df_libro_mayor5["CODIGO_PROYECTO"].unique()

array(['P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611', 'P-1612',
       'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639', 'P-1641',
       'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648', 'P-1649',
       'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661', 'P-1663',
       'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669', 'P-1670',
       'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676', 'P-1677',
       'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683', 'P-1684',
       'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689', 'P-1690',
       'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695', 'P-1696',
       'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701', 'P-1703',
       'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708', 'P-1709',
       'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716', 'P-1717',
       'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722', 'P-1723',
       'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728', 'P-1729',
       '1. TOTAL'], dtype=object)

In [53]:
df_libro_mayor5.to_csv('archivos_respaldo/df_libro_mayor5.csv', index = False)

### 3.1 - TRATAMIENTO 1 - LIBRO MAYOR SUBSETS

#### 3.1.16 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO"**

In [54]:
fecha_actual = datetime.date.today()
fecha_actual

datetime.date(2023, 6, 11)

In [55]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd = df_libro_mayor5.copy()
df_libro_mayor_ytd.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1477,P-1680,COSTO SUPERVISIÓN,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
643,P-1592,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
972,P-1648,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
1788,P-1693,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
187,P-1692,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,6370434,-6370434,0.00


In [56]:
# Se crea un filtro para obtener todos aquellos que tengan valores hasta el MES anterior al mes en curso

# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Filtrar los registros del dataframe que corresponden al mes actual y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month < mes_actual]

df_libro_mayor_ytd_closed.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2526,P-1727,INGRESO,2023-01-01,2023.00,1.00,0,0,0,0.00
354,P-1706,INGRESO,2023-03-01,2023.00,3.00,10029880,0,10029880,0.00
1260,P-1669,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
279,P-1699,INGRESO,2023-03-01,2023.00,3.00,20833008,0,20833008,0.00
1274,P-1670,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00
1692,P-1689,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
1157,P-1664,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00
1937,P-1699,COSTO SUPERVISIÓN,2023-03-01,2023.00,3.00,0,0,0,0.00
1755,P-1692,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00
2307,P-1718,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00


In [57]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_piv = pd.pivot_table(df_libro_mayor_ytd_closed, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)
#df_libro_mayor4_piv = pd.pivot_table(df_libro_mayor4, values=['Debito_2','Credito_2','TOTAL'], index=['Codigo_proyecto','Tipo_cuenta'], columns = ['Fecha2'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')

df_libro_mayor_ytd_closed_piv = pd.DataFrame(df_libro_mayor_ytd_closed_piv.to_records())

df_libro_mayor_ytd_closed_piv.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
19,P-1603,INGRESO,0
324,P-1726,COSTO MANO DE OBRA,-8656540
317,P-1724,COSTO MATERIALES,-7710229
56,P-1643,COSTO MANO DE OBRA,0
159,P-1681,INGRESO,765714
186,P-1688,COSTO SUPERVISIÓN,0
228,P-1699,COSTO MANO DE OBRA,-21466214
303,P-1720,INGRESO,0
280,P-1715,COSTO MANO DE OBRA,-16935839
118,P-1669,COSTO SUPERVISIÓN,0


#### 3.1.16-1 - CREACIÓN DF **"YTD AÑO EN CURSO HASTA MES CERRADO 2"**
Este dataframe es para considerar como mes cerrado 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [58]:
# Obtener el mes actual
mes_actual = datetime.date.today().month
print(mes_actual)

# Calcular el mes que es dos meses antes del actual
mes_filtrado = mes_actual - 2 if mes_actual > 2 else (12 + mes_actual - 2) 

# Filtrar los registros del dataframe que corresponden al mes filtrado y a todos los meses anteriores
df_libro_mayor_ytd["FECHA"] = pd.to_datetime(df_libro_mayor_ytd["FECHA"])
df_libro_mayor_ytd_closed_2M = df_libro_mayor_ytd[df_libro_mayor_ytd["FECHA"].dt.month <= mes_filtrado]

df_libro_mayor_ytd_closed_2M.sample(10)

6


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1548,P-1683,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,0,0,0.00
417,P-1714,COSTO MANO DE OBRA,2023-02-01,2023.00,2.00,0,3151491,-3151491,0.00
492,P-1721,INGRESO,2023-01-01,2023.00,1.00,21932816,14369776,7563040,0.00
1143,P-1663,COSTO MATERIALES,2023-03-01,2023.00,3.00,0,0,0,0.00
1775,P-1693,COSTO MATERIALES,2023-04-01,2023.00,4.00,0,0,0,0.00
2365,P-1720,COSTO SUPERVISIÓN,2023-02-01,2023.00,2.00,0,0,0,0.00
220,P-1694,INGRESO,2023-04-01,2023.00,4.00,12582467,0,12582467,0.00
1857,P-1696,COSTO SUPERVISIÓN,2023-01-01,2023.00,1.00,0,0,0,0.00
60,P-1672,COSTO MATERIALES,2023-01-01,2023.00,1.00,15000,0,15000,0.00
218,P-1694,INGRESO,2023-02-01,2023.00,2.00,7211400,0,7211400,0.00


In [59]:
# crear pivot table sin margenes

df_libro_mayor_ytd_closed_2M = pd.pivot_table(df_libro_mayor_ytd_closed_2M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd_closed_2M = pd.DataFrame(df_libro_mayor_ytd_closed_2M.to_records())

df_libro_mayor_ytd_closed_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
106,P-1666,COSTO SUPERVISIÓN,0
176,P-1686,COSTO MANO DE OBRA,-12899988
270,P-1712,COSTO SUPERVISIÓN,0
2,1. TOTAL,COSTO SUPERVISIÓN,-17923405
333,P-1728,COSTO MATERIALES,-1822779


#### 3.1.17 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO**

In [60]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.
 
df_libro_mayor5_mec = df_libro_mayor5.copy()
df_libro_mayor5_mec.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  : ", df_libro_mayor5_mec.shape)
df_libro_mayor5_mec.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec  :  (2614, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
2199,P-1713,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
2570,P-1729,INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
912,P-1644,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00


In [61]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
392,P-1712,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,5436047,-5436047,0.00,LIBRO MAYOR
1479,P-1680,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00,LIBRO MAYOR
1113,P-1661,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR
1634,P-1687,INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1879,P-1697,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR


In [62]:
# Se seleccionan los meses que son utiles para crear la vista de mes en curso.
# Para el real se necesita: INGRESO, COSTO MATERIALES, Y SUPERVISIÓN (ÉSTE ÚLTIMO DEL MES ANTERIOR)
# Para el proyectado se necesita: SUPERVISIÓN (DEL MES ANTERIOR)

In [63]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta


df_libro_mayor5_mec['FECHA'] = pd.to_datetime(df_libro_mayor5_mec['FECHA'])

filtro_ingreso_materiales = ((df_libro_mayor5_mec['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec['FECHA'].dt.month == datetime.now().month)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_supervision_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior = (datetime.now() - timedelta(days=32)).month
filtro_mo_mes_anterior = (df_libro_mayor5_mec['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec['FECHA'].dt.month == mes_anterior)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec2 = df_libro_mayor5_mec[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  : ", df_libro_mayor5_mec2.shape)

df_libro_mayor5_mec2.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec2  :  (419, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1794,P-1693,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
954,P-1645,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
1228,P-1668,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1156,P-1664,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1012,P-1653,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
137,P-1686,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,1631758,-1631758,0.00,LIBRO MAYOR
1075,P-1658,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2476,P-1725,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1098,P-1660,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
92,P-1680,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,6044211,-6044211,0.00,LIBRO MAYOR


In [64]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
320,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,12890883,-12890883,0.00,LIBRO MAYOR
326,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,16121702,-16121702,0.00,LIBRO MAYOR
2020,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2021,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2034,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2035,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [65]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec2[df_libro_mayor5_mec2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
556,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,114373715,-114373715,0.00,LIBRO MAYOR
562,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,4092922,122363761,-118270839,0.00,LIBRO MAYOR
568,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,53939883,58891059,-4951176,0.00,LIBRO MAYOR
573,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,4643261,-4643261,0.00,LIBRO MAYOR
2596,1. TOTAL,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2597,1. TOTAL,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2610,1. TOTAL,INGRESO,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
2611,1. TOTAL,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.17-1 - CREACIÓN DF **LIBRO MAYOR PARA MES EN CURSO 2**
Este dataframe es para considerar como mes en curso de 2 meses atrás, de esta forma se puede evaluar el mes cerrado.

In [66]:
# Se crea un nuevo dataframe basado en df_libro_mayor5, la palabra "mec" significa mes en curso.

df_libro_mayor5_mec_2M = df_libro_mayor5.copy()
df_libro_mayor5_mec_2M.shape
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M.sample(3)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (2614, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1640,P-1687,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
2242,P-1715,INGRESO,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
1380,P-1675,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00


In [67]:
# Se crea una nueva columna para categorizar el origen de los datos.

df_libro_mayor5_mec_2M["ORIGEN"]="LIBRO MAYOR"
df_libro_mayor5_mec_2M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
2061,P-1705,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR
46,P-1667,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,1950257,-1950257,0.00,LIBRO MAYOR
1826,P-1695,INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2551,P-1728,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00,LIBRO MAYOR
2277,P-1716,COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00,LIBRO MAYOR


In [68]:
# Filtro para TIPO_CUENTA y FECHA
from datetime import datetime, timedelta

df_libro_mayor5_mec_2M['FECHA'] = pd.to_datetime(df_libro_mayor5_mec_2M['FECHA'])

# Filtro para INGRESO Y COSTO DE MATERIALES
mes_anterior = (datetime.now() - timedelta(days=30)).month

filtro_ingreso_materiales = ((df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'INGRESO') | 
                             (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MATERIALES')) & \
                            (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior)

# Filtro para SUPERVISIÓN del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_supervision_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO SUPERVISIÓN') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)


# Filtro para COSTO_MANO_OBRA del mes anterior
mes_anterior2 = (datetime.now() - timedelta(days=60)).month

filtro_mo_mes_anterior = (df_libro_mayor5_mec_2M['TIPO_CUENTA'] == 'COSTO MANO DE OBRA') & \
                                  (df_libro_mayor5_mec_2M['FECHA'].dt.month == mes_anterior2)

# Combinar los filtros en un filtro general
filtro_general = filtro_ingreso_materiales | filtro_supervision_mes_anterior  | filtro_mo_mes_anterior

# Crear el sub-dataframe filtrado
df_libro_mayor5_mec_2M = df_libro_mayor5_mec_2M[filtro_general]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)

df_libro_mayor5_mec_2M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
1251,P-1669,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
795,P-1636,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2497,P-1726,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
1371,P-1675,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
1801,P-1694,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR


In [69]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
319,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,14242535,-14242535,0.00,LIBRO MAYOR
325,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,13967551,-13967551,0.00,LIBRO MAYOR
331,P-1704,INGRESO,2023-05-01,2023.00,5.00,132309997,53250000,79059997,0.00,LIBRO MAYOR
2016,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2017,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2018,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2019,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


In [70]:
# INSPECCIÓN VISUAL 2
df_libro_mayor5_mec_2M[df_libro_mayor5_mec_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN,ORIGEN
555,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,111885468,-111885468,0.00,LIBRO MAYOR
561,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,8533707,178313193,-169779486,0.00,LIBRO MAYOR
567,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,631496823,170698310,460798513,0.00,LIBRO MAYOR
572,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,5096121,-5096121,0.00,LIBRO MAYOR
2592,1. TOTAL,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2593,1. TOTAL,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,0,0,0.00,LIBRO MAYOR
2594,1. TOTAL,INGRESO,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR
2595,1. TOTAL,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,0,0,0.00,LIBRO MAYOR


#### 3.1.18 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual)**

In [71]:
# Se crea una copia del dataframe 

df_libro_mayor_ytd2 = df_libro_mayor5.copy()
df_libro_mayor_ytd2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
667,P-1603,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,0.00
1544,P-1683,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
139,P-1686,INGRESO,2023-03-01 00:00:00,2023.00,3.00,8723656,0,8723656,0.00
2092,P-1707,COSTO MANO DE OBRA,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
1193,P-1666,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
2603,1. TOTAL,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00
50,P-1668,COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,68000,0,68000,0.00
1809,P-1694,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
1502,P-1681,INGRESO,2023-03-01 00:00:00,2023.00,3.00,0,0,0,0.00
1153,P-1664,COSTO SUPERVISIÓN,2023-04-01 00:00:00,2023.00,4.00,0,0,0,0.00


In [72]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv = pd.pivot_table(df_libro_mayor_ytd2, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv = pd.DataFrame(df_libro_mayor_ytd2_closed_piv.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  : ", df_libro_mayor_ytd2_closed_piv.shape)

df_libro_mayor_ytd2_closed_piv.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv  :  (341, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
266,P-1709,COSTO MATERIALES,-26491924
272,P-1712,INGRESO,28945644
73,P-1649,COSTO MANO DE OBRA,0
177,P-1686,COSTO MANO DE OBRA,-16633780
232,P-1699,INGRESO,94757672
119,P-1669,COSTO SUPERVISIÓN,0
250,P-1705,COSTO MATERIALES,-15324171
132,P-1673,INGRESO,0
99,P-1663,COSTO SUPERVISIÓN,0
4,1. TOTAL,INGRESO,1574411233


#### 3.1.18-2 - CREACIÓN DF **LIBRO MAYOR PARA YTD MES PROYECTADO (Incluyendo mes actual) 2**

In [73]:
# Obtener el mes actual
mes_actual = datetime.now().month

# Filtrar los registros hasta el mes anterior al mes actual
df_libro_mayor_ytd2_1M = df_libro_mayor_ytd2[df_libro_mayor_ytd2['MES'] < mes_actual]

# Mostrar el nuevo DataFrame
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  : ", df_libro_mayor_ytd2_1M.shape)
df_libro_mayor_ytd2_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_1M  :  (2230, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
987,P-1649,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
136,P-1686,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,45824,2314292,-2268468,0.00
1901,P-1698,COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,0.00
2120,P-1708,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,0.00
636,P-1592,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,0,0,0.00


In [74]:
# crear pivot table sin margenes

df_libro_mayor_ytd2_closed_piv_1M = pd.pivot_table(df_libro_mayor_ytd2_1M, values=['TOTAL'], index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum', fill_value=0)

df_libro_mayor_ytd2_closed_piv_1M = pd.DataFrame(df_libro_mayor_ytd2_closed_piv_1M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  : ", df_libro_mayor_ytd2_closed_piv_1M.shape)

df_libro_mayor_ytd2_closed_piv_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor_ytd2_closed_piv_1M  :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
118,P-1669,COSTO SUPERVISIÓN,0
75,P-1649,INGRESO,0
137,P-1675,COSTO MATERIALES,0
93,P-1661,COSTO MATERIALES,3200
272,P-1713,COSTO MANO DE OBRA,-4283233
41,P-1636,COSTO MATERIALES,0
312,P-1723,COSTO MANO DE OBRA,-25834725
63,P-1644,INGRESO,0
121,P-1670,COSTO MATERIALES,6945
228,P-1699,COSTO MANO DE OBRA,-21466214


#### 3.1.19 - CREACIÓN DE DATAFRAME BASE LIBRO MAYOR 2023 PARA EL CRUCE OTROS DATAFRAMES

##### NOTA IMPORTANTE
##### Se considera que todos los dataframes que pintan los dashboards deben tener como base los proyectos abiertos en el Libro Mayor 2023. Para esto se crea el dataframe base con el cual debe comenzar cualquier cruce de datos y que considera el código del proyecto, todos los tipos de cuenta por cada proyecto y con valor en cero, además se cruza con la tabla codigo de proyecto.

In [75]:
df_libro_mayor5_base = df_libro_mayor5.copy()

In [76]:
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base = df_libro_mayor5_base[['CODIGO_PROYECTO','TIPO_CUENTA']].drop_duplicates()

# Se elimina fila de TOTAL
df_libro_mayor5_base = df_libro_mayor5_base[df_libro_mayor5_base['CODIGO_PROYECTO'] != '1. TOTAL']

# Se Reemplazan valores de la column TIPO_CUENTA
df_libro_mayor5_base['TIPO_CUENTA'] = df_libro_mayor5_base['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor5_base['TOTAL_LB']= 0

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(10)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor5_base   :  (2614, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (336, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
598,P-1591,1. INGRESO,0
2351,P-1720,2. COSTO MATERIALES,0
49,P-1668,2. COSTO MATERIALES,0
2209,P-1714,4. COSTO SUPERVISIÓN,0
718,P-1620,1. INGRESO,0
93,P-1680,1. INGRESO,0
888,P-1643,3. COSTO MANO DE OBRA,0
2327,P-1719,2. COSTO MATERIALES,0
720,P-1620,3. COSTO MANO DE OBRA,0
2017,P-1704,4. COSTO SUPERVISIÓN,0


In [77]:
# Se Agrega COSTO DE SUPERVISIÓN

nueva_fila = pd.DataFrame({'CODIGO_PROYECTO': ['1. TOTAL'],
                           'TIPO_CUENTA': ['4. COSTO SUPERVISIÓN'],
                           'TOTAL_LB': [0]})

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, nueva_fila], ignore_index=True)

df_libro_mayor5_base[df_libro_mayor5_base["CODIGO_PROYECTO"]=="1. TOTAL"]
print(df_libro_mayor5_base["CODIGO_PROYECTO"].unique())
print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

['P-1557' 'P-1591' 'P-1592' 'P-1603' 'P-1611' 'P-1612' 'P-1620' 'P-1626'
 'P-1632' 'P-1636' 'P-1639' 'P-1641' 'P-1642' 'P-1643' 'P-1644' 'P-1645'
 'P-1648' 'P-1649' 'P-1653' 'P-1656' 'P-1658' 'P-1660' 'P-1661' 'P-1663'
 'P-1664' 'P-1666' 'P-1667' 'P-1668' 'P-1669' 'P-1670' 'P-1672' 'P-1673'
 'P-1674' 'P-1675' 'P-1676' 'P-1677' 'P-1679' 'P-1680' 'P-1681' 'P-1682'
 'P-1683' 'P-1684' 'P-1685' 'P-1686' 'P-1687' 'P-1688' 'P-1689' 'P-1690'
 'P-1691' 'P-1692' 'P-1693' 'P-1694' 'P-1695' 'P-1696' 'P-1697' 'P-1698'
 'P-1699' 'P-1700' 'P-1701' 'P-1703' 'P-1704' 'P-1705' 'P-1706' 'P-1707'
 'P-1708' 'P-1709' 'P-1712' 'P-1713' 'P-1714' 'P-1715' 'P-1716' 'P-1717'
 'P-1718' 'P-1719' 'P-1720' 'P-1721' 'P-1722' 'P-1723' 'P-1724' 'P-1725'
 'P-1726' 'P-1727' 'P-1728' 'P-1729' '1. TOTAL']
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)


In [78]:
# Crear DataFrame con TIPO_CUENTA y valor cero para cada combinación única

df_nuevo = pd.DataFrame({
    'CODIGO_PROYECTO': df_libro_mayor5_base['CODIGO_PROYECTO'].repeat(4),
    'TIPO_CUENTA': ['1. INGRESO', '2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'] * len(df_libro_mayor5_base),
    'TOTAL_LB': 0
})

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             : ", df_nuevo.shape)

df_libro_mayor5_base = pd.concat([df_libro_mayor5_base, df_nuevo], ignore_index=True)

# Se remueve cualquier duplicado.
df_libro_mayor5_base.drop_duplicates(inplace=True)

print("DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)
df_libro_mayor5_base.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base :  (337, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_nuevo             :  (1348, 3)
DIMENSIONES ANTERIOR DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
113,P-1706,2. COSTO MATERIALES,0
272,P-1681,4. COSTO SUPERVISIÓN,0
125,P-1712,2. COSTO MATERIALES,0
193,P-1632,3. COSTO MANO DE OBRA,0
264,P-1676,1. INGRESO,0


In [79]:
# INSPECCIÓN VISUAL 1
df_libro_mayor5_base.sort_values(by=['CODIGO_PROYECTO','TIPO_CUENTA']).head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB
1681,1. TOTAL,1. INGRESO,0
1682,1. TOTAL,2. COSTO MATERIALES,0
1683,1. TOTAL,3. COSTO MANO DE OBRA,0
336,1. TOTAL,4. COSTO SUPERVISIÓN,0
168,P-1557,1. INGRESO,0
169,P-1557,2. COSTO MATERIALES,0
0,P-1557,3. COSTO MANO DE OBRA,0
170,P-1557,4. COSTO SUPERVISIÓN,0
171,P-1591,1. INGRESO,0
172,P-1591,2. COSTO MATERIALES,0


In [80]:
# MERGE CON CODIGO_PROYECTO

# Se crea dataframe de codigos de proyecto y se aplica mayúscula
df_cod_proyecto_base = df_cod_proyecto.applymap(lambda s: s.upper() if isinstance(s, str) else s)

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2 = pd.merge(df_libro_mayor5_base, df_cod_proyecto_base, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base : ", df_libro_mayor5_base.shape)

df_libro_mayor5_base2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor5_base :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
221,P-1653,1. INGRESO,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.
287,P-1690,4. COSTO SUPERVISIÓN,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.
171,P-1591,1. INGRESO,0,EDIFICIO ESTORIL,EBCO S.A.
104,P-1703,2. COSTO MATERIALES,0,SANTA VICTORIA DANACORP,DANACORP S.A
228,P-1658,3. COSTO MANO DE OBRA,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.
261,P-1674,4. COSTO SUPERVISIÓN,0,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.
16,P-1645,1. INGRESO,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,NaN,NaN
157,P-1725,2. COSTO MATERIALES,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO
324,P-1722,3. COSTO MANO DE OBRA,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA


In [81]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"

df_libro_mayor5_base2.loc[df_libro_mayor5_base2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_libro_mayor5_base2.sample(10) 

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
231,P-1660,3. COSTO MANO DE OBRA,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.
59,P-1686,3. COSTO MANO DE OBRA,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.
235,P-1661,4. COSTO SUPERVISIÓN,0,SAN VICENTE DE PAUL,DANACORP S.A
210,P-1644,1. INGRESO,0,MIRADOR DE SAN CRISTOBAL,EBCO S.A.
279,P-1685,3. COSTO MANO DE OBRA,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.
1,P-1591,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EBCO S.A.
190,P-1626,3. COSTO MANO DE OBRA,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.
140,P-1718,3. COSTO MANO DE OBRA,0,PATIO COMIDAS,EBCO S.A.
91,P-1697,1. INGRESO,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A
303,P-1705,4. COSTO SUPERVISIÓN,0,LOS CLARINES,CONSTRUCTORA INARCO S.A.


In [82]:
# INSPECCIÓN VISUAL 1

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL


In [83]:
# INSPECCIÓN VISUAL 2

df_libro_mayor5_base2[df_libro_mayor5_base2['CODIGO_PROYECTO']=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.


### 3.2 - TRATAMIENTO DE DESPACHOS REALIZADOS

#### 3.2.1 - Creación de dataframe copia

In [84]:
df_desp_realizados2 = df_desp_realizados.copy()
df_desp_realizados2.head(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
0,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1033,P08202340,1,"Tee 4"" Ranurada A536",UN,6296,6296
1,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1026,P08201152,7,"Tee 3"" Ranurada UL/FM",UN,4190,29330
2,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1040,P08202739,1,"Tee Mecanica 4"" x 3"" ranurada A-536 UL/F",UN,4467,4467
3,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1040,P08202656,2,"Tee Mecanica 3"" x 1 1/4"" HI NPT",UN,3001,6002
4,P-1693,VCO,2022-03-24T00:00:00-03:00,44865,1070,P08200874,1,"Tee Mecanica 3"" x 1"" HI NPT",UN,2442,2442


#### 3.2.2 - Se eliminan proyectos relacionados a STT, P-Proyectos, P-Servicios, P-Mantención

In [85]:
df_desp_realizados2["COD_PROYECTO"].unique()

array(['P-1693', 'P-Servicios', 'P-1675', 'P-1672', 'P-1667', 'P-1698',
       'P-1685', 'P-Proyectos', nan, 'P-1691', 'P-1676', 'P-1669',
       'P-1677', 'P-1668', 'P-1684', 'P-1689', 'P-1683', 'P-1681',
       'P-1645', 'P-1700', 'P-Mantención', 'P-1704', 'P-1661', 'P-1690',
       'P-1615', 'P-1695', 'P-1682', 'P-1712', 'P-1696', 'P-1692',
       'P-1680', 'P-1688', 'P-1694', 'P-1707', 'P-1701', 'P-1686',
       'P-1648', 'P-1703', 'P-ANALISIS PROYE', 'P-1697', 'P-1670',
       'P-1639', 'P-1716', 'P-1718', 'P-1715', 'P-1702', 'P-PROYECTOS',
       'P-1721', 'P-1714', 'P-1623', 'P-1709', 'P-1625', 'P-1687',
       'P-1723', 'P-1705', 'P-1642', 'P-1724', 'P-1592', 'P-1699',
       'P-1713', 'P-1726', 'P-1664', 'P-1727', 'P-1620', 'P-1725',
       'P-1673', 'P-1652', 'P-1728', 'P-1638', 'P-1632', 'P-1658',
       'P-1626', 'P-1656', 'P-1637', 'P-1649', 'P-1629', 'P-1671',
       'P-1653', 'P-1636', 'P-1706', 'P-1708', 'P-1660', 'P-1663',
       'P-1666', 'P-1613', 'P-1678', 'P-1717',

In [86]:
valores_eliminar = ['STT', 'P-Proyectos', 'P-Servicios', 'P-Mantención','P-Oficina''P-ANALISIS PROYE','P-ANALISIS PROYE','P-PROYECTOS','P-Oficina','nan']

# Convertir los valores a eliminar y la columna 'PROYECTO' a mayúsculas
valores_eliminar_mayusculas = [valor.upper() for valor in valores_eliminar]
df_desp_realizados2['PROYECTO_MAYUS'] = df_desp_realizados2['COD_PROYECTO'].str.upper()

# Filtrar las filas que no contengan los valores a eliminar
df_desp_realizados3 = df_desp_realizados2[~df_desp_realizados2['PROYECTO_MAYUS'].isin(valores_eliminar_mayusculas)]

# Eliminar la columna auxiliar 'PROYECTO_MAYUS'
df_desp_realizados3 = df_desp_realizados3.drop(columns=['PROYECTO_MAYUS'])

df_desp_realizados3.sample(10)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL
19871,P-1684,VCO,2023-06-06T00:00:00-04:00,51569,1067,C02032948,16,MUFA FO PLANA HOR 48F H012,UN,30047,480752
9931,P-1693,VCO,2022-07-29T00:00:00-04:00,46516,1087,O19502485,2,Arriendos baño quimico,D,81400,162800
3841,P-1667,VCO,2022-10-14T00:00:00-03:00,47603,1034,I16430107,10,Cinta Aisladora 3M Temflex 1500,UN,399,3990
10546,P-1712,VCO,2022-08-19T00:00:00-03:00,46801,1075,P08182589,18,"Carrete Manguera 1"" Semi-Rigida 30 M",UN,99739,1795302
18380,P-1712,VCO,2023-06-05T00:00:00-04:00,51514,1137,S09241106,3,Disco duro SATA 4 TB Purple,UN,64154,192462
10101,P-1726,VCO,2023-04-21T00:00:00-03:00,50689,1026,P08200119,5,"Flange Manguito Ø6"" ranurado ANSI B.16.5",UN,14142,70710
7386,P-1692,VCO,2022-07-19T00:00:00-04:00,46339,1074,C06112541,200,Carga 75 ohm terminal F aislado,UN,310,62000
1435,P-1684,VCO,2022-05-20T00:00:00-04:00,45471,1036,S11290198,40,"Magnetico Embutido 1/4"" Blanco",UN,700,28000
10122,P-1707,VCO,2023-05-03T00:00:00-03:00,50825,1060,C06112991,1,TORRETA + MASTIL GALVANIZADA NARANJO 2.2,UN,195502,195502
14495,P-1680,VCO,2023-02-20T00:00:00-03:00,49510,1034,S11300681,12,Expansor 8 zonas ZX8 p/ EVO Paradox,UN,43586,523032


#### 3.2.2 - Se eliminan Tipos de movimientos diferentes a VCO y GDP

In [87]:
# Reemplazar los valores NaN en la columna "TIPO MOVIMIENTO" con una cadena vacía
df_desp_realizados3['TIPO_x0020_MOVIMIENTO'] = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].fillna('')

# Filtrar las filas donde "TIPO MOVIMIENTO" sea igual a "VCO" o "GDP" y excluir otros valores

mask = df_desp_realizados3['TIPO_x0020_MOVIMIENTO'].isin(['VCO', 'GDP'])
df_desp_realizados3 = df_desp_realizados3[mask]

In [88]:
# Se realiza la evaluación de dimensiones después de la eliminación de filas
print("\nDataframe Original antes de los cambios:")
print(df_desp_realizados.shape)
print("Dataframe después de eliminar registros:")
print(df_desp_realizados3.shape)

df_desp_realizados3.to_csv('df_desp_realizados3.csv', index = False)


Dataframe Original antes de los cambios:
(20204, 11)
Dataframe después de eliminar registros:
(12011, 11)


In [89]:
# Convertir la columna 'FECHA GUIA' a objetos de fecha y hora, manejando campos en blanco o no válidos como fechas
df_desp_realizados3['FECHA GUIA 2'] = pd.to_datetime(df_desp_realizados3['FECHA_x0020_GUIA'], errors='coerce')

# Extraer el MES y el AÑO en nuevas columnas utilizando funciones lambda y apply()
df_desp_realizados3['MES'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.month if pd.notnull(x) else None)
df_desp_realizados3['AÑO'] = df_desp_realizados3['FECHA GUIA 2'].apply(lambda x: x.year if pd.notnull(x) else None)

df_desp_realizados3.sample(5)

,COD_PROYECTO,TIPO_x0020_MOVIMIENTO,FECHA_x0020_GUIA,NUMERO_x0020_GUIA,PARTIDA_x0020_PRESUPUESTARIA,CODIGO_x0020_PARTE,CANTIDAD,DESCRIPCI_xFFFD_N_x0020_PRODUCTO,UNIDAD_x0020_DE_x0020_MEDIDA,COSTO_x0020_UNITARIO,COSTO_x0020_TOTAL,FECHA GUIA 2,MES,AÑO
16453,P-1648,GDP,2022-12-01T00:00:00-03:00,48264,1071,I17441125,3,Broca HSS metal 13mm,UN,5210,15630,2022-12-01 00:00:00-03:00,12,2022
12133,P-1700,GDP,2022-12-21T00:00:00-03:00,48530,1041,P08200316,4,"Union copla 2"" ranurada flexible",UN,2013,8052,2022-12-21 00:00:00-03:00,12,2022
19622,P-1703,VCO,2022-10-11T00:00:00-03:00,47539,1021,I16430107,4,Cinta Aisladora 3M Temflex 1500,UN,399,1596,2022-10-11 00:00:00-03:00,10,2022
4694,P-1693,VCO,2022-04-25T00:00:00-03:00,45094,1086,I15410604,50,"Tuerca hexagonal zincada 1/2""",UN,44,2200,2022-04-25 00:00:00-03:00,4,2022
14799,P-1667,VCO,2022-09-07T00:00:00-03:00,47116,1006,T14392553,3,Fuente 220VAC/12VDC 2A HK-AD,UN,2327,6981,2022-09-07 00:00:00-03:00,9,2022


#### 3.2.4 - Se realizan tablas pivot

In [90]:
df_desp_realizados3['COSTO_x0020_TOTAL'] = df_desp_realizados3['COSTO_x0020_TOTAL'].astype(int)
df_desp_realizados3['CANTIDAD'] = df_desp_realizados3['CANTIDAD'].astype(float)

In [91]:
df_desp_realizados3.columns

Index(['COD_PROYECTO', 'TIPO_x0020_MOVIMIENTO', 'FECHA_x0020_GUIA',
       'NUMERO_x0020_GUIA', 'PARTIDA_x0020_PRESUPUESTARIA',
       'CODIGO_x0020_PARTE', 'CANTIDAD', 'DESCRIPCI_xFFFD_N_x0020_PRODUCTO',
       'UNIDAD_x0020_DE_x0020_MEDIDA', 'COSTO_x0020_UNITARIO',
       'COSTO_x0020_TOTAL', 'FECHA GUIA 2', 'MES', 'AÑO'],
      dtype='object')

In [92]:
# Crear el pivot dataframe
df_desp_realizados4 = pd.pivot_table(df_desp_realizados3, 
                          values=['CANTIDAD', 'COSTO_x0020_TOTAL'], 
                          index=['COD_PROYECTO', 'AÑO', 'MES'], 
                          aggfunc='sum', 
                          columns='TIPO_x0020_MOVIMIENTO')

df_desp_realizados4 = pd.DataFrame(df_desp_realizados4.to_records())

df_desp_realizados4.sample(10)

,COD_PROYECTO,AÑO,MES,"('CANTIDAD', 'GDP')","('CANTIDAD', 'VCO')","('COSTO_x0020_TOTAL', 'GDP')","('COSTO_x0020_TOTAL', 'VCO')"
211,P-1677,2022,5,NaN,2393.00,NaN,2895617.00
80,P-1653,2022,6,4.00,NaN,5016.00,NaN
476,P-1704,2022,10,1.00,2177.00,23459.00,9795762.00
390,P-1695,2023,5,15.00,8976.00,981954.00,9619052.00
364,P-1693,2022,6,2.00,545.00,166000.00,1726423.00
5,P-1592,2023,2,NaN,1.00,NaN,102993.00
94,P-1658,2023,3,1.00,NaN,18401.00,NaN
43,P-1642,2022,8,20.00,20.00,220000.00,220000.00
90,P-1657,2022,8,1.00,NaN,6810.00,NaN
309,P-1688,2023,2,NaN,1279.00,NaN,2657700.00


#### 3.2.5 - Se renombran columnas 

In [93]:
df_desp_realizados4 = df_desp_realizados4.rename(columns={'COD_PROYECTO': 'CODIGO_PROYECTO'})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'GDP')": "CANT. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('CANTIDAD', 'VCO')": "CANT. VCO"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'GDP')": "TOTAL. GDP"})
df_desp_realizados4 = df_desp_realizados4.rename(columns={"('COSTO_x0020_TOTAL', 'VCO')": "TOTAL. VCO"})

df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO
267,P-1684,2023,3,400.00,1734.00,3200.00,717239.00
567,P-1723,2023,3,NaN,8181.00,NaN,2840434.00
538,P-1714,2023,4,NaN,630.00,NaN,3476784.00
172,P-1672,2022,6,300.00,1244.00,45000.00,1696060.00
324,P-1690,2022,5,3.00,5013.00,59100.00,2753224.00


#### 3.2.6 - Se reemplazas columnas NaN por ceros y se crea columna de TOTAL GPD-VCO

In [94]:
df_desp_realizados4.fillna(0, inplace=True)
df_desp_realizados4['TOTAL. VCO - GDP'] = df_desp_realizados4['TOTAL. VCO'] - df_desp_realizados4['TOTAL. GDP']
df_desp_realizados4.sample(5)

,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
420,P-1698,2022,11,4.00,1.00,13892.00,18479.00,4587.00
559,P-1718,2023,5,0.00,2655.00,0.00,3475005.00,3475005.00
563,P-1721,2023,3,0.00,2516.00,0.00,4616205.00,4616205.00
45,P-1642,2023,2,80.00,0.00,8880.00,0.00,-8880.00
25,P-1626,2022,11,0.00,1.00,0.00,45576.00,45576.00


#### 3.2.7 - Se seleccionan solamente los registros del mes y año en curso 

In [95]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

# Filtrar el dataframe por año y mes
df_desp_realizados5 = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == current_month)]
print("Dimensiones después del filtro", df_desp_realizados5.shape)
df_desp_realizados5.sample(5)

2023
6
Dimensiones antes del filtro (592, 8)
Dimensiones después del filtro (35, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
589,P-1729,2023,6,0.00,6.00,0.00,114402.00,114402.00
391,P-1695,2023,6,0.00,873.00,0.00,1647749.00,1647749.00
553,P-1717,2023,6,112.00,843.00,18388.00,205837.00,187449.00
382,P-1694,2023,6,0.00,12465.00,0.00,8095051.00,8095051.00
33,P-1636,2023,6,0.00,1.00,0.00,166850.00,166850.00


#### 3.2.7-2 - Se seleccionan solamente los registros del mes y año en curso 2
Se considera esta línea para poblar la vista del mes anterior.

In [96]:
# Obtener el año y mes actual
#now = datetime.datetime.now()

now = datetime.now()
current_year = now.year
print(current_year)
current_month = now.month
print(current_month)
print("Dimensiones antes del filtro", df_desp_realizados4.shape)

mes_anterior = (datetime.now() - timedelta(days=30)).month

# Filtrar el dataframe por año y mes
df_desp_realizados5_1M = df_desp_realizados4[(df_desp_realizados4['AÑO'] == current_year) & (df_desp_realizados4['MES'] == mes_anterior)]
print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)
df_desp_realizados5_1M.sample(5)

2023
6
Dimensiones antes del filtro (592, 8)
Dimensiones después del filtro (46, 8)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP
101,P-1660,2023,5,1.00,0.00,6045.00,0.00,-6045.00
155,P-1669,2023,5,7.00,0.00,245707.00,0.00,-245707.00
181,P-1672,2023,5,71.00,67.00,331943.00,880835.00,548892.00
222,P-1677,2023,5,12.00,0.00,570000.00,0.00,-570000.00
488,P-1705,2023,5,55.00,4735.00,76065.00,5196251.00,5120186.00


#### 3.2.8 - Se agrega la fila TOTAL

In [97]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5 = pd.concat([df_desp_realizados5, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TOTAL"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5["TIPO_CUENTA"]= "COSTO MATERIALES"

df_desp_realizados5.sample()

print("Dimensiones después del filtro", df_desp_realizados5.shape)

df_desp_realizados5.sample(5)

Dimensiones antes del filtro (35, 8)
Dimensiones después del filtro (36, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
26,P-1717,2023,6,112.00,843.00,18388.00,205837.00,187449.00,187449.00,COSTO MATERIALES
4,P-1670,2023,6,5.00,0.00,3410.00,0.00,-3410.00,-3410.00,COSTO MATERIALES
25,P-1716,2023,6,0.00,8.00,0.00,2308672.00,2308672.00,2308672.00,COSTO MATERIALES
11,P-1688,2023,6,0.00,2912.00,0.00,648868.00,648868.00,648868.00,COSTO MATERIALES
30,P-1724,2023,6,0.00,2.00,0.00,20168.00,20168.00,20168.00,COSTO MATERIALES


#### 3.2.8-2 - Se agrega la fila TOTAL 2

In [98]:
# Obtener la suma de cada columna y guardarla en un diccionario

print("Dimensiones antes del filtro", df_desp_realizados5_1M.shape)

total_row = {
    'CODIGO_PROYECTO': 'TOTAL',
    'AÑO': df_desp_realizados5_1M['AÑO'].iloc[0], # Obtener el valor del año de la primera fila
    'MES': df_desp_realizados5_1M['MES'].iloc[0], # Obtener el valor del mes de la primera fila
    'CANT. GDP': df_desp_realizados5_1M['CANT. GDP'].sum(),
    'CANT. VCO': df_desp_realizados5_1M['CANT. VCO'].sum(),
    'TOTAL. GDP': df_desp_realizados5_1M['TOTAL. GDP'].sum(),
    'TOTAL. VCO': df_desp_realizados5_1M['TOTAL. VCO'].sum(),
    'TOTAL. VCO - GDP': df_desp_realizados5_1M['TOTAL. VCO - GDP'].sum()
}

total_df = pd.DataFrame(total_row, index=[0]) # Crear un nuevo DataFrame con la fila 'TOTAL'
df_desp_realizados5_1M = pd.concat([df_desp_realizados5_1M, total_df], ignore_index=True) # Concatenar el nuevo DataFrame con el original

#df_desp_realizados5["TIPO_CUENTA"]= df_desp_realizados5["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TOTAL"]= df_desp_realizados5_1M["TOTAL. VCO - GDP"]
df_desp_realizados5_1M["TIPO_CUENTA"]= "COSTO MATERIALES"

print("Dimensiones después del filtro", df_desp_realizados5_1M.shape)

df_desp_realizados5_1M.sample(5)

Dimensiones antes del filtro (46, 8)
Dimensiones después del filtro (47, 10)


,CODIGO_PROYECTO,AÑO,MES,CANT. GDP,CANT. VCO,TOTAL. GDP,TOTAL. VCO,TOTAL. VCO - GDP,TOTAL,TIPO_CUENTA
38,P-1716,2023,5,0.00,896.00,0.00,1481523.00,1481523.00,1481523.00,COSTO MATERIALES
44,P-1726,2023,5,0.00,2127.00,0.00,9232066.00,9232066.00,9232066.00,COSTO MATERIALES
31,P-1706,2023,5,0.00,9311.00,0.00,5198516.00,5198516.00,5198516.00,COSTO MATERIALES
33,P-1709,2023,5,0.00,12.00,0.00,2999784.00,2999784.00,2999784.00,COSTO MATERIALES
30,P-1705,2023,5,55.00,4735.00,76065.00,5196251.00,5120186.00,5120186.00,COSTO MATERIALES


In [99]:
df_desp_realizados5.to_csv('archivos_respaldo/df_desp_realizados5.csv', mode='a')
df_desp_realizados5_1M.to_csv('archivos_respaldo/df_desp_realizados5_1M.csv', mode='a')

### 3.4 TRATAMIENTO "PROYECTADO"

#### 3.4.1 CREACIÓN NUEVO DATAFRAME DE COPIA.

In [100]:
df_proyectado2 = df_proyectado
df_proyectado2.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4212000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.1-2 CREACIÓN NUEVO DATAFRAME DE COPIA 2.

In [101]:
df_proyectado2_1M = df_proyectado_1M
df_proyectado2_1M.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 16361,Unnamed: 16362,Unnamed: 16363,Unnamed: 16364,Unnamed: 16365,Unnamed: 16366,Unnamed: 16367,Unnamed: 16368,Unnamed: 16369,Unnamed: 16370
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,OBRA,PROYECTO,PROYECTO2,MATERIALES,DESPACHOS A LA FECHA,CONTROL MATERIALES,MANO DE OBRA,PRESENTACION EPP,PROYECCIÓN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,CERRADA,P-1600,TERRANOVA,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CERRADA,P-1601,CUATRE CASAS,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,CERRADA,P-1615,ECO SANTIAGO,NaN,0,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 3.4.2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES

In [102]:
df_proyectado2 = df_proyectado2.drop(range(0, 6))
new_header = df_proyectado2.iloc[0]
df_proyectado2 = df_proyectado2[1:]
df_proyectado2.columns = new_header
df_proyectado2 = df_proyectado2.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
68,P-1717,ECO QUILIN,1000000,200000,NaN
69,P-1718,PATIO DE COMIDA,19500000,6000000,NaN
52,P-1699,VILLA PANAMERICANA LOTE B,52000000,22000000,NaN
75,P-1724,185 VOLTA,NaN,NaN,NaN
15,P-1645,LOS TRES ANTONIO,NaN,NaN,NaN
42,P-1689,CASINO,NaN,NaN,NaN
50,P-1697,FORTALEZA ESTOCOLMO,5000000,1000000,NaN
48,P-1695,ESTORIL,NaN,3000000,NaN
71,P-1720,CLARO DECOMBE,NaN,NaN,NaN
14,P-1642,CORINTOS,NaN,NaN,NaN


#### 3.4.2-2 LIMPIEZA DE DATAFRAME Y SELECCIÓN DE COLUMNAS UTILES 2

In [103]:
df_proyectado2_1M = df_proyectado2_1M.drop(range(0, 6))
new_header = df_proyectado2_1M.iloc[0]
df_proyectado2_1M = df_proyectado2_1M[1:]
df_proyectado2_1M.columns = new_header
df_proyectado2_1M = df_proyectado2_1M.loc[:, ["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
df_proyectado2_1M.sample(10)

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
70,P-1719,METALPARK,NaN,NaN,NaN
61,P-1708,Eco España,1568763,NaN,NaN
75,P-1724,185 VOLTA,90720,NaN,NaN
50,P-1697,FORTALEZA ESTOCOLMO,16105752.69,2500000,NaN
63,P-1712,CC VIÑA Outlet Park,15036000,8500000,NaN
47,P-1694,ECO HUEMUL,17000065,7000000,NaN
8,P-1601,CUATRE CASAS,NaN,NaN,NaN
25,P-1670,MK SEGURIDAS,NaN,NaN,NaN
59,P-1706,California,17637537,5000000,NaN
46,P-1693,VOLTA,NaN,NaN,NaN


In [104]:
# INSPECCIÓN VISUAL 1
df_proyectado2_1M[df_proyectado2_1M["PROYECTO"]=="P-1726"]

6,PROYECTO,PROYECTO2,INGRESO,MATERIALES,MANO DE OBRA
77,P-1726,LA GLORIA,25429159,10000000,NaN


#### 3.4.3 TRATAMIENTO DE DATOS NAN

In [105]:
# seleccionar las columnas deseadas
df_proyectado3 = df_proyectado2[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3 = df_proyectado3.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3 = df_proyectado3.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3['INGRESO'] = df_proyectado3['INGRESO'].fillna(0)
df_proyectado3['MATERIALES'] = df_proyectado3['MATERIALES'].fillna(0)
df_proyectado3['MANO DE OBRA'] = df_proyectado3['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3 = df_proyectado3.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3['CODIGO_PROYECTO'] = df_proyectado3['CODIGO_PROYECTO'].str.upper()
df_proyectado3['PROYECTO'] = df_proyectado3['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)

df_proyectado3.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (74, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
57,P-1713,51 EDIFICIO RESERVA SAN FRANCISCO,16500000,6000000,0
9,P-1654,EXEQUIEL FERNANDEZ 1670,0,0,0
71,P-1727,BUDNIK,26500000,10000000,0
21,P-1674,BEIERSDORF,0,0,0
63,P-1719,METALPARK,0,0,0
29,NaN,BPA05,0,0,0
70,P-1726,LA GLORIA,12000000,2000000,0
13,P-1661,SAN VICENTE,0,0,0
69,P-1725,LO CAÑA,14000000,5500000,0
72,P-1728,VOLTA SEGREGACIÓN,0,0,0


In [106]:
print(df_proyectado3.columns)

Index(['CODIGO_PROYECTO', 'PROYECTO', 'INGRESO', 'MATERIALES', 'MANO DE OBRA'], dtype='object', name=6)


In [107]:
# Eliminar la columna '6'
df_proyectado3 = df_proyectado3.rename_axis(None, axis=1)
df_proyectado3 = df_proyectado3.dropna(subset=['CODIGO_PROYECTO'])

print(df_proyectado3.shape)
df_proyectado3.sample(15)

(72, 5)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
30,P-1684,GOYCOLEA,25000000,12035806,0
14,P-1663,OFICINAS EQUIFAX,0,0,0
22,P-1675,PALENA,0,0,0
66,P-1722,ECO CERRILLO,0,0,0
16,P-1668,QUILLAYES,0,0,0
46,P-1700,ORIZON,0,0,0
42,P-1696,SANTA VICTORIA,0,0,0
3,P-1621,RENAULT,0,0,0
71,P-1727,BUDNIK,26500000,10000000,0
50,P-1704,MERCADO LIBRE,62000000,27000000,0


#### 3.4.3-2 TRATAMIENTO DE DATOS NAN 2

In [108]:
# seleccionar las columnas deseadas
df_proyectado3_1M = df_proyectado2_1M[["PROYECTO", "PROYECTO2", "INGRESO","MATERIALES","MANO DE OBRA"]]
# eliminar la columna índice
df_proyectado3_1M = df_proyectado3_1M.reset_index(drop=True)
# eliminar las filas que contengan NaN en la columna P
df_proyectado3_1M = df_proyectado3_1M.dropna(subset=['PROYECTO'])
# reemplazar los valores NaN en la columna INGRESO por cero
df_proyectado3_1M['INGRESO'] = df_proyectado3_1M['INGRESO'].fillna(0)
df_proyectado3_1M['MATERIALES'] = df_proyectado3_1M['MATERIALES'].fillna(0)
df_proyectado3_1M['MANO DE OBRA'] = df_proyectado3_1M['MANO DE OBRA'].fillna(0)

# renombrar la columna P a CODIGO_PROYECTO
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO': 'CODIGO_PROYECTO'})
df_proyectado3_1M = df_proyectado3_1M.rename(columns={'PROYECTO2': 'PROYECTO'})

# poner en mayúscula el contenido de las columnas CODIGO_PROYECTO y PROYECTO
df_proyectado3_1M['CODIGO_PROYECTO'] = df_proyectado3_1M['CODIGO_PROYECTO'].str.upper()
df_proyectado3_1M['PROYECTO'] = df_proyectado3_1M['PROYECTO'].str.upper()

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)

df_proyectado3_1M.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (73, 5)


6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
46,P-1700,ORIZON,0.00,0,0
52,P-1706,CALIFORNIA,17637537.00,5000000,0
12,P-1660,CORONEL ALVARADO,0.00,0,0
41,P-1695,ESTORIL,13003381.00,10500000,0
42,P-1696,SANTA VICTORIA,0.00,0,0
49,P-1703,SANTA VICTORIA DANACORP,0.00,0,0
68,P-1724,185 VOLTA,90720.00,0,0
4,P-1652,ABDON CIFUENTES,0.00,0,0
61,P-1717,ECO QUILIN,0.00,0,0
43,P-1697,FORTALEZA ESTOCOLMO,16105752.69,2500000,0


In [109]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

6,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA
70,P-1726,LA GLORIA,25429159.00,10000000,0


#### 3.4.4 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO

In [110]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
current_month = datetime.datetime.now().month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso = df_proyectado3['INGRESO'].sum()
total_materiales = df_proyectado3['MATERIALES'].sum()
total_mano_obra = df_proyectado3['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3 = pd.concat([df_proyectado3, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3["TIPO_CUENTA"] = "INGRESO"
df_proyectado3["MES"] = current_month 
df_proyectado3["AÑO"] = current_year 
df_proyectado3["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3 = df_proyectado3.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3['TOTAL'] = df_proyectado3[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  : ", df_proyectado3.shape)
df_proyectado3.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3  :  (73, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
29,P-1684,GOYCOLEA,25000000,-12035806,0,12964194,INGRESO,6,2023,PROYECTADO
71,P-1729,ESTACION CENTRAL,0,0,0,0,INGRESO,6,2023,PROYECTADO
41,P-1697,FORTALEZA ESTOCOLMO,5000000,-1000000,0,4000000,INGRESO,6,2023,PROYECTADO
28,P-1682,PLAZA CEPPI,0,0,0,0,INGRESO,6,2023,PROYECTADO
46,P-1702,KARUN VITACURA,0,0,0,0,INGRESO,6,2023,PROYECTADO


In [111]:
df_proyectado3['INGRESO'].sum()

847600000

#### 3.4.4-2 SE AGREGA LA COLUMNA TOTAL PARA EL DATAFRAME Y SE FILTRA POR EL MES EN CURSO 2

In [112]:
# Obtener el año y mes actual

import datetime

current_year = datetime.datetime.now().year
last_month = (fecha_actual - timedelta(days=30)).month

# Calcula la sumatoria de la columna 'INGRESO'
total_ingreso    = df_proyectado3_1M['INGRESO'].sum()
total_materiales = df_proyectado3_1M['MATERIALES'].sum()
total_mano_obra  = df_proyectado3_1M['MANO DE OBRA'].sum()

# Crea la fila 'TOTAL'
total_row = {'CODIGO_PROYECTO': 'TOTAL', 'PROYECTO': '', 'INGRESO': total_ingreso,'MATERIALES': total_materiales,'MANO DE OBRA': total_mano_obra}

# Crea un DataFrame con la fila 'TOTAL'
df_total = pd.DataFrame([total_row])

# Concatena el DataFrame original con el DataFrame que contiene la fila 'TOTAL'
df_proyectado3_1M = pd.concat([df_proyectado3_1M, df_total], ignore_index=True)

#Agreación de columnas para estandarización
df_proyectado3_1M["TOTAL"] = df_proyectado3["INGRESO"]
df_proyectado3_1M["TIPO_CUENTA"] = "INGRESO"
df_proyectado3_1M["MES"] = last_month 
df_proyectado3_1M["AÑO"] = current_year 
df_proyectado3_1M["ORIGEN"] = "PROYECTADO" 

# Se reemplazan valores NaN en 0 
df_proyectado3_1M = df_proyectado3_1M.fillna(value=0)

# Multiplicar las columnas MATERIALES y MANO DE OBRA por -1 para obtener valores negativos
df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']] = df_proyectado3_1M[['MATERIALES', 'MANO DE OBRA']].mul(-1)

# Se calcula la columna TOTAL como la suma de INGRESO, MATERIALES y MANO DE OBRA
df_proyectado3_1M['TOTAL'] = df_proyectado3_1M[['INGRESO', 'MATERIALES', 'MANO DE OBRA']].sum(axis=1)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  : ", df_proyectado3_1M.shape)
df_proyectado3_1M.sample(5)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_proyectado3_1M  :  (74, 10)


,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
53,P-1707,LAS TRANQUERAS,7084468.00,-7000000,0,84468.00,INGRESO,5,2023,PROYECTADO
40,P-1694,ECO HUEMUL,17000065.00,-7000000,0,10000065.00,INGRESO,5,2023,PROYECTADO
6,P-1638,COLON,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
10,P-1655,DECATHLON,0.00,0,0,0.00,INGRESO,5,2023,PROYECTADO
62,P-1718,PATIO DE COMIDA,18676629.00,-12000000,0,6676629.00,INGRESO,5,2023,PROYECTADO


In [113]:
# INSPECCIÓN VISUAL 1
df_proyectado3_1M[df_proyectado3_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,INGRESO,MATERIALES,MANO DE OBRA,TOTAL,TIPO_CUENTA,MES,AÑO,ORIGEN
70,P-1726,LA GLORIA,25429159.00,-10000000,0,15429159.00,INGRESO,5,2023,PROYECTADO


#### 3.4.5 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS. 

In [114]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4 = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)

df_proyectado4.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (73, 10)
Dimensiones solo INGRESO (73, 7)
Dimensiones solo MANO DE OBRA (73, 7)
Dimensiones solo MATERIALES (73, 7)
Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
36,P-1692,JOSE PEREZ COTAPO,-500000,6,2023,PROYECTADO,MATERIALES
65,P-1723,VILLA PANAMERICANA LOTE C,0,6,2023,PROYECTADO,MANO DE OBRA
35,P-1691,ROSARIO ROSALES,0,6,2023,PROYECTADO,MATERIALES
24,P-1677,GALVARINO,0,6,2023,PROYECTADO,MANO DE OBRA
25,P-1678,PIRER,0,6,2023,PROYECTADO,INGRESO
45,P-1701,ZENTENO,0,6,2023,PROYECTADO,INGRESO
17,P-1669,DEPARTAMENTAL II,0,6,2023,PROYECTADO,INGRESO
5,P-1632,CHORILLOS CLOROX,0,6,2023,PROYECTADO,MATERIALES
18,P-1670,MK SEGURIDAS,0,6,2023,PROYECTADO,MATERIALES
70,P-1728,VOLTA SEGREGACIÓN,0,6,2023,PROYECTADO,MATERIALES


#### 3.4.5-2 ESTRATEGIA PARA TRANSFORMAR LAS COLUMNAS DE MANO DE OBRA, MATERIALES E INGRESOS EN FILAS 2  

In [115]:
print("Dimensiones solo df_proyectado3 antes de cambios",df_proyectado3_1M.shape)

# Crear el subDataFrame sin las columnas MANO DE OBRA, MATERIALES, TOTAL y TIPO_CUENTA
df_sub = df_proyectado3_1M.drop(['MANO DE OBRA', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor INGRESO
df_sub['TIPO_CUENTA'] = 'INGRESO'

# Renombrar la columna INGRESO como TOTAL
df_sub = df_sub.rename(columns={'INGRESO': 'TOTAL'})
print("Dimensiones solo INGRESO",df_sub.shape)

# Crear el subDataFrame sin las columnas INGRESO, MATERIALES, TOTAL y TIPO_CUENTA
df_sub2 = df_proyectado3_1M.drop(['INGRESO', 'MATERIALES', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub2['TIPO_CUENTA'] = 'MANO DE OBRA'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub2 = df_sub2.rename(columns={'MANO DE OBRA': 'TOTAL'})
print("Dimensiones solo MANO DE OBRA",df_sub2.shape)


# Crear el subDataFrame sin las columnas INGRESO, MANO DE OBRA, TOTAL y TIPO_CUENTA
df_sub3 = df_proyectado3_1M.drop(['INGRESO', 'MANO DE OBRA', 'TOTAL', 'TIPO_CUENTA'], axis=1)

# Agregar la columna TIPO_CUENTA con el valor MANO DE OBRA
df_sub3['TIPO_CUENTA'] = 'MATERIALES'

# Renombrar la columna MANO DE OBRA como TOTAL
df_sub3 = df_sub3.rename(columns={'MATERIALES': 'TOTAL'})
print("Dimensiones solo MATERIALES",df_sub3.shape)

# Concatenar los DataFrames verticalmente
df_proyectado4_1M = pd.concat([df_sub, df_sub2, df_sub3])

# Mostrar el nuevo DataFrame concatenado

print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)

df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado3 antes de cambios (74, 10)
Dimensiones solo INGRESO (74, 7)
Dimensiones solo MANO DE OBRA (74, 7)
Dimensiones solo MATERIALES (74, 7)
Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
53,P-1707,LAS TRANQUERAS,-7000000.00,5,2023,PROYECTADO,MATERIALES
68,P-1724,185 VOLTA,90720.00,5,2023,PROYECTADO,INGRESO
38,P-1692,JOSE PEREZ COTAPO,-500000.00,5,2023,PROYECTADO,MATERIALES
46,P-1700,ORIZON,0.00,5,2023,PROYECTADO,MATERIALES
48,P-1702,KARUN VITACURA,0.00,5,2023,PROYECTADO,MATERIALES
4,P-1652,ABDON CIFUENTES,0.00,5,2023,PROYECTADO,INGRESO
13,P-1661,SAN VICENTE,0.00,5,2023,PROYECTADO,INGRESO
17,P-1669,DEPARTAMENTAL II,0.00,5,2023,PROYECTADO,MANO DE OBRA
58,P-1714,DEPARTAMENTAL II,2436155.00,5,2023,PROYECTADO,INGRESO
5,P-1632,CHORILLOS CLOROX,0.00,5,2023,PROYECTADO,MATERIALES


In [116]:
# INSPECCIÓN VISUAL 1
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,MATERIALES


#### 3.4.6 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA

In [117]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4['TIPO_CUENTA'] = df_proyectado4['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4.shape)
df_proyectado4.sample(10)

Dimensiones solo df_proyectado4 después de cambios (219, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
11,P-1656,CLUB HIPICO,0,6,2023,PROYECTADO,2. COSTO MATERIALES
64,P-1722,ECO CERRILLO,0,6,2023,PROYECTADO,1. INGRESO
46,P-1702,KARUN VITACURA,0,6,2023,PROYECTADO,2. COSTO MATERIALES
42,P-1698,PCI ANDES,0,6,2023,PROYECTADO,2. COSTO MATERIALES
71,P-1729,ESTACION CENTRAL,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
35,P-1691,ROSARIO ROSALES,0,6,2023,PROYECTADO,1. INGRESO
31,P-1687,ROJAS MAGALLANES,0,6,2023,PROYECTADO,2. COSTO MATERIALES
23,P-1676,PORTAL LO CAMPINO,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA
40,P-1696,SANTA VICTORIA,0,6,2023,PROYECTADO,1. INGRESO
62,P-1720,CLARO DECOMBE,0,6,2023,PROYECTADO,2. COSTO MATERIALES


In [118]:
# TEST TOTAL "INGRESO" 

df_proyectado4_test = df_proyectado4[df_proyectado4['TIPO_CUENTA'] == '1. INGRESO']
df_proyectado4_test                                     

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
0,P-1600,TERRANOVA,0,6,2023,PROYECTADO,1. INGRESO
1,P-1601,CUATRE CASAS,0,6,2023,PROYECTADO,1. INGRESO
2,P-1615,ECO SANTIAGO,0,6,2023,PROYECTADO,1. INGRESO
3,P-1621,RENAULT,0,6,2023,PROYECTADO,1. INGRESO
4,P-1652,ABDON CIFUENTES,0,6,2023,PROYECTADO,1. INGRESO
5,P-1632,CHORILLOS CLOROX,0,6,2023,PROYECTADO,1. INGRESO
6,P-1638,COLON,0,6,2023,PROYECTADO,1. INGRESO
7,P-1642,CORINTOS,0,6,2023,PROYECTADO,1. INGRESO
8,P-1645,LOS TRES ANTONIO,0,6,2023,PROYECTADO,1. INGRESO
9,P-1654,EXEQUIEL FERNANDEZ 1670,0,6,2023,PROYECTADO,1. INGRESO


#### 3.4.6-2 CAMBIO DE VALORES EN COLUMNA TIPO_CUENTA 2

In [119]:
# Cambiar los valores de la columna TIPO_CUENTA
df_proyectado4_1M['TIPO_CUENTA'] = df_proyectado4_1M['TIPO_CUENTA'].replace({'MATERIALES': '2. COSTO MATERIALES', 'MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO'})

# Mostrar el DataFrame con los valores cambiados
print("Dimensiones solo df_proyectado4 después de cambios",df_proyectado4_1M.shape)
df_proyectado4_1M.sample(10)

Dimensiones solo df_proyectado4 después de cambios (222, 7)


,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
39,P-1693,VOLTA,0.00,5,2023,PROYECTADO,1. INGRESO
16,P-1668,QUILLAYES,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
44,P-1698,PCI ANDES,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
5,P-1632,CHORILLOS CLOROX,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
41,P-1695,ESTORIL,-10500000.00,5,2023,PROYECTADO,2. COSTO MATERIALES
10,P-1655,DECATHLON,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
22,P-1675,PALENA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
21,P-1674,BEIERSDORF,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
47,P-1701,ZENTENO,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES
2,P-1615,ECO SANTIAGO,0.00,5,2023,PROYECTADO,1. INGRESO


In [120]:
df_proyectado4_1M[df_proyectado4_1M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA
70,P-1726,LA GLORIA,25429159.00,5,2023,PROYECTADO,1. INGRESO
70,P-1726,LA GLORIA,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA
70,P-1726,LA GLORIA,-10000000.00,5,2023,PROYECTADO,2. COSTO MATERIALES


In [121]:
df_proyectado4.to_csv('archivos_respaldo/df_proyectado4.csv', mode='a')
df_proyectado4_1M.to_csv('archivos_respaldo/df_proyectado4_1M.csv', mode='a')

### 3.5 TRATAMIENTO LIBRO MAYOR AÑOS ANTERIORES

#### 3.5.1 - REVISIÓN DE LOS DATOS

In [122]:
df_libro_mayor_old.shape

(17368, 13)

In [123]:
df_libro_mayor_old.describe()

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario
count,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368,17368
unique,10,2,12,531,5,1312,3453,134,1500,5119,8462,1714,2
top,4101010-000,2022,9,12/31/22 0:00,CIN,1252,REMUNERACIONES DIRECTAS PROYECTO,,76525290-3,,0,0,
freq,12133,9797,1775,252,11773,251,1359,699,406,1958,3199,15265,17359


#### 3.5.2 - Estandarización campo "Debito" y Crédito

In [124]:
# Se crean nuevos campos para trabajar "Debito" y "Credito".

df_libro_mayor_old['Debito_2'] = pd.to_numeric(df_libro_mayor_old['Debito'], errors='coerce')
df_libro_mayor_old['Credito_2'] = pd.to_numeric(df_libro_mayor_old['Credito'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_libro_mayor_old['Credito_2'] = df_libro_mayor_old['Credito_2'].fillna(0)

df_libro_mayor_old.head(5)

,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.3 - Creación campo Codigo_proyecto

In [125]:
# Se crean una nueva columna para manejar el código del proyecto 

df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['Primer_Analisis']

cols = df_libro_mayor_old.columns.tolist() # Obtiene el nombre de todas las columnas en una lista
cols.insert(0, cols.pop(cols.index('CODIGO_PROYECTO'))) # Extrae la columna 'Codigo_proyecto', la inserta en la posición 0 y actualiza la lista
df_libro_mayor_old = df_libro_mayor_old.reindex(columns=cols) # Reindexa el dataframe con las columnas en el orden actualizado

# Transformación a mayúscula código del proyecto
df_libro_mayor_old['CODIGO_PROYECTO'] = df_libro_mayor_old['CODIGO_PROYECTO'].str.upper()

df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00


#### 3.5.4 - Creación de campo total

In [126]:
df_libro_mayor_old['TOTAL'] = df_libro_mayor_old['Credito_2'] - df_libro_mayor_old['Debito_2']
df_libro_mayor_old.head(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL
1,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,EP N°26 - OBRA: S-0333 EDIFICIO NC 3000,P-1592,76525290-3,10756,0,6481068,,0.00,6481068.00,6481068.00
2,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,"UF 222,93 x T/C $29.072,21",P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
3,P-1592,3101030-000,2021,1,1/6/21 0:00,CVT,47,SEGÚN CONTRATO N°8000033317-00010 - HES N°1000...,P-1592,76525290-3,10756,0,0,,0.00,0.00,0.00
4,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,EP N°4 - OBRA: VIVIPRA,P-1658,79606430-7,10763,0,4301576,,0.00,4301576.00,4301576.00
5,P-1658,3101030-000,2021,1,1/6/21 0:00,CVT,60,"UF 148 x T/C $29.064,70",P-1658,79606430-7,10763,0,0,,0.00,0.00,0.00


#### 3.5.5 - Creación campo tipo_cuenta "basado en el campo cta_Contable"

In [127]:
# Función para categorizar la columna 'Cta_Contable'
def categorizar_cta_contable(cta_contable):
    if cta_contable =="3101030-000" or cta_contable == "3401002-000":
        return '1. INGRESO'
    elif cta_contable == "4101010-000":
        return '2. COSTO MATERIALES'
    elif cta_contable == "4150002-000":
        return '3. COSTO MANO DE OBRA'
    elif cta_contable == "4150002-002":
        return '4. COSTO SUPERVISIÓN'
    else:
        return 'OTRA'

# Crear la nueva columna 'Tipo_cuenta' aplicando la función
df_libro_mayor_old['Tipo_cuenta'] = df_libro_mayor_old['Cta_Contable'].apply(categorizar_cta_contable)
df_libro_mayor_old.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
13526,P-1677,4101010-000,2022,9,9/23/22 0:00,CIN,653,Cerradura Electrica Scanavini (chapa),P-1677,S10271067,47298,67450,0,,67450.00,0.00,-67450.00,2. COSTO MATERIALES
2119,P-1642,4101010-000,2021,2,2/17/21 0:00,CIN,853,"Tornillo volcanita 6 x 1 1/4""",P-1642,I17440305,40766,700,0,,700.00,0.00,-700.00,2. COSTO MATERIALES
12183,P-1685,4101010-000,2022,8,8/2/22 0:00,CIN,63,"Golilla presion 1/2"" zincada",P-1685,I15410754,46550,260,0,,260.00,0.00,-260.00,2. COSTO MATERIALES
4279,P-1653,4101010-000,2021,6,6/15/21 0:00,CIN,729,Brazo Solo Batientes BFT Lux 2.5M 150kg,P-1653,S12321349,42390,433216,0,,433216.00,0.00,-433216.00,2. COSTO MATERIALES
15045,P-1680,4101010-000,2022,12,12/1/22 0:00,CIN,21,Centralita Bosch DS3MX,P-1680,S11280105,48272,154000,0,,154000.00,0.00,-154000.00,2. COSTO MATERIALES


In [128]:
df_libro_mayor_old['Tipo_cuenta'].unique()

array(['1. INGRESO', '2. COSTO MATERIALES', 'OTRA',
       '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'], dtype=object)

In [129]:
# Revisión de los datos creados

df_libro_mayor_old_piv = pd.pivot_table(df_libro_mayor_old, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old_piv = pd.DataFrame(df_libro_mayor_old_piv.to_records())

#df_libro_mayor_piv.head(5)
df_libro_mayor_old_piv.sample(10)

,Tipo_cuenta,Año,Mes,Credito_2,Debito_2,TOTAL
106,OTRA,2021,8,0,10668933,-10668933
119,OTRA,2022,9,0,15108178,-15108178
58,3. COSTO MANO DE OBRA,2021,8,0,1844654,-1844654
95,4. COSTO SUPERVISIÓN,2022,9,0,3907465,-3907465
94,4. COSTO SUPERVISIÓN,2022,8,0,4151822,-4151822
64,3. COSTO MANO DE OBRA,2022,2,0,15328808,-15328808
26,2. COSTO MATERIALES,2021,11,2334550,23327140,-20992590
32,2. COSTO MATERIALES,2021,6,2095577,34292055,-32196478
98,OTRA,2021,11,0,8238579,-8238579
70,3. COSTO MANO DE OBRA,2022,8,0,70301037,-70301037


#### 3.5.6 - Se eliminan números de proyectos que no deben ser parte del análisis

In [130]:
# Se crea una copia del dataframe original debido que se reduciran las dimensiones
df_libro_mayor_old2 = df_libro_mayor_old.copy()

# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)


Proyectos disponibles antes de la reducción : 
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANALISIS PROYE' 

##### 3.5.6.1 - Se crea un subdataframe para COSTOS SUPERVISIÓN (separando lo que tiene P-OFICINA y cta_contable 4150002-002)

In [131]:
# Se crea un dataframe separado para tratar los que poseen proyecto "P-OFICINA" ya que los costos de supervisión solo aplican al total
df_libro_mayor_old2_2 = df_libro_mayor_old2.copy()

# Filtrar el DataFrame
df_libro_mayor_old2_2 = df_libro_mayor_old2_2[(df_libro_mayor_old2_2['CODIGO_PROYECTO'] == 'P-OFICINA') & (df_libro_mayor_old2_2['Cta_Contable'] == '4150002-002')]

# Mostrar el DataFrame filtrado
df_libro_mayor_old2_2


,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN
16837,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,0,2183185,0,,2183185.00,0.00,-2183185.00,4. COSTO SUPERVISIÓN
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN
16839,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2584134,0,,2584134.00,0.00,-2584134.00,4. COSTO SUPERVISIÓN
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN
16841,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2719503,0,,2719503.00,0.00,-2719503.00,4. COSTO SUPERVISIÓN
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN


In [132]:
# Elimina los registros NaN en la columna Codigo_proyecto
df_libro_mayor_old2 = df_libro_mayor_old2.dropna(subset=['CODIGO_PROYECTO'])

# Define una lista de valores a eliminar
valores_a_eliminar = ['P-PROYECTOS', 'P-SERVICIOS','P-OFICINA', 'P-ANALISIS PROYE', '13056802-5', '16411600-K', '18800250-1','P-MANTENCIÓN','18800250','13056802','16411600']

# Crea una máscara booleana que identifica las filas que contienen los valores a eliminar
mask = df_libro_mayor_old2['CODIGO_PROYECTO'].isin(valores_a_eliminar)

# Crea un nuevo dataframe sin las filas que contienen los valores a eliminar
df_libro_mayor_old2 = df_libro_mayor_old2[~mask]


In [133]:
# Se identifican los códigos de proyecto disponibles
print("Proyectos disponibles antes de la reducción : ")
print(df_libro_mayor_old.shape)
print(df_libro_mayor_old["CODIGO_PROYECTO"].unique())
print("=======================================================================")
print("Dimensiones del dataframe antes de la reducción : ")
print(df_libro_mayor_old2.shape)
print(df_libro_mayor_old2["CODIGO_PROYECTO"].unique())


Proyectos disponibles antes de la reducción : 
(17368, 18)
['P-1592' 'P-1658' 'P-1638' 'P-1615' 'P-1639' 'S-1601' 'P-1644' 'P-1628'
 'P-1652' 'P-1666' 'P-1653' 'P-1654' 'P-1656' 'P-1665' 'P-1664' 'P-1670'
 'P-1629' 'P-1649' 'P-1657' 'P-1661' 'P-1671' 'P-1645' 'P-1626' 'P-1611'
 'P-1642' 'P-1648' 'P-1659' 'P-1617' 'P-1627' 'P-1641' 'P-1660' 'P-1636'
 'P-1625' 'P-1667' 'P-1672' 'P-1637' 'P-1623' 'P-1674' 'P-1668' 'P-1676'
 'P-1646' 'P-1675' 'P-1678' 'P-1643' 'P-1632' 'P-1673' 'P-1669' 'P-1677'
 'P-1681' 'P-1683' 'P-1679' 'P-1680' 'P-1684' 'P-1689' 'P-1686' 'P-1685'
 'P-1690' 'P-1691' 'P-OFICINA' 'P-PROYECTOS' 'P-1663' 'P-1640' 'P-1655'
 'P-1631' 'P-1647' 'P-1610' 'P-1612' 'P-1527' 'P-1590' 'P-1616' 'P-1622'
 'P-1633' '' 'P-1692' 'P-1687' 'P-1693' 'P-1694' 'P-1695' 'P-1698'
 'P-1696' 'P-1688' 'P-1697' 'P-1700' 'P-1701' 'P-1702' 'P-1703' 'P-1709'
 'P-1707' 'P-1704' 'P-1712' 'P-1713' 'P-1708' 'P-1682' 'P-1714' 'P-1706'
 'P-1705' 'P-1699' 'P-1715' 'P-1716' 'P-1718' 'P-1717' 'P-1613'
 'P-ANAL

#### 3.5.7 - Se crea un nuevo campo de fecha basado en la fecha original

In [134]:
df_libro_mayor_old2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2['Año'].astype(str) + '-' + df_libro_mayor_old2['Mes'].astype(str) + '-1')
df_libro_mayor_old2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
5253,P-1673,4101010-000,2021,9,9/1/21 0:00,CIN,548,Gas Butano,P-1673,I17452325,43019,996,0,,996.00,0.00,-996.00,2. COSTO MATERIALES,2021-09-01
1112,P-1674,3401002-000,2021,9,9/30/21 0:00,AJC,839,Ajuste por anticipos y retenciones,P-1674,P-1674,,0,54224,,0.00,54224.00,54224.00,1. INGRESO,2021-09-01
7792,P-1688,3101030-000,2022,4,4/13/22 0:00,CVT,35,SERVC. DE SUM. E INST.CORRIENTES DEBILES.,P-1688,76071313-9,12376,0,0,,0.00,0.00,0.00,1. INGRESO,2022-04-01
8698,P-1682,3101030-000,2022,12,12/27/22 0:00,CVT,589,FORMA DE PAGO: CREDITO CONTRATO 2-15,P-1682,77594729-2,13273,0,0,,0.00,0.00,0.00,1. INGRESO,2022-12-01
7574,P-1683,3101030-000,2022,1,1/3/22 0:00,CVT,1,SEGUN CONTRATO N° 7000023146 DE FECHA 31 DE AG...,P-1683,76525290-3,12092,0,0,,0.00,0.00,0.00,1. INGRESO,2022-01-01


##### 3.5.7.1 - Se crea un nuevo campo de fecha basado en la fecha original para  COSTOS SUPERVISIÓN

In [135]:
df_libro_mayor_old2_2['Fecha2'] = pd.to_datetime(df_libro_mayor_old2_2['Año'].astype(str) + '-' + df_libro_mayor_old2_2['Mes'].astype(str) + '-1')
df_libro_mayor_old2_2.sample(5)

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2
16843,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,7435204,,2502914,0,,2502914.00,0.00,-2502914.00,4. COSTO SUPERVISIÓN,2022-12-01
16842,P-OFICINA,4150002-002,2022,12,12/31/22 0:00,AJC,1252,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2153705,0,,2153705.00,0.00,-2153705.00,4. COSTO SUPERVISIÓN,2022-12-01
16836,P-OFICINA,4150002-002,2022,9,9/30/22 0:00,AJC,1325,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,0,1724280,0,,1724280.00,0.00,-1724280.00,4. COSTO SUPERVISIÓN,2022-09-01
16838,P-OFICINA,4150002-002,2022,10,10/31/22 0:00,AJC,1220,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2234754,0,,2234754.00,0.00,-2234754.00,4. COSTO SUPERVISIÓN,2022-10-01
16840,P-OFICINA,4150002-002,2022,11,11/30/22 0:00,AJC,1385,REMUNERACIONES SUPERVISORES,P-Oficina,13677891,,2369840,0,,2369840.00,0.00,-2369840.00,4. COSTO SUPERVISIÓN,2022-11-01


In [136]:
df_libro_mayor_old2_2['Tipo_cuenta'].unique()

array(['4. COSTO SUPERVISIÓN'], dtype=object)

In [371]:
df_libro_mayor_old2_2[df_libro_mayor_old2_2["Primer_Analisis"] == "P-1673"]

,CODIGO_PROYECTO,Cta_Contable,Año,Mes,Fecha,Tipo_Comp,Num_Comp,Glosa,Primer_Analisis,Referencia,Num_Referencia,Debito,Credito,Comentario,Debito_2,Credito_2,TOTAL,Tipo_cuenta,Fecha2


#### 3.5.8 - Se realiza pivote final del dataframe

In [137]:
# Se exploran las columnas:
df_libro_mayor_old2.columns

Index(['CODIGO_PROYECTO', 'Cta_Contable', 'Año', 'Mes', 'Fecha', 'Tipo_Comp',
       'Num_Comp', 'Glosa', 'Primer_Analisis', 'Referencia', 'Num_Referencia',
       'Debito', 'Credito', 'Comentario ', 'Debito_2', 'Credito_2', 'TOTAL',
       'Tipo_cuenta', 'Fecha2'],
      dtype='object', name=0)

In [138]:
# crear pivot table sin margenes

df_libro_mayor_old2_piv = pd.pivot_table(df_libro_mayor_old2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_piv = pd.DataFrame(df_libro_mayor_old2_piv.to_records())

df_libro_mayor_old2_piv.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
839,P-1671,2. COSTO MATERIALES,2021-07-01 00:00:00,2021,7,0,972615,-972615
197,P-1638,2. COSTO MATERIALES,2021-07-01 00:00:00,2021,7,0,334084,-334084
220,P-1639,1. INGRESO,2021-08-01 00:00:00,2021,8,74583,0,74583
324,P-1645,2. COSTO MATERIALES,2022-10-01 00:00:00,2022,10,0,18500,-18500
1105,P-1681,2. COSTO MATERIALES,2022-03-01 00:00:00,2022,3,0,120000,-120000


In [367]:
df_libro_mayor_old2_piv[df_libro_mayor_old2_piv["CODIGO_PROYECTO"] == "P-1704"]

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1523,P-1704,1. INGRESO,2022-07-01 00:00:00,2022,7,179643612,0,179643612
1524,P-1704,1. INGRESO,2022-08-01 00:00:00,2022,8,84292194,0,84292194
1525,P-1704,1. INGRESO,2022-09-01 00:00:00,2022,9,97802473,34754970,63047503
1526,P-1704,1. INGRESO,2022-10-01 00:00:00,2022,10,49425495,0,49425495
1527,P-1704,1. INGRESO,2022-11-01 00:00:00,2022,11,42952357,0,42952357
1528,P-1704,1. INGRESO,2022-12-01 00:00:00,2022,12,89246807,42952357,46294450
1529,P-1704,2. COSTO MATERIALES,2022-05-01 00:00:00,2022,5,0,273115,-273115
1530,P-1704,2. COSTO MATERIALES,2022-06-01 00:00:00,2022,6,518865,27455060,-26936195
1531,P-1704,2. COSTO MATERIALES,2022-07-01 00:00:00,2022,7,0,24099085,-24099085
1532,P-1704,2. COSTO MATERIALES,2022-08-01 00:00:00,2022,8,0,26228061,-26228061


In [139]:
# Estas son las dimensiones del dataframe pivoteado:
df_libro_mayor_old2_piv.shape

(1632, 8)

##### 3.5.8.1 - Se realiza pivote final del dataframe paera COSTO SUPERVISIÓN

In [140]:
# crear pivot table sin margenes

df_libro_mayor_old2_2_piv = pd.pivot_table(df_libro_mayor_old2_2, values=['Debito_2','Credito_2','TOTAL'], index=['CODIGO_PROYECTO','Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0, margins=True, margins_name='TOTAL')
df_libro_mayor_old2_2_piv = pd.DataFrame(df_libro_mayor_old2_2_piv.to_records())

df_libro_mayor_old2_2_piv = df_libro_mayor_old2_2_piv[df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] != 'TOTAL']

# Se reemplaza el valor P-OFICINA por el valor TOTAL con lo cual se utilizará para unirlo al dataframe que crea las filas para el cálculo de los totales
df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'] = df_libro_mayor_old2_2_piv['CODIGO_PROYECTO'].replace('P-OFICINA', 'TOTAL')

df_libro_mayor_old2_2_piv.sample()

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
1,TOTAL,4. COSTO SUPERVISIÓN,2022-10-01 00:00:00,2022,10,0,4818888,-4818888


#### 3.5.9 - Se crea subdataframe para proyecto que considera el total con nombre "TOTAL", que se detalla por tipo_cuenta


In [141]:
df_libro_mayor_old2_piv2 = pd.pivot_table(df_libro_mayor_old2_piv, values=['Debito_2','Credito_2','TOTAL'], index=['Tipo_cuenta','Fecha2','Año','Mes'], aggfunc='sum', fill_value=0)
df_libro_mayor_old2_piv2 = pd.DataFrame(df_libro_mayor_old2_piv2.to_records())

# Agregamos la columna Codigo_proyecto con valor "TOTAL" en la primera columna
df_libro_mayor_old2_piv2.insert(loc=0, column='CODIGO_PROYECTO', value='TOTAL')

# Eliminamos los registros con Tipo_cuenta igual a "TOTAL"
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != 'TOTAL']
df_libro_mayor_old2_piv2 = df_libro_mayor_old2_piv2[df_libro_mayor_old2_piv2['Tipo_cuenta'] != '']

df_libro_mayor_old2_piv2.sample(5)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
9,TOTAL,1. INGRESO,2021-09-01 00:00:00,2021,9,138688902,66847588,71841314
76,TOTAL,4. COSTO SUPERVISIÓN,2021-04-01 00:00:00,2021,4,0,2990002,-2990002
17,TOTAL,1. INGRESO,2022-05-01 00:00:00,2022,5,162679537,53148947,109530590
38,TOTAL,2. COSTO MATERIALES,2022-02-01 00:00:00,2022,2,1368432,9480935,-8112503
1,TOTAL,1. INGRESO,2021-01-01 00:00:00,2021,1,153082726,51479131,101603595


##### 3.5.9.1 - Se anexa el dataframe creado con los totales con el dataframe creado a partir del punto 3.1.6.1 que contiene los costos de supervisión

In [142]:
df_libro_mayor_old2_piv_fin = pd.concat([df_libro_mayor_old2_piv2, df_libro_mayor_old2_2_piv], ignore_index=True)
df_libro_mayor_old2_piv_fin.sample(10)

,CODIGO_PROYECTO,Tipo_cuenta,Fecha2,Año,Mes,Credito_2,Debito_2,TOTAL
33,TOTAL,2. COSTO MATERIALES,2021-10-01 00:00:00,2021,10,1846393,21297701,-19451308
78,TOTAL,4. COSTO SUPERVISIÓN,2021-07-01 00:00:00,2021,7,0,2784855,-2784855
104,TOTAL,OTRA,2022-12-01 00:00:00,2022,12,0,7252501,-7252501
53,TOTAL,3. COSTO MANO DE OBRA,2021-06-01 00:00:00,2021,6,0,807337,-807337
21,TOTAL,1. INGRESO,2022-10-01 00:00:00,2022,10,368845820,102127936,266717884
14,TOTAL,1. INGRESO,2022-03-01 00:00:00,2022,3,158594333,101303947,57290386
103,TOTAL,OTRA,2022-10-01 00:00:00,2022,10,0,9743100,-9743100
19,TOTAL,1. INGRESO,2022-08-01 00:00:00,2022,8,432805709,235626646,197179063
58,TOTAL,3. COSTO MANO DE OBRA,2021-11-01 00:00:00,2021,11,0,299728,-299728
91,TOTAL,OTRA,2021-08-01 00:00:00,2021,8,0,10668933,-10668933


#### 3.5.10 - Se concatenan los 2 dataframes resultantes

In [143]:
print("=======================================================================")
print("Dimensiones del dataframe antes de la concatenación : ")
print(df_libro_mayor_old2_piv_fin.shape)

Dimensiones del dataframe antes de la concatenación : 
(109, 8)


In [144]:
df_libro_mayor_old3 = pd.concat([df_libro_mayor_old2_piv, df_libro_mayor_old2_piv_fin])
df_libro_mayor_old3.sort_values(by=['CODIGO_PROYECTO']).sample(5)
print(df_libro_mayor_old3.shape)

(1741, 8)


#### 3.5.11 - Estandarización de campos 

In [145]:
# Se estandarizan los campos con sus nombres y formatos, para un mayor entendimiento.

df_libro_mayor_old3 = df_libro_mayor_old3.rename(columns={
    'Codigo_proyecto': 'CODIGO_PROYECTO',
    'Tipo_cuenta': 'TIPO_CUENTA',
    'Fecha2': 'FECHA',
    'Año': 'AÑO',
    'Mes': 'MES',
    'Credito_2': 'CREDITO',
    'Debito_2': 'DEBITO'
})

df_libro_mayor_old3['AÑO'] = pd.to_numeric(df_libro_mayor_old3['AÑO'], downcast='integer')
df_libro_mayor_old3['MES'] = pd.to_numeric(df_libro_mayor_old3['MES'], downcast='integer')

df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL
202,P-1638,2. COSTO MATERIALES,2022-04-01 00:00:00,2022.00,4.00,0,8900,-8900
871,P-1672,2. COSTO MATERIALES,2021-09-01 00:00:00,2021.00,9.00,509014,2020908,-1511894
1498,P-1702,1. INGRESO,2022-05-01 00:00:00,2022.00,5.00,28137774,28137774,0
632,P-1664,3. COSTO MANO DE OBRA,2021-05-01 00:00:00,2021.00,5.00,0,0,0
140,P-1626,2. COSTO MATERIALES,2022-11-01 00:00:00,2022.00,11.00,0,45576,-45576
222,P-1639,2. COSTO MATERIALES,2021-01-01 00:00:00,2021.00,1.00,0,2943174,-2943174
1365,P-1693,3. COSTO MANO DE OBRA,2022-05-01 00:00:00,2022.00,5.00,0,3795578,-3795578
1246,P-1689,1. INGRESO,2022-04-01 00:00:00,2022.00,4.00,1223496,1198700,24796
229,P-1639,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,0,10000,-10000
966,P-1676,1. INGRESO,2021-06-01 00:00:00,2021.00,6.00,28376803,28376803,0


#### 3.5.14 Creación de columna MARGEN

In [146]:
df_libro_mayor_old3_2 = df_libro_mayor_old3.copy()

#df_libro_mayor4_2['TOTAL'] = pd.to_numeric(df_libro_mayor4_2['AÑO'], downcast='float')

df_margen = df_libro_mayor_old3_2.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_1637/3531166699.py:6: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['INGRESO','COSTO SUPERVISIÓN', 'COSTO MANO DE OBRA', 'COSTO MATERIALES']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == 'INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,MES,MARGEN
623,P-1702,7.00,0.00
278,P-1661,8.00,0.00
68,P-1626,1.00,0.00
525,P-1690,2.00,0.00
279,P-1661,9.00,0.00
12,P-1527,3.00,0.00
25,P-1610,2.00,0.00
665,P-1714,9.00,0.00
684,TOTAL,3.00,0.00
657,P-1712,9.00,0.00


In [147]:
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_libro_mayor_old3.shape)
print("=======================================================================")
print("Dimensiones del dataframe antes del merge : ")
print(df_margen.shape)

Dimensiones del dataframe antes del merge : 
(1741, 8)
Dimensiones del dataframe antes del merge : 
(694, 3)


In [148]:
df_libro_mayor_old3 = pd.merge(df_libro_mayor_old3, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')
#EJEMPLO
#df_libro_mayor5[df_libro_mayor5["CODIGO_PROYECTO"]=="P-1706"]
df_libro_mayor_old3[df_libro_mayor_old3["CODIGO_PROYECTO"]=="TOTAL"]

# Se estandariza el valor TOTAL a 1.TOTAL para que pueda ser seleccionado desde los filtros del Dashboard.
df_libro_mayor_old3['CODIGO_PROYECTO'] = df_libro_mayor_old3['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

In [149]:
print("=======================================================================")
print("Dimensiones del dataframe después de la concatenación : ")
print(df_libro_mayor_old3.shape)

Dimensiones del dataframe después de la concatenación : 
(1741, 9)


In [150]:
df_libro_mayor_old3.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1091,P-1680,3. COSTO MANO DE OBRA,2022-06-01 00:00:00,2022.00,6.00,0,2188398,-2188398,0.00
1048,P-1677,3. COSTO MANO DE OBRA,2022-06-01 00:00:00,2022.00,6.00,0,1483640,-1483640,0.00
845,P-1671,2. COSTO MATERIALES,2022-01-01 00:00:00,2022.00,1.00,105300,0,105300,0.00
433,P-1653,2. COSTO MATERIALES,2021-02-01 00:00:00,2021.00,2.00,0,482621,-482621,0.00
283,P-1644,2. COSTO MATERIALES,2021-02-01 00:00:00,2021.00,2.00,26781,869280,-842499,0.00
1040,P-1677,2. COSTO MATERIALES,2022-12-01 00:00:00,2022.00,12.00,0,1800252,-1800252,0.00
1187,P-1685,3. COSTO MANO DE OBRA,2022-05-01 00:00:00,2022.00,5.00,0,1013320,-1013320,0.00
537,P-1660,2. COSTO MATERIALES,2021-06-01 00:00:00,2021.00,6.00,333933,979187,-645254,0.00
1045,P-1677,3. COSTO MANO DE OBRA,2022-03-01 00:00:00,2022.00,3.00,0,620874,-620874,0.00
1292,P-1691,1. INGRESO,2021-12-01 00:00:00,2021.00,12.00,10540797,10540797,0,0.00


In [151]:
df_libro_mayor_old3.to_csv('archivos_respaldo/df_libro_mayor_old3.csv', mode='a')

### 3.6 TRATAMIENTO DE "PLANILLA PRESUPUESTO"

In [152]:
# Se crea una copia del dataframe llamado "presupuesto"
df_presupuesto2 = df_presupuesto.copy()

In [153]:
# Eliminar primera fila completa
df_presupuesto2 = df_presupuesto2.iloc[1:]

# Filtrar y mantener solo las columnas deseadas
df_presupuesto2 = df_presupuesto2[['PROYECTO', 'MONTO EN $']]

df_presupuesto2 = df_presupuesto2.rename(columns={'PROYECTO': 'CODIGO_PROYECTO', 'MONTO EN $': 'VALOR'})

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR
44,P-1700,149471447.02
43,P-1699,406744605.61
37,P-1693,141116486.00
56,P-1714,61454685.57
9,P-1661,48586986.00
14,P-1670,202553946.00
32,P-1688,50214086.81
6,P-1655,51593018.53
64,P-1722,286693424.50
18,S-1601,16486000.00


In [154]:
# Se agregan nuevas columnas tipo de cuenta y Origen.

df_presupuesto2["TIPO_CUENTA"]= "1. INGRESO"
df_presupuesto2["ORIGEN"]= "PLANILLA PRESUPUESTO"

df_presupuesto2.sample(10)

,CODIGO_PROYECTO,VALOR,TIPO_CUENTA,ORIGEN
46,P-1702,0.00,1. INGRESO,PLANILLA PRESUPUESTO
70,P-1728,7874868.00,1. INGRESO,PLANILLA PRESUPUESTO
54,P-1712,215321946.83,1. INGRESO,PLANILLA PRESUPUESTO
66,P-1724,28935271.00,1. INGRESO,PLANILLA PRESUPUESTO
9,P-1661,48586986.00,1. INGRESO,PLANILLA PRESUPUESTO
57,P-1715,335709882.43,1. INGRESO,PLANILLA PRESUPUESTO
29,P-1685,153894009.96,1. INGRESO,PLANILLA PRESUPUESTO
21,P-1677,83882285.07,1. INGRESO,PLANILLA PRESUPUESTO
5,P-1654,23880352.49,1. INGRESO,PLANILLA PRESUPUESTO
44,P-1700,149471447.02,1. INGRESO,PLANILLA PRESUPUESTO


### 3.7 TRATAMIENTO DE "TABLA PRESUPUESTO"

In [155]:
# Se crea una copia del dataframe para realizar el tratamiento.

df_tabla_presupuesto2 = df_tabla_prespuesto.copy()
print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.sample(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13826, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
11803,P-1715,1201,Cerco Electrico-Energizadores,UNIDA,4,178286,713144
10408,P-1702,1104,TC-COPLA SC/APC-SC/APC,,110,226,24860
3470,P-1633,1099,CCTV IP-Cámara Bullet Dahua IPC-HFW2331RN-VFS-...,,1,65129,65129
7094,P-1670,1054,Intrusion-Modulo Ethernet DSC TL280,,1,140000,140000
10953,P-1708,1085,TC-Marcador,,8500,133,1130500


In [156]:
# Se estandarizan los tipos de campo y se reemplazan valores NaN

df_tabla_presupuesto2['COSTO_UNITARIO'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_UNITARIO'], errors='coerce')
df_tabla_presupuesto2['COSTO_TOTAL'] = pd.to_numeric(df_tabla_presupuesto2['COSTO_TOTAL'], errors='coerce')

# Se reemplaza el valor de los NoNe y Nan 

df_tabla_presupuesto2['COSTO_UNITARIO'] = df_tabla_presupuesto2['COSTO_UNITARIO'].fillna(0)
df_tabla_presupuesto2['COSTO_TOTAL'] = df_tabla_presupuesto2['COSTO_TOTAL'].fillna(0)

print("DIMENSIONES DATAFRAME df_tabla_prespuesto2", df_tabla_presupuesto2.shape)
df_tabla_presupuesto2.head(5)

DIMENSIONES DATAFRAME df_tabla_prespuesto2 (13826, 7)


,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00


In [157]:
#### Creación de campo tipo_costo basado en el valor del a columna "PRODUCTO"

def clasificar_tipo_costo(producto):
    if   producto.upper().startswith("MANO DE OBRA"):
        return "3. COSTO MANO DE OBRA"
    elif producto.upper().startswith("SUPER"):
        return "4. COSTO SUPERVISIÓN"
    else:
        return "2. COSTO MATERIALES"

df_tabla_presupuesto2['TIPO_COSTO'] = df_tabla_presupuesto2['PRODUCTO'].apply(clasificar_tipo_costo)

df_tabla_presupuesto2.head(5)

,CODIGO_PROYECTO,PARTIDA_PRESUPUESTARIA,PRODUCTO,UNIDAD,CANTIDAD,COSTO_UNITARIO,COSTO_TOTAL,TIPO_COSTO
0,P-1602,1001,DETECCIÓN-Central de detección Hochiki FNP112...,,1,428490.00,428490.00,2. COSTO MATERIALES
1,P-1602,1002,DETECCIÓN-Sensor Humo Hochiki ALN-C,,49,20866.00,1022434.00,2. COSTO MATERIALES
2,P-1602,1003,DETECCIÓN-Sensor de temperatura Hochiki ATJ-EA,,4,23846.00,95384.00,2. COSTO MATERIALES
3,P-1602,1004,DETECCIÓN-Base Sensores Hochiki HSB-NSA-6,,53,2608.00,138224.00,2. COSTO MATERIALES
4,P-1602,1005,DETECCIÓN-Palanca Manual Hochiki DCP-AMS,,13,27200.00,353600.00,2. COSTO MATERIALES


In [158]:
# crear pivot table sin margenes

df_tabla_presupuesto3 = pd.pivot_table(df_tabla_presupuesto2, values=['COSTO_TOTAL'], index=['CODIGO_PROYECTO','TIPO_COSTO'], aggfunc='sum', fill_value=0)

df_tabla_presupuesto3 = pd.DataFrame(df_tabla_presupuesto3.to_records())
df_tabla_presupuesto3.sample(10)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL
386,S-1601,3. COSTO MANO DE OBRA,3702296.00
204,P-1670,2. COSTO MATERIALES,77955381.00
165,P-1657,2. COSTO MATERIALES,71831404.00
171,P-1659,2. COSTO MATERIALES,21300512.00
287,P-1699,2. COSTO MATERIALES,156685807.00
77,P-1627,4. COSTO SUPERVISIÓN,1355789.00
274,P-1693,4. COSTO SUPERVISIÓN,2506554.00
301,P-1704,3. COSTO MANO DE OBRA,156095149.00
132,P-1646,2. COSTO MATERIALES,13088304.00
84,P-1630,2. COSTO MATERIALES,109872802.00


In [159]:

# Se crea columna VALOR y se considera el valor en negativo.
df_tabla_presupuesto3['TOTAL'] = df_tabla_presupuesto3['COSTO_TOTAL'].apply(lambda x: x * -1 if x != 0 else 0)

# Se crea la columna de ORIGEN del archivo
df_tabla_presupuesto3["ORIGEN"] = "TABLA_PRESUPUESTO"

print("DIMENSIÓN DE LA TABLA ES", df_tabla_presupuesto3.shape)
df_tabla_presupuesto3.sample(5)

DIMENSIÓN DE LA TABLA ES (388, 5)


,CODIGO_PROYECTO,TIPO_COSTO,COSTO_TOTAL,TOTAL,ORIGEN
195,P-1667,2. COSTO MATERIALES,36944607.00,-36944607.00,TABLA_PRESUPUESTO
285,P-1698,3. COSTO MANO DE OBRA,6815995.00,-6815995.00,TABLA_PRESUPUESTO
210,P-1672,2. COSTO MATERIALES,44333167.00,-44333167.00,TABLA_PRESUPUESTO
300,P-1704,2. COSTO MATERIALES,387816472.00,-387816472.00,TABLA_PRESUPUESTO
294,P-1701,4. COSTO SUPERVISIÓN,3113372.00,-3113372.00,TABLA_PRESUPUESTO


In [160]:
# INSPECCIÓN VISUAL
df_tabla_presupuesto_testing = pd.pivot_table(df_tabla_presupuesto3, values=['TOTAL'], index=['TIPO_COSTO'], aggfunc='sum', fill_value=0)
df_tabla_presupuesto_testing

,TOTAL
TIPO_COSTO,
2. COSTO MATERIALES,-5745141472.04
3. COSTO MANO DE OBRA,-1482885459.00
4. COSTO SUPERVISIÓN,-311477854.00


In [161]:
df_tabla_presupuesto3.to_csv('archivos_respaldo/df_tabla_presupuesto3.csv', mode='a')

## 4.- **MERGE Y TRATAMIENTO DE DATAFRAMES 

### 4.1 - **MERGE PARA -> CUADRO FINANCIERO PROYECTOS 
Merge entre Dataframes: df_libro_mayor5 y df_cod_proyecto

In [162]:
print("Dimensiones antes del merge:")
print(df_libro_mayor5.shape)

df_libro_mayor6 = pd.merge(df_libro_mayor5, df_cod_proyecto, on=['CODIGO_PROYECTO'], how='left')

print("Dimensiones después del merge:")
print(df_libro_mayor6.shape)

Dimensiones antes del merge:
(2614, 9)
Dimensiones después del merge:
(2614, 11)


In [163]:
# Reemplazar los valores NaN por "TOTAL" cuando el CODIGO_PROYECTO sea "TOTAL"

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_libro_mayor6[cols_to_replace] = df_libro_mayor6[cols_to_replace].replace('TOTAL', '1. TOTAL')

# Quizás el código anterior sea redundante.
mask = df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'
df_libro_mayor6.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Se elimina la columna MERGE la que es innecesaria tener hasta ahora
df_libro_mayor6 = df_libro_mayor6.drop('MARGEN', axis=1)

df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]== "1. TOTAL"].head(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
551,1. TOTAL,,,NaN,NaN,2281381657,1909051349,372330308,1. TOTAL,1. TOTAL
552,1. TOTAL,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,72482994,-72482994,1. TOTAL,1. TOTAL
553,1. TOTAL,COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,82645544,-82645544,1. TOTAL,1. TOTAL
554,1. TOTAL,COSTO MANO DE OBRA,2023-03-01 00:00:00,2023.00,3.00,0,96760244,-96760244,1. TOTAL,1. TOTAL
555,1. TOTAL,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,111885468,-111885468,1. TOTAL,1. TOTAL


#### 4.1.1 - AGREGACIÓN DE NUEVA COLUMNA DE TOTAL PARA AGREGAR EN GRÁFICO.
Se agrega nueva columna de total pero considerando el valor de los tipos de costo con valores positivos.

In [164]:
# Crear la nueva columna 'TOTAL_2' con los valores de 'TOTAL' positivos
df_libro_mayor6['TOTAL_2'] = np.where(df_libro_mayor6['TIPO_CUENTA'] != 'INGRESO', np.abs(df_libro_mayor6['TOTAL']), df_libro_mayor6['TOTAL'])
df_libro_mayor6.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
833,P-1639,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,Lyon 2546,Moller y Perez - Cotapos Construcciones Indust...,0
2472,P-1725,COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,Lo Cañas,Constructora Brisas de Batuco,0
1769,P-1692,COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,José Perez Cotapos,EBCO S.A.,0
1137,P-1663,COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,Oficinas Equifax,Constructora VRK Ingenieria Limitada,0
683,P-1611,COSTO MATERIALES,2023-02-01 00:00:00,2023.00,2.00,0,0,0,Edificio Plaza Egaña,EBCO S.A.,0
2242,P-1715,INGRESO,2023-02-01 00:00:00,2023.00,2.00,0,0,0,CD Lo Echevers,Quezada y Boetsch,0
1160,P-1664,COSTO MANO DE OBRA,2023-01-01 00:00:00,2023.00,1.00,0,0,0,MK II PCI,Comercial K SpA.,0
955,P-1645,COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada,0
351,P-1706,COSTO MATERIALES,2023-05-01 00:00:00,2023.00,5.00,0,5198516,-5198516,California,CONSTRUCTORA E INVERSIONES VITAL SPA.,5198516
157,P-1688,COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,2773325,-2773325,Suiza,Moller y Perez - Cotapos Construcciones Indust...,2773325


#### 4.1.2 - AJUSTES DE COLUMNAS 

In [165]:
# Renombrar los valores de la columna CODIGO_PROYECTO
df_libro_mayor6['TIPO_CUENTA'] = df_libro_mayor6['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})

df_libro_mayor6['CODIGO_PROYECTO'] = df_libro_mayor6['CODIGO_PROYECTO'].replace({
    'TOTAL': '1. TOTAL'})

In [166]:
# Ordenar el DataFrame por la columna CODIGO_PROYECTO
df_libro_mayor6 = df_libro_mayor6.sort_values('CODIGO_PROYECTO')

# Mover la fila con CODIGO_PROYECTO = '1. TOTAL' al primer lugar
df_libro_mayor6 = pd.concat([df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] == '1. TOTAL'],
                            df_libro_mayor6[df_libro_mayor6['CODIGO_PROYECTO'] != '1. TOTAL']])

df_libro_mayor6['NOMBRE_OBRA'] = df_libro_mayor6['NOMBRE_OBRA'].str.upper()
df_libro_mayor6['NOMBRE_CLIENTE'] = df_libro_mayor6['NOMBRE_CLIENTE'].str.upper()

# Reiniciar los índices del DataFrame
df_libro_mayor6 = df_libro_mayor6.reset_index(drop=True)
df_libro_mayor6.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2
1214,P-1686,2. COSTO MATERIALES,2023-04-01 00:00:00,2023.00,4.00,0,4886851,-4886851,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,4886851
592,P-1660,3. COSTO MANO DE OBRA,2023-04-01 00:00:00,2023.00,4.00,0,0,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0
1241,P-1686,4. COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,0
590,P-1660,1. INGRESO,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0
1539,P-1695,3. COSTO MANO DE OBRA,2023-02-01 00:00:00,2023.00,2.00,0,1534891,-1534891,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,1534891


#### 4.1.3 - CREACIÓN DE COLUMNA MARGEN

In [167]:
df_margen = df_libro_mayor6.groupby(['CODIGO_PROYECTO', 'MES']).apply(lambda x: pd.Series({
    'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
}))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

#warnings.filterwarnings("ignore", category=FutureWarning, message="Inferring datetime64.*")

# mostrar el DataFrame resultante
df_margen.sample(10)

df_libro_mayor6 = pd.merge(df_libro_mayor6, df_margen, on=['CODIGO_PROYECTO', 'MES'], how='left')

# INSPECCIÓN VISUAL 1
df_libro_mayor6[df_libro_mayor6["CODIGO_PROYECTO"]=="P-1704"]


/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_1637/1823500159.py:2: RuntimeWarning: invalid value encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
/var/folders/26/wv7th2z157sgk1cmh3h23d440000gn/T/ipykernel_1637/1823500159.py:2: RuntimeWarning: divide by zero encountered in scalar divide
  'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA','4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_2,MARGEN
1803,P-1704,1. INGRESO,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1804,P-1704,4. COSTO SUPERVISIÓN,2023-05-01 00:00:00,2023.00,5.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.66
1805,P-1704,1. INGRESO,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1806,P-1704,2. COSTO MATERIALES,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1807,P-1704,4. COSTO SUPERVISIÓN,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1808,P-1704,3. COSTO MANO DE OBRA,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1809,P-1704,2. COSTO MATERIALES,2023-06-01 00:00:00,2023.00,6.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
1810,P-1704,4. COSTO SUPERVISIÓN,2023-03-01 00:00:00,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04
1811,P-1704,4. COSTO SUPERVISIÓN,2023-01-01 00:00:00,2023.00,1.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.25
1812,P-1704,2. COSTO MATERIALES,2023-03-01 00:00:00,2023.00,3.00,0,0,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.04


In [168]:
df_libro_mayor6.to_csv('archivos_respaldo/df_libro_mayor6.csv', mode='a')

### 4.2 - **MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO
MERGE ENTRE df_libro_mayor_ytd_closed_piv y df_cod_proyecto

In [169]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_piv.shape

Dimensiones antes del merge:


(340, 3)

In [170]:
df_ytd_cerrado = df_libro_mayor_ytd_closed_piv.copy()

In [171]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones mes_en_curso antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado = df_ytd_cerrado.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones mes_en_curso2 antes de cambios: ", df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

Dimensiones mes_en_curso antes de cambios:  (340, 3)
Dimensiones mes_en_curso2 antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
230,P-1699,COSTO SUPERVISIÓN,0,Villa Panamericana Lote B,Empresa Constructora DLP S.A.
85,P-1658,COSTO MATERIALES,47401,VIVIPRA,Vicherat y Pradenas Ltda.
77,P-1653,COSTO MATERIALES,9000,Eduardo de la Barra,CONSTRUCTORA E INVERSIONES VITAL SPA.
223,P-1697,INGRESO,53923452,Estocolmo,Constructora Fortaleza S.A
300,P-1720,COSTO MANO DE OBRA,-65665,NaN,NaN
298,P-1719,COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.
94,P-1661,COSTO SUPERVISIÓN,0,San Vicente de Paul,DANACORP S.A
294,P-1718,COSTO SUPERVISIÓN,0,Patio Comidas,EBCO S.A.
337,P-1729,COSTO MATERIALES,0,NaN,NaN
129,P-1673,COSTO MATERIALES,68000,Inacap Temuco - Etapa A,Universidad Tecnologica de Chile Inacap


In [172]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado['TIPO_CUENTA'] = df_ytd_cerrado['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
279,P-1714,1. INGRESO,20583182,Departamental II,Constructora Fuerza Sur Dos Spa
159,P-1681,1. INGRESO,765714,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC
298,P-1719,4. COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.
250,P-1705,4. COSTO SUPERVISIÓN,0,Los Clarines,CONSTRUCTORA INARCO S.A.
116,P-1669,3. COSTO MANO DE OBRA,-156861,Departamental,EMP. CONSTRUCTORA INGENIEROS S.A.
121,P-1670,2. COSTO MATERIALES,6945,MK II Seguridad y Redes,Comercial K SpA.
258,P-1707,4. COSTO SUPERVISIÓN,0,Las Tranqueras,Constructora Santolaya Ltda.
162,P-1682,4. COSTO SUPERVISIÓN,0,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.
299,P-1719,1. INGRESO,131348,CD METALPAR,EBCO S.A.
306,P-1721,4. COSTO SUPERVISIÓN,0,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.


In [173]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-605662121,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,NaN,NaN
3,1. TOTAL,1. INGRESO,1579362409,NaN,NaN


In [174]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado[cols_to_replace] = df_ytd_cerrado[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
151,P-1679,1. INGRESO,0,Juan Sabaj,Constructora Grevia
28,P-1620,3. COSTO MANO DE OBRA,0,"OF Apoquindo- Habilitaciones, Integracion y ad...",CONSTRUCTORA INARCO S.A.
289,P-1717,2. COSTO MATERIALES,0,Eco Quilín I,Constructora ICF SPA
135,P-1674,1. INGRESO,0,Oficinas Beiersdorf Chorrillos,Constructora Suksa S.A.


In [175]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado[df_ytd_cerrado["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605662121,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL


#### 4.2.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [176]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   : ", df_ytd_cerrado.shape)

df_ytd_cerrado2 = df_ytd_cerrado.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
238,P-1701,4. COSTO SUPERVISIÓN,0
240,P-1703,3. COSTO MANO DE OBRA,-9066189
63,P-1644,1. INGRESO,0
147,P-1677,1. INGRESO,2369790
91,P-1660,1. INGRESO,0


In [177]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
74,P-1691,1. INGRESO,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.
62,P-1687,3. COSTO MANO DE OBRA,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.
194,P-1632,4. COSTO SUPERVISIÓN,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.
312,P-1716,4. COSTO SUPERVISIÓN,0,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.


In [178]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado3 = df_libro_mayor5_base2.merge(df_ytd_cerrado2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
266,P-1677,4. COSTO SUPERVISIÓN,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0
94,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-21466214
99,P-1700,1. INGRESO,0,CD ORIZON,EBCO S.A.,16747965
39,P-1675,1. INGRESO,0,PALENA II,CONSTRUCTORA MYA SPA.,398016
267,P-1679,1. INGRESO,0,JUAN SABAJ,CONSTRUCTORA GREVIA,0


In [179]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605662121
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965


In [180]:
# INSPECCIÓN VISUAL 2

df_ytd_cerrado3[df_ytd_cerrado3["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066
154,P-1724,2. COSTO MATERIALES,0,NaN,NaN,-7710229
155,P-1724,1. INGRESO,0,NaN,NaN,28935273
327,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN,0


#### 4.2.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [181]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
9,P-1632,0.00
32,P-1673,0.00
66,P-1709,0.52
2,P-1591,0.00
83,P-1728,0.00
80,P-1725,0.28
10,P-1636,0.00
3,P-1592,0.00
38,P-1680,0.10
61,P-1704,0.41


In [182]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [183]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3 = df_ytd_cerrado3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 : ", df_ytd_cerrado3.shape)

df_ytd_cerrado3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
300,P-1701,4. COSTO SUPERVISIÓN,0,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,0,-0.44
89,P-1697,3. COSTO MANO DE OBRA,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-11388112,0.46
65,P-1688,3. COSTO MANO DE OBRA,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-10238516,0.15
327,P-1724,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,0.72
105,P-1703,1. INGRESO,0,SANTA VICTORIA DANACORP,DANACORP S.A,35296641,0.39


In [184]:
df_ytd_cerrado3.to_csv('archivos_respaldo/df_ytd_cerrado3.csv', index = False)

### 4.2-1 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO (PERSPECTIVA MES ANTERIOR)
MERGE ENTRE df_libro_mayor_ytd_closed_2M y df_cod_proyecto

In [185]:
print("Dimensiones antes del merge:")
df_libro_mayor_ytd_closed_2M.shape

Dimensiones antes del merge:


(340, 3)

In [186]:
df_ytd_cerrado_2M = df_libro_mayor_ytd_closed_2M.copy()

In [187]:
# Se realiza merge con Nombre de Obra y Cliente.

print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M = df_ytd_cerrado_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
print("Dimensiones df_ytd_cerrado_2M antes de cambios: ", df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 3)
Dimensiones df_ytd_cerrado_2M antes de cambios:  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
158,P-1681,COSTO SUPERVISIÓN,0,CD Rosen (CCTV Provisorio),Colchones ROSEN SAIC
249,P-1705,COSTO MATERIALES,-9939983,Los Clarines,CONSTRUCTORA INARCO S.A.
319,P-1724,INGRESO,28844553,NaN,NaN
139,P-1675,INGRESO,398016,Palena II,CONSTRUCTORA MYA SpA.
273,P-1713,COSTO MATERIALES,-1269662,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...
338,P-1729,COSTO SUPERVISIÓN,0,NaN,NaN
331,P-1727,INGRESO,11005000,NaN,NaN
32,P-1626,COSTO MANO DE OBRA,0,Clinica Bio Bio,Constructora LYD S.A.
42,P-1636,COSTO SUPERVISIÓN,0,Santo Domingo 1410,DANACORP S.A
212,P-1695,COSTO MANO DE OBRA,-5042385,Edificio Estoril,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO


In [188]:
# Se reemplazan nombres de TIPO_CUENTA

df_ytd_cerrado_2M['TIPO_CUENTA'] = df_ytd_cerrado_2M['TIPO_CUENTA'].replace({
    'INGRESO': '1. INGRESO',
    'COSTO MATERIALES': '2. COSTO MATERIALES',
    'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA',
    'COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'
})
df_ytd_cerrado_2M.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
27,P-1612,1. INGRESO,0,Barrio Parque Santiago 7,CONSTRUCTORA ALTURAS DEL SUR S.A
288,P-1717,3. COSTO MANO DE OBRA,0,Eco Quilín I,Constructora ICF SPA
252,P-1706,3. COSTO MANO DE OBRA,-4875121,California,CONSTRUCTORA E INVERSIONES VITAL SPA.
328,P-1727,3. COSTO MANO DE OBRA,-4714680,NaN,NaN
87,P-1658,1. INGRESO,0,VIVIPRA,Vicherat y Pradenas Ltda.
100,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,Comercial K SpA.
313,P-1723,2. COSTO MATERIALES,-10817986,Villa Panamericana Lote C,Constructora Mena y Ovalle S.A.
143,P-1676,1. INGRESO,0,Lo Campino,EBCO S.A.
145,P-1677,2. COSTO MATERIALES,-1949557,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...
258,P-1707,4. COSTO SUPERVISIÓN,0,Las Tranqueras,Constructora Santolaya Ltda.


In [189]:
# INSPECCIÓN VISUAL 1 
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,NaN,NaN
1,1. TOTAL,2. COSTO MATERIALES,-435882635,NaN,NaN
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,NaN,NaN
3,1. TOTAL,1. INGRESO,1118563896,NaN,NaN


In [190]:
#Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar en el dashboard.

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
df_ytd_cerrado_2M[cols_to_replace] = df_ytd_cerrado_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = df_ytd_cerrado['CODIGO_PROYECTO'] == '1. TOTAL'
df_ytd_cerrado_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'
df_ytd_cerrado_2M.sample(4)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
262,P-1708,4. COSTO SUPERVISIÓN,0,Eco España,Constructora ICF SPA
255,P-1706,1. INGRESO,18664851,California,CONSTRUCTORA E INVERSIONES VITAL SPA.
80,P-1656,3. COSTO MANO DE OBRA,0,Mirador Club Hípico,Constructora Vital Edificacion SPA
232,P-1700,3. COSTO MANO DE OBRA,-4264241,CD Orizon,EBCO S.A.


In [191]:
# INSPECCIÓN VISUAL 1
df_ytd_cerrado_2M[df_ytd_cerrado_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435882635,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896,1. TOTAL,1. TOTAL


#### 4.2-1.1 - MERGE CON "df_libro_mayor5_base2" PARA CREAR DATAFRAME FINAL

In [192]:
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   : ", df_ytd_cerrado_2M.shape)

df_ytd_cerrado_2M_2 = df_ytd_cerrado_2M.filter(["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"])

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 : ", df_ytd_cerrado_2M_2.shape)

df_ytd_cerrado_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
242,P-1703,4. COSTO SUPERVISIÓN,0
43,P-1636,1. INGRESO,0
151,P-1679,1. INGRESO,0
182,P-1687,4. COSTO SUPERVISIÓN,0
123,P-1670,1. INGRESO,0


In [193]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [194]:
# Exploración dataframe base

df_libro_mayor5_base2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
101,P-1701,2. COSTO MATERIALES,0,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.
310,P-1714,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
35,P-1672,1. INGRESO,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.
94,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.
206,P-1642,4. COSTO SUPERVISIÓN,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.


In [195]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

df_ytd_cerrado_2M_3 = df_libro_mayor5_base2.merge(df_ytd_cerrado_2M_2, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
5,P-1612,2. COSTO MATERIALES,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0
320,P-1720,2. COSTO MATERIALES,0,NaN,NaN,0
213,P-1645,3. COSTO MANO DE OBRA,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0
289,P-1692,4. COSTO SUPERVISIÓN,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,0


In [196]:
# INSPECCIÓN VISUAL 1

df_ytd_cerrado_2M_3[df_ytd_cerrado_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435882635
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250


#### 4.2-1.2 - SE REALIZA CREACIÓN DE COLUMNA MARGEN

In [197]:
# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_cerrado_2M_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES','3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       : ", df_margen.shape)

# mostrar el DataFrame resultante
df_margen.sample(10)

DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_margen       :  (85, 2)


,CODIGO_PROYECTO,MARGEN
84,P-1729,0.00
63,P-1706,0.32
71,P-1716,0.58
58,P-1700,0.28
54,P-1696,0.10
76,P-1721,0.30
56,P-1698,1.00
78,P-1723,-0.16
9,P-1632,0.00
77,P-1722,0.00


In [198]:
df_margen[df_margen["CODIGO_PROYECTO"]=="P-1724"]

,CODIGO_PROYECTO,MARGEN
79,P-1724,0.72


In [199]:
# Merge Dataframe para agregar MARGEN

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3 = df_ytd_cerrado_2M_3.merge(df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3   :  (340, 6)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
331,P-1728,1. INGRESO,0,NaN,NaN,0,0.00
244,P-1666,4. COSTO SUPERVISIÓN,0,CD AXIONLONG,EBCO S.A.,0,0.00
7,P-1626,2. COSTO MATERIALES,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460,0.00
21,P-1658,2. COSTO MATERIALES,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.,47401,0.00
203,P-1641,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0,0.00


In [200]:
df_ytd_cerrado_2M_3.to_csv('archivos_respaldo/df_ytd_cerrado_2M_3.csv', index = False)

### 4.3 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO 
(DATAFRAMES: df_libro_mayor5_mec2 y df_proyectado4)

#### 4.3.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec2" Y AJUSTES INICIALES

In [201]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec2.sample(5)
df_libro_mayor5_mec3 = df_libro_mayor5_mec2.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
398,P-1712,COSTO MATERIALES,2023-06-01,2023.00,6.00,-7219006,LIBRO MAYOR
1771,P-1692,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR
988,P-1649,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1732,P-1691,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
796,P-1636,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [202]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  : ", df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3[df_libro_mayor5_mec3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec3  :  (419, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
320,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883,LIBRO MAYOR
326,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,-16121702,LIBRO MAYOR
2020,P-1704,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2021,P-1704,COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2034,P-1704,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR
2035,P-1704,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR


In [203]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec3['DESTINO'] = df_libro_mayor5_mec3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3.sample(10)


(419, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
252,P-1697,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-4485402,LIBRO MAYOR,REAL Y PROYECTADO
156,P-1688,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-1621597,LIBRO MAYOR,REAL Y PROYECTADO
162,P-1688,COSTO MATERIALES,2023-06-01,2023.00,6.00,-648868,LIBRO MAYOR,REAL
1794,P-1693,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
940,P-1645,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1492,P-1681,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
700,P-1612,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
772,P-1632,COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1171,P-1664,COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
738,P-1620,INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL


In [204]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec3['TIPO_CUENTA'] = df_libro_mayor5_mec3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec3.shape)
df_libro_mayor5_mec3["TIPO_CUENTA"].unique()

(419, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES

In [205]:
# Ajuste del dataframe proveniente de PROYECTADO

df_proyectado4.sample(5)
df_proyectado5 = df_proyectado4.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5['FECHA'] = pd.to_datetime(df_proyectado5['MES'].astype(str) + '-1-' + df_proyectado5['AÑO'].astype(str))

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
39,P-1695,-3000000,6,2023,PROYECTADO,2. COSTO MATERIALES,2023-06-01
19,P-1671,0,6,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-06-01
61,P-1719,0,6,2023,PROYECTADO,1. INGRESO,2023-06-01
37,P-1693,0,6,2023,PROYECTADO,2. COSTO MATERIALES,2023-06-01
52,P-1708,0,6,2023,PROYECTADO,1. INGRESO,2023-06-01


In [206]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5['DESTINO'] = df_proyectado5.apply(determinar_origen, axis=1)
df_proyectado5 = df_proyectado5[df_proyectado5['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5 = df_proyectado5.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
60,P-1718,2. COSTO MATERIALES,2023-06-01,2023,6,-6000000,PROYECTADO,PROYECTADO
56,P-1714,2. COSTO MATERIALES,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO
54,P-1712,1. INGRESO,2023-06-01,2023,6,12500000,PROYECTADO,PROYECTADO
44,P-1700,2. COSTO MATERIALES,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO
17,P-1669,2. COSTO MATERIALES,2023-06-01,2023,6,0,PROYECTADO,PROYECTADO


#### 4.3.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO

In [207]:
# Se realiza concatenación entre ambos dataframes

print("Dimensiones df_libro_mayor5_mec3 antes de cambios: ", df_libro_mayor5_mec3.shape)
print("Dimensiones df_proyectado5 antes de cambios      : ", df_proyectado5.shape)

mes_en_curso = pd.concat([df_libro_mayor5_mec3, df_proyectado5])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso.shape)
mes_en_curso.sample(10)

Dimensiones df_libro_mayor5_mec3 antes de cambios:  (419, 8)
Dimensiones df_proyectado5 antes de cambios      :  (146, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (565, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
1157,P-1664,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
2404,P-1722,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1410,P-1676,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
439,P-1715,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-3013082,LIBRO MAYOR,REAL
31,P-1687,1. INGRESO,2023-06-01,2023.00,6.00,0,PROYECTADO,PROYECTADO
940,P-1645,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
300,P-1701,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,70215,LIBRO MAYOR,REAL
32,P-1688,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-1000000,PROYECTADO,PROYECTADO
49,P-1705,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-4500000,PROYECTADO,PROYECTADO
47,P-1703,1. INGRESO,2023-06-01,2023.00,6.00,0,PROYECTADO,PROYECTADO


#### 4.3.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [208]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso.shape)
mes_en_curso2 = mes_en_curso.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso2 = mes_en_curso2[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso2 antes de cambios: ", mes_en_curso2.shape)
mes_en_curso2.sample(10)

Dimensiones mes_en_curso antes de cambios:  (565, 8)
Dimensiones mes_en_curso2 antes de cambios:  (565, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
261,P-1688,Suiza,Moller y Perez - Cotapos Construcciones Indust...,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
354,P-1714,Departamental II,Constructora Fuerza Sur Dos Spa,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
216,P-1676,Lo Campino,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
312,P-1701,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
54,P-1715,CD Lo Echevers,Quezada y Boetsch,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-3013082,LIBRO MAYOR,REAL
276,P-1692,José Perez Cotapos,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
175,P-1664,MK II PCI,Comercial K SpA.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
59,P-1718,Patio Comidas,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-5751818,LIBRO MAYOR,REAL Y PROYECTADO
343,P-1712,Centro Comercial Viña Outlet Park,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
412,P-1729,NaN,NaN,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO


In [209]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso2.loc[(mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso2['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso2.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
523,P-1687,Rojas Magallanes,E.MOLINA MOREL CONSTRUCTORA S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,PROYECTADO,PROYECTADO
280,P-1693,Volta V0800,ECOBIO SPA,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
337,P-1708,Eco España,Constructora ICF SPA,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
128,P-1642,Corinto,EMP. CONSTRUCTORA INGENIEROS S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
453,P-1690,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.,1. INGRESO,2023-06-01,2023.00,6.00,5000000,PROYECTADO,PROYECTADO


In [210]:
# INSPECCIÓN VISUAL 1

mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
35,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,-12890883,LIBRO MAYOR,REAL Y PROYECTADO
36,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-16121702,LIBRO MAYOR,REAL
319,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
320,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
321,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
322,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
467,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,62000000,PROYECTADO,PROYECTADO
540,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,-27000000,PROYECTADO,PROYECTADO


#### 4.3.5 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [211]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso2.shape)

# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso2.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

# concatenar nuevas filas al DataFrame original
mes_en_curso2 = pd.concat([mes_en_curso2, pd.DataFrame(df_rows_to_add)], ignore_index=True)

# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso2['CODIGO_PROYECTO'] = mes_en_curso2['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso2[cols_to_replace] = mes_en_curso2[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso2['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso2.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

# Cambiar los valores de la columna TIPO_CUENTA

print("Dimensiones Dataframe después de cambios", mes_en_curso2.shape)


Dimensiones Dataframe antes de cambios (565, 10)
Dimensiones Dataframe después de cambios (1352, 10)


In [212]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [213]:
# INSPECCIÓN VISUAL 1
mes_en_curso2[mes_en_curso2["CODIGO_PROYECTO"]=="P-1644"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
135,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
136,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL Y PROYECTADO
137,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
138,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,LIBRO MAYOR,REAL
663,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
664,P-1644,Mirador de San Cristobal,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
665,P-1644,Mirador de San Cristobal,EBCO S.A.,3. COSTO MANO DE OBRA,2023-05-01,2023.00,5.00,0,AGREGACION,REAL Y PROYECTADO
666,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,AGREGACION,REAL
667,P-1644,Mirador de San Cristobal,EBCO S.A.,1. INGRESO,2023-06-01,2023.00,6.00,0,AGREGACION,PROYECTADO
668,P-1644,Mirador de San Cristobal,EBCO S.A.,2. COSTO MATERIALES,2023-06-01,2023.00,6.00,0,AGREGACION,REAL


In [214]:
mes_en_curso2["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.3.6 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [215]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_proy = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy = pd.DataFrame(mes_en_curso_proy.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  : ", mes_en_curso_proy.shape)

mes_en_curso_proy.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1352, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (873, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (873, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy  :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
216,P-1691,1. INGRESO,0
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261
269,P-1704,4. COSTO SUPERVISIÓN,0
262,P-1703,1. INGRESO,0
279,P-1707,2. COSTO MATERIALES,-3000000
317,P-1718,4. COSTO SUPERVISIÓN,0
296,P-1713,3. COSTO MANO DE OBRA,-1005445
101,P-1658,2. COSTO MATERIALES,0
95,P-1655,2. COSTO MATERIALES,0
38,P-1621,1. INGRESO,0


In [216]:
# TESTING VISUAL 1
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
290,P-1712,1. INGRESO,12500000
291,P-1712,2. COSTO MATERIALES,-7700000
292,P-1712,3. COSTO MANO DE OBRA,-5436047
293,P-1712,4. COSTO SUPERVISIÓN,0


In [217]:
# TESTING VISUAL 2
mes_en_curso_proy[mes_en_curso_proy["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,1. INGRESO,423800000
1,1. TOTAL,2. COSTO MATERIALES,-176535806
2,1. TOTAL,3. COSTO MANO DE OBRA,-114373715
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261


##### 4.3.6.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [218]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   : ", mes_en_curso_proy.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy2 = df_libro_mayor5_base2.merge(mes_en_curso_proy, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       : ", mes_en_curso_proy2.shape)

mes_en_curso_proy2['TOTAL'] = mes_en_curso_proy2['TOTAL'].fillna(0)

mes_en_curso_proy2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy   :  (362, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy2       :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
90,P-1697,2. COSTO MATERIALES,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-1000000
131,P-1714,2. COSTO MATERIALES,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0
319,P-1719,4. COSTO SUPERVISIÓN,0,CD METALPAR,EBCO S.A.,0
34,P-1672,2. COSTO MATERIALES,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,0
89,P-1697,3. COSTO MANO DE OBRA,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-4485402


In [219]:
# TESTING VISUAL 1

mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7700000
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,12500000
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [220]:
# TESTING VISUAL 2
mes_en_curso_proy2[mes_en_curso_proy2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


##### 4.3.6.2 - SE RECALCULAN LOS TOTALES

In [221]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy2 = mes_en_curso_proy2.drop(mes_en_curso_proy2[(mes_en_curso_proy2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 : ", mes_en_curso_proy2.shape)

df_total = mes_en_curso_proy2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy3 = pd.concat([mes_en_curso_proy2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy3.loc[mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
95,P-1699,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-22000000
231,P-1660,3. COSTO MANO DE OBRA,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0
256,P-1673,1. INGRESO,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0
220,P-1649,4. COSTO SUPERVISIÓN,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0
44,P-1677,1. INGRESO,0.00,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0
25,P-1664,2. COSTO MATERIALES,0.00,MK II PCI,COMERCIAL K SPA.,0
126,P-1712,1. INGRESO,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,12500000
297,P-1698,4. COSTO SUPERVISIÓN,0.00,CD ANDES,EBCO S.A.,0
19,P-1653,2. COSTO MATERIALES,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0
92,P-1698,2. COSTO MATERIALES,0.00,CD ANDES,EBCO S.A.,0


In [222]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261


In [223]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3 = mes_en_curso_proy3.drop(mes_en_curso_proy3[(mes_en_curso_proy3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 : ", mes_en_curso_proy3.shape)

mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


In [224]:
# TESTING VISUAL 1
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,75000000
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0


In [225]:
# TESTING VISUAL 2
mes_en_curso_proy3[mes_en_curso_proy3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


##### 4.3.6.3 - CREACIÓN DE MARGEN

In [226]:
#CREACIÓN DE MARGENES

mes_en_curso_proy4 = mes_en_curso_proy3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
6,P-1612,0.00
46,P-1688,-0.05
5,P-1611,0.00
76,P-1721,-0.03
69,P-1714,0.00
44,P-1686,-0.15
51,P-1693,0.00
48,P-1690,0.47
39,P-1681,0.00
79,P-1724,0.00


In [227]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 : ", mes_en_curso_proy4.shape)

mes_en_curso_proy5 = pd.merge(mes_en_curso_proy4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 : ", mes_en_curso_proy5.shape)
mes_en_curso_proy5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
124,P-1712,3. COSTO MANO DE OBRA,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047,-0.05
263,P-1675,4. COSTO SUPERVISIÓN,0.00,PALENA II,CONSTRUCTORA MYA SPA.,0,0.00
304,P-1706,4. COSTO SUPERVISIÓN,0.00,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0,0.35
320,P-1720,2. COSTO MATERIALES,0.00,NaN,NaN,0,0.00
88,P-1696,1. INGRESO,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0,0.00
48,P-1680,2. COSTO MATERIALES,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,-5000000,0.15
141,P-1718,2. COSTO MATERIALES,0.00,PATIO COMIDAS,EBCO S.A.,-6000000,0.40
227,P-1658,1. INGRESO,0.00,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0,0.00
236,P-1663,1. INGRESO,0.00,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0,0.00
222,P-1653,3. COSTO MANO DE OBRA,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0,0.00


In [228]:
# TESTING VISUAL 1
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-15564904,0.38
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-31000000,0.38
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,75000000,0.38
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0,0.38


In [229]:
# TESTING VISUAL 2
mes_en_curso_proy5[mes_en_curso_proy5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261,0.30
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,423800000,0.30
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-176535806,0.30
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715,0.30


In [230]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy5.to_csv('archivos_respaldo/mes_en_curso_proy5.csv', mode='a')

#### 4.3.7 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO

##### 4.3.7.1 - TRATAMIENTO mes_en_curso2

In [231]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       : ", mes_en_curso2.shape)

mes_en_curso2_1 = mes_en_curso2.copy()
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'].str.contains('REAL')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   : ", mes_en_curso2_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     : ", mes_en_curso2_1.shape)

mes_en_curso_real = pd.pivot_table(mes_en_curso2_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real = pd.DataFrame(mes_en_curso_real.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  : ", mes_en_curso_real.shape)

mes_en_curso_real.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2       :  (1352, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1   :  (940, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso2_1     :  (940, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real  :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
291,P-1712,2. COSTO MATERIALES,-7219006
171,P-1679,4. COSTO SUPERVISIÓN,0
91,P-1653,4. COSTO SUPERVISIÓN,0
306,P-1716,1. INGRESO,0
322,P-1720,1. INGRESO,0
214,P-1690,3. COSTO MANO DE OBRA,-652512
235,P-1695,4. COSTO SUPERVISIÓN,0
153,P-1674,4. COSTO SUPERVISIÓN,0
340,P-1724,3. COSTO MANO DE OBRA,0
44,P-1632,1. INGRESO,0


In [232]:
# TESTING VISUAL 1

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
290,P-1712,1. INGRESO,-3823676
291,P-1712,2. COSTO MATERIALES,-7219006
292,P-1712,3. COSTO MANO DE OBRA,-5436047
293,P-1712,4. COSTO SUPERVISIÓN,0


In [233]:
# TESTING VISUAL 2

mes_en_curso_real[mes_en_curso_real["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,1. INGRESO,-4951176
1,1. TOTAL,2. COSTO MATERIALES,-118270839
2,1. TOTAL,3. COSTO MANO DE OBRA,-114373715
3,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261


##### 4.3.7.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [234]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   : ", mes_en_curso_real.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real2 = df_libro_mayor5_base2.merge(mes_en_curso_real, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       : ", mes_en_curso_real.shape)

mes_en_curso_real2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real   :  (362, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real       :  (362, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
261,P-1674,4. COSTO SUPERVISIÓN,0,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0
178,P-1603,3. COSTO MANO DE OBRA,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0
79,P-1693,2. COSTO MATERIALES,0,VOLTA V0800,ECOBIO SPA,0
87,P-1696,2. COSTO MATERIALES,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0
289,P-1692,4. COSTO SUPERVISIÓN,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,0


In [235]:
# TESTING VISUAL 1

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5436047
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7219006
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-3823676
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [236]:
# TESTING VISUAL 2

mes_en_curso_real2[mes_en_curso_real2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,-4951176
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-118270839
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.3 - SE RECALCULAN LOS TOTALES

In [237]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real2 = mes_en_curso_real2.drop(mes_en_curso_real2[(mes_en_curso_real2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 : ", mes_en_curso_real2.shape)

df_total = mes_en_curso_real2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real3 = pd.concat([mes_en_curso_real2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real3.loc[mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3.sample(10)   

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
253,P-1670,4. COSTO SUPERVISIÓN,0.00,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,0
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0
62,P-1687,3. COSTO MANO DE OBRA,0.00,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,0
129,P-1713,1. INGRESO,0.00,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,0
313,P-1717,1. INGRESO,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0
311,P-1715,4. COSTO SUPERVISIÓN,0.00,CD LO ECHEVERS,QUEZADA Y BOETSCH,0
41,P-1676,2. COSTO MATERIALES,0.00,LO CAMPINO,EBCO S.A.,1000393
78,P-1693,3. COSTO MANO DE OBRA,0.00,VOLTA V0800,ECOBIO SPA,-1271604
23,P-1661,2. COSTO MATERIALES,0.00,SAN VICENTE DE PAUL,DANACORP S.A,11176
30,P-1669,3. COSTO MANO DE OBRA,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0


In [238]:
# TESTING VISUAL 1

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,-4951176
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-118270839
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-4643261


In [239]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3 = mes_en_curso_real3.drop(mes_en_curso_real3[(mes_en_curso_real3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 : ", mes_en_curso_real3.shape)

mes_en_curso_real3[mes_en_curso_real3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-4643261
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,-4951176
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-118270839
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-114373715


##### 4.3.7.4 - CREACIÓN DE MARGEN

In [240]:
#CREACIÓN DE MARGENES

mes_en_curso_real4 = mes_en_curso_real3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
80,P-1725,-1.48
64,P-1707,0.00
2,P-1591,0.00
30,P-1670,0.00
14,P-1643,0.00
41,P-1683,0.00
3,P-1592,0.00
33,P-1674,0.00
27,P-1667,0.00
32,P-1673,0.00


In [241]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", mes_en_curso_real4.shape)

mes_en_curso_real5 = pd.merge(mes_en_curso_real4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 : ", mes_en_curso_real5.shape)
mes_en_curso_real5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
320,P-1720,2. COSTO MATERIALES,0.00,NaN,NaN,0,0.00
57,P-1684,1. INGRESO,0.00,EDIFICIO GOYCOLEA,EBCO S.A.,0,0.00
173,P-1591,4. COSTO SUPERVISIÓN,0.00,EDIFICIO ESTORIL,EBCO S.A.,0,0.00
313,P-1717,1. INGRESO,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,0.00
131,P-1714,2. COSTO MATERIALES,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0,0.00
125,P-1712,2. COSTO MATERIALES,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-7219006,4.31
126,P-1712,1. INGRESO,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-3823676,4.31
253,P-1670,4. COSTO SUPERVISIÓN,0.00,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,0,0.00
281,P-1686,4. COSTO SUPERVISIÓN,0.00,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,0,0.00
220,P-1649,4. COSTO SUPERVISIÓN,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0,0.00


In [242]:
# TESTING VISUAL 1

mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-1666000,0.00
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2308672,0.00
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.00


In [243]:
# TESTING VISUAL 3
mes_en_curso_real5[mes_en_curso_real5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-12890883,0.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-16121702,0.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00


In [244]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real5.to_csv('archivos_respaldo/mes_en_curso_real5.csv', mode='a')

### 4.4 - **MERGE PARA -> MES EN CURSO ENTRE LIBRO MAYOR + PROYECTADO (PERSPECTIVA MES ANTERIOR)
(DATAFRAMES: df_libro_mayor5_mec_2M y df_proyectado4) 


#### 4.4.1 - CREACIÓN DE COPIA DE DATAFRAME "df_libro_mayor5_mec_2M" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de LIBRO MAYOR

In [245]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
269,P-1699,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-8267470,LIBRO MAYOR
891,P-1643,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1248,P-1669,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
577,P-1557,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
351,P-1706,COSTO MATERIALES,2023-05-01,2023.00,5.00,-5198516,LIBRO MAYOR


In [246]:
# Ajuste del dataframe proveniente de LIBRO MAYOR

df_libro_mayor5_mec_2M.sample(5)
df_libro_mayor5_mec_2M_3 = df_libro_mayor5_mec_2M.drop(['CREDITO', 'DEBITO', 'MARGEN'], axis=1)
df_libro_mayor5_mec_2M_3.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
624,P-1592,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1994,P-1703,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
1440,P-1679,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
576,P-1557,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
1584,P-1685,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR


In [247]:
# INSPECCIÓN VISUAL INICIAL 1

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  : ", df_libro_mayor5_mec_2M.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M  :  (456, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN
319,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535,LIBRO MAYOR
325,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551,LIBRO MAYOR
331,P-1704,INGRESO,2023-05-01,2023.00,5.00,79059997,LIBRO MAYOR
2016,P-1704,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2017,P-1704,COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0,LIBRO MAYOR
2018,P-1704,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR
2019,P-1704,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR


In [248]:
# INSPECCIÓN VISUAL INICIAL

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  : ", df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3[df_libro_mayor5_mec_2M_3["CODIGO_PROYECTO"]=="P-1704"]

# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del libro mayor

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['INGRESO', 'COSTO MATERIALES']:
        return 'REAL'
    elif row['TIPO_CUENTA'] in ['COSTO SUPERVISIÓN','COSTO MANO DE OBRA']:
        return 'REAL Y PROYECTADO'

df_libro_mayor5_mec_2M_3['DESTINO'] = df_libro_mayor5_mec_2M_3.apply(determinar_origen, axis=1)
print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3.sample(10)


DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_mec_2M_3  :  (456, 7)
(456, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
1896,P-1698,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
1443,P-1679,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
816,P-1639,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
7,P-1592,COSTO MATERIALES,2023-05-01,2023.00,5.00,51537,LIBRO MAYOR,REAL
2544,P-1728,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0,LIBRO MAYOR,REAL Y PROYECTADO
518,P-1725,COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-2120143,LIBRO MAYOR,REAL Y PROYECTADO
1491,P-1681,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
2258,P-1716,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
699,P-1612,COSTO MATERIALES,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL
1130,P-1663,INGRESO,2023-05-01,2023.00,5.00,0,LIBRO MAYOR,REAL


In [249]:
# Cambiar los valores de la columna TIPO_CUENTA
df_libro_mayor5_mec_2M_3['TIPO_CUENTA'] = df_libro_mayor5_mec_2M_3['TIPO_CUENTA'].replace({'COSTO MATERIALES': '2. COSTO MATERIALES', 'COSTO MANO DE OBRA': '3. COSTO MANO DE OBRA', 'INGRESO': '1. INGRESO','COSTO SUPERVISIÓN': '4. COSTO SUPERVISIÓN'})

print(df_libro_mayor5_mec_2M_3.shape)
df_libro_mayor5_mec_2M_3["TIPO_CUENTA"].unique()

(456, 8)


array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

#### 4.4.2 - CREACIÓN DE COPIA DE DATAFRAME "df_proyectado4" Y AJUSTES INICIALES
Ajuste del dataframe proveniente de PROYECTADO

In [250]:
df_proyectado4_1M.sample(5)
df_proyectado5_1M = df_proyectado4_1M.drop(['PROYECTO'], axis=1)

# Se crea la columna Fecha
df_proyectado5_1M['FECHA'] = pd.to_datetime(df_proyectado5_1M['MES'].astype(str) + '-1-' + df_proyectado5_1M['AÑO'].astype(str))

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TOTAL,MES,AÑO,ORIGEN,TIPO_CUENTA,FECHA
5,P-1632,0.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01
72,P-1728,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-05-01
11,P-1656,0.00,5,2023,PROYECTADO,3. COSTO MANO DE OBRA,2023-05-01
8,P-1645,0.00,5,2023,PROYECTADO,2. COSTO MATERIALES,2023-05-01
49,P-1703,0.00,5,2023,PROYECTADO,1. INGRESO,2023-05-01


In [251]:
# Se crea nueva columna que define el destino del uso del trozo del DATAFRAME proveniente del proyectado

def determinar_origen(row):
    if row['TIPO_CUENTA'] in ['1. INGRESO', '2. COSTO MATERIALES']:
        return 'PROYECTADO'

# Se realiza 
df_proyectado5_1M['DESTINO'] = df_proyectado5_1M.apply(determinar_origen, axis=1)
df_proyectado5_1M = df_proyectado5_1M[df_proyectado5_1M['TIPO_CUENTA'] != '3. COSTO MANO DE OBRA']


# Se reordenan las columnas para estandarizarlo como la que viene desde Libro Mayot
df_proyectado5_1M = df_proyectado5_1M.reindex(columns=['CODIGO_PROYECTO', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO'])

df_proyectado5_1M.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
14,P-1663,2. COSTO MATERIALES,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
35,P-1689,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
33,P-1687,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO
65,P-1721,1. INGRESO,2023-05-01,2023,5,17010449.00,PROYECTADO,PROYECTADO
63,P-1719,1. INGRESO,2023-05-01,2023,5,0.00,PROYECTADO,PROYECTADO


#### 4.4.3 - CONCATENACIÓN ENTRE DATAFRAMES DEL MES EN CURSO LIBRO MAYOR Y PROYECTADO
Se realiza concatenación entre ambos dataframes

In [252]:
print("Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios: ", df_libro_mayor5_mec_2M_3.shape)
print("Dimensiones df_proyectado5_1M antes de cambios       : ", df_proyectado5_1M.shape)

mes_en_curso_2M = pd.concat([df_libro_mayor5_mec_2M_3, df_proyectado5_1M])

print("Dimensiones dataframe final mes_en_curso después de concatenación  : ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)


Dimensiones df_libro_mayor5_mec_2M_3 antes de cambios:  (456, 8)
Dimensiones df_proyectado5_1M antes de cambios       :  (148, 8)
Dimensiones dataframe final mes_en_curso después de concatenación  :  (604, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
541,P-1727,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-9901554.00,LIBRO MAYOR,REAL
1778,P-1693,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
2187,P-1713,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
2378,P-1721,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
2186,P-1713,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
54,P-1708,1. INGRESO,2023-05-01,2023.00,5.00,1568763.00,PROYECTADO,PROYECTADO
65,P-1721,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-7500000.00,PROYECTADO,PROYECTADO
1248,P-1669,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
865,P-1642,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
39,P-1693,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO


#### 4.4.4 - MERGE CON "df_cod_proyecto" PARA CREAR DATAFRAME FINAL

In [253]:
print("Dimensiones mes_en_curso antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M = mes_en_curso_2M.merge(df_cod_proyecto, on='CODIGO_PROYECTO', how='left')
mes_en_curso_2M = mes_en_curso_2M[['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE', 'TIPO_CUENTA', 'FECHA', 'AÑO', 'MES', 'TOTAL', 'ORIGEN', 'DESTINO']]
print("Dimensiones mes_en_curso_2M antes de cambios: ", mes_en_curso_2M.shape)
mes_en_curso_2M.sample(10)

Dimensiones mes_en_curso antes de cambios:  (604, 8)
Dimensiones mes_en_curso_2M antes de cambios:  (604, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
129,P-1603,Farellones,Empresa Constructora DLP S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
404,P-1718,Patio Comidas,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
315,P-1692,José Perez Cotapos,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
205,P-1661,San Vicente de Paul,DANACORP S.A,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
58,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535.00,LIBRO MAYOR,REAL Y PROYECTADO
418,P-1721,Strip Center Luis Pasteur,Inmobiliaria Luis Pasteur S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
370,P-1707,Las Tranqueras,Constructora Santolaya Ltda.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
356,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
214,P-1664,MK II PCI,Comercial K SpA.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
295,P-1687,Rojas Magallanes,E.MOLINA MOREL CONSTRUCTORA S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL


In [254]:
#Para reemplazar los valores en blanco en las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" por el valor "TOTAL" cada vez que el valor de la columna "CODIGO_PROYECTO" es igual a "TOTAL", puedes hacer lo siguiente:

mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_OBRA'].isnull()), 'NOMBRE_OBRA'] = '1. TOTAL'
mes_en_curso_2M.loc[(mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_2M['NOMBRE_CLIENTE'].isnull()), 'NOMBRE_CLIENTE'] = '1. TOTAL'
mes_en_curso_2M.sample(5)

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
451,P-1729,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
179,P-1645,Los tres Antonios,Sociedad Constructora Echavarri Hermanos Limitada,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
494,P-1692,José Perez Cotapos,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,PROYECTADO,PROYECTADO
59,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551.00,LIBRO MAYOR,REAL
83,P-1714,Departamental II,Constructora Fuerza Sur Dos Spa,1. INGRESO,2023-05-01,2023.00,5.00,2436155.00,LIBRO MAYOR,REAL


In [255]:
# INSPECCIÓN VISUAL 1

mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
58,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-14242535.00,LIBRO MAYOR,REAL Y PROYECTADO
59,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551.00,LIBRO MAYOR,REAL
60,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,79059997.00,LIBRO MAYOR,REAL
356,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
357,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
358,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
359,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
506,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,68606659.28,PROYECTADO,PROYECTADO
580,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-24000000.00,PROYECTADO,PROYECTADO


#### 4.3.4 - AGREGACIÓN DE TODOS TIPOS DE CUENTA

In [256]:
print("Dimensiones Dataframe antes de cambios", mes_en_curso_2M.shape)

Dimensiones Dataframe antes de cambios (604, 10)


In [257]:
# definir nuevas filas
df_rows_to_add = []

for _, row in mes_en_curso_2M.iterrows():
    if row["TIPO_CUENTA"] == "1. INGRESO" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "1. INGRESO"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "1. INGRESO"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "2. COSTO MATERIALES" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL"
        df_rows_to_add.append(row1)

        row2 = row.copy()
        row2["TIPO_CUENTA"] = "2. COSTO MATERIALES"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "PROYECTADO"
        df_rows_to_add.append(row2)

    elif row["TIPO_CUENTA"] == "3. COSTO MANO DE OBRA" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)

  
    elif row["TIPO_CUENTA"] == "4. COSTO SUPERVISIÓN" and row["TOTAL"] == 0:
        row1 = row.copy()
        row1["TIPO_CUENTA"] = "4. COSTO SUPERVISIÓN"
        row1["TOTAL"] = 0
        row1["ORIGEN"] = "AGREGACION"
        row1["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row1)
        
        row2 = row.copy()
        row2["TIPO_CUENTA"] = "3. COSTO MANO DE OBRA"
        row2["TOTAL"] = 0
        row2["ORIGEN"] = "AGREGACION"
        row2["DESTINO"] = "REAL Y PROYECTADO"
        df_rows_to_add.append(row2)

In [258]:
# concatenar nuevas filas al DataFrame original
mes_en_curso_2M = pd.concat([mes_en_curso_2M, pd.DataFrame(df_rows_to_add)], ignore_index=True)


In [259]:
# Se renombra "TOTAL" a "1. TOTAL" para que se pueda visualizar como primera opción en el dashboard.
mes_en_curso_2M['CODIGO_PROYECTO'] = mes_en_curso_2M['CODIGO_PROYECTO'].apply(lambda x: '1. TOTAL' if x == 'TOTAL' else x)

cols_to_replace = ['CODIGO_PROYECTO', 'NOMBRE_OBRA', 'NOMBRE_CLIENTE']
mes_en_curso_2M[cols_to_replace] = mes_en_curso_2M[cols_to_replace].replace('TOTAL', '1. TOTAL')
mask = mes_en_curso_2M['CODIGO_PROYECTO'] == '1. TOTAL'
mes_en_curso_2M.loc[mask, ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("Dimensiones Dataframe después de cambios", mes_en_curso_2M.shape)

mes_en_curso_2M.sample(10)

Dimensiones Dataframe después de cambios (1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
67,P-1707,Las Tranqueras,Constructora Santolaya Ltda.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-1568341.00,LIBRO MAYOR,REAL Y PROYECTADO
78,P-1713,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-172081.00,LIBRO MAYOR,REAL Y PROYECTADO
1310,P-1638,Colón,Constructora Grevia,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO
59,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-13967551.00,LIBRO MAYOR,REAL
607,P-1713,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO
120,P-1591,Edificio Estoril,EBCO S.A.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1046,P-1706,California,CONSTRUCTORA E INVERSIONES VITAL SPA.,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO
389,P-1714,Departamental II,Constructora Fuerza Sur Dos Spa,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
1214,P-1652,Abdon Cifuentes,EBCO S.A.,1. INGRESO,2023-05-01,2023.00,5.00,0.00,AGREGACION,PROYECTADO
169,P-1643,Chorrillos M4,Constructora Suksa S.A.,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO


In [260]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="P-1726"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
105,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-2654140.00,LIBRO MAYOR,REAL Y PROYECTADO
106,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-9263697.00,LIBRO MAYOR,REAL
107,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,LIBRO MAYOR,REAL
436,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
437,P-1726,NaN,NaN,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
438,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
439,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
526,P-1726,NaN,NaN,1. INGRESO,2023-05-01,2023.00,5.00,25429159.00,PROYECTADO,PROYECTADO
600,P-1726,NaN,NaN,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-10000000.00,PROYECTADO,PROYECTADO
1170,P-1726,NaN,NaN,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,AGREGACION,REAL Y PROYECTADO


In [261]:
mes_en_curso_2M["TIPO_CUENTA"].unique()

array(['3. COSTO MANO DE OBRA', '2. COSTO MATERIALES', '1. INGRESO',
       '4. COSTO SUPERVISIÓN'], dtype=object)

In [262]:
# INSPECCIÓN VISUAL 1
mes_en_curso_2M[mes_en_curso_2M["CODIGO_PROYECTO"]=="1. TOTAL"]


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,FECHA,AÑO,MES,TOTAL,ORIGEN,DESTINO
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,-111885468.00,LIBRO MAYOR,REAL Y PROYECTADO
113,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-169779486.00,LIBRO MAYOR,REAL
114,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,460798513.00,LIBRO MAYOR,REAL
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,-5096121.00,LIBRO MAYOR,REAL Y PROYECTADO
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,2023-04-01,2023.00,4.00,0.00,LIBRO MAYOR,REAL Y PROYECTADO
454,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
455,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,0.00,LIBRO MAYOR,REAL
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,2023-05-01,2023.00,5.00,461955807.36,PROYECTADO,PROYECTADO
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,2023-05-01,2023.00,5.00,-212300000.00,PROYECTADO,PROYECTADO


#### 4.4.7 - **CREACIÓN DATAFRAME PARA -> PROYECTADO

In [263]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('PROYECTADO')]

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   : ", mes_en_curso_2M_1.shape)


print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_proy_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_proy_2M = pd.DataFrame(mes_en_curso_proy_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  : ", mes_en_curso_proy_2M.shape)

mes_en_curso_proy_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1   :  (878, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (878, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
3,1. TOTAL,2. COSTO MATERIALES,-212300000.00
9,P-1557,4. COSTO SUPERVISIÓN,0.00
315,P-1717,4. COSTO SUPERVISIÓN,0.00
6,P-1557,1. INGRESO,0.00
232,P-1694,3. COSTO MANO DE OBRA,-4736731.00
29,P-1611,4. COSTO SUPERVISIÓN,0.00
361,P-1729,2. COSTO MATERIALES,0.00
215,P-1690,2. COSTO MATERIALES,-1500000.00
314,P-1717,3. COSTO MANO DE OBRA,0.00
349,P-1726,2. COSTO MATERIALES,-10000000.00


In [264]:
# TESTING VISUAL 1
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
348,P-1726,1. INGRESO,25429159.00
349,P-1726,2. COSTO MATERIALES,-10000000.00
350,P-1726,3. COSTO MANO DE OBRA,-2654140.00
351,P-1726,4. COSTO SUPERVISIÓN,0.00


In [265]:
# TESTING VISUAL 2
mes_en_curso_proy_2M[mes_en_curso_proy_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,461955807.36
3,1. TOTAL,2. COSTO MATERIALES,-212300000.00
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00


##### 4.4.7.1 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [266]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     : ", mes_en_curso_proy_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    : ", df_libro_mayor5_base2.shape)

mes_en_curso_proy_2M_2 = df_libro_mayor5_base2.merge(mes_en_curso_proy_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  : ", mes_en_curso_proy_2M_2.shape)

mes_en_curso_proy_2M_2['TOTAL'] = mes_en_curso_proy_2M_2['TOTAL'].fillna(0)

mes_en_curso_proy_2M_2.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M     :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2    :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_2  :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
202,P-1641,3. COSTO MANO DE OBRA,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0.00
64,P-1687,1. INGRESO,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00
123,P-1709,1. INGRESO,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,11127770.00
334,P-1729,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0.00


In [267]:
# TESTING VISUAL 1

mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00


In [268]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_2[mes_en_curso_proy_2M_2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.2 - SE RECALCULAN LOS TOTALES

In [269]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_proy_2M_2 = mes_en_curso_proy_2M_2.drop(mes_en_curso_proy_2M_2[(mes_en_curso_proy_2M_2['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_2['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 : ", mes_en_curso_proy_2M_2.shape)

df_total = mes_en_curso_proy_2M_2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_proy_2M_3 = pd.concat([mes_en_curso_proy_2M_2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_proy_2M_3.loc[mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3.sample(10) 

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_2 :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
311,P-1715,4. COSTO SUPERVISIÓN,0.00,CD LO ECHEVERS,QUEZADA Y BOETSCH,0.00
238,P-1663,4. COSTO SUPERVISIÓN,0.00,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0.00
102,P-1701,1. INGRESO,0.00,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,0.00
221,P-1653,1. INGRESO,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00
279,P-1685,3. COSTO MANO DE OBRA,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0.00
229,P-1658,4. COSTO SUPERVISIÓN,0.00,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0.00
335,P-1729,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00
58,P-1685,2. COSTO MATERIALES,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0.00


In [270]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121.00


In [271]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3 = mes_en_curso_proy_2M_3.drop(mes_en_curso_proy_2M_3[(mes_en_curso_proy_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_proy_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_proy_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 : ", mes_en_curso_proy_2M_3.shape)

mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


In [272]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00


In [273]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_3[mes_en_curso_proy_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00


##### 4.4.7.3 - CREACIÓN DE MARGEN

In [274]:
#CREACIÓN DE MARGENES

mes_en_curso_proy_2M_4 = mes_en_curso_proy_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_proy_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
21,P-1658,0.00
24,P-1663,0.00
81,P-1726,0.50
0,1. TOTAL,0.29
22,P-1660,0.00
35,P-1676,0.00
74,P-1719,0.00
33,P-1674,0.00
45,P-1687,0.00
37,P-1679,0.00


In [275]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 : ", mes_en_curso_proy_2M_4.shape)

mes_en_curso_proy_2M_5 = pd.merge(mes_en_curso_proy_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 : ", mes_en_curso_proy_2M_5.shape)
mes_en_curso_proy_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_proy_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
248,P-1668,3. COSTO MANO DE OBRA,0.00,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00,0.00
307,P-1709,4. COSTO SUPERVISIÓN,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,0.00,0.75
319,P-1719,4. COSTO SUPERVISIÓN,0.00,CD METALPAR,EBCO S.A.,0.00,0.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00
180,P-1611,1. INGRESO,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,0.00
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00,0.50
105,P-1703,1. INGRESO,0.00,SANTA VICTORIA DANACORP,DANACORP S.A,0.00,0.00
16,P-1645,1. INGRESO,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0.00,0.00
277,P-1684,4. COSTO SUPERVISIÓN,0.00,EDIFICIO GOYCOLEA,EBCO S.A.,0.00,0.55
62,P-1687,3. COSTO MANO DE OBRA,0.00,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,0.00,0.00


In [276]:
# TESTING VISUAL 1
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1723"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
150,P-1723,3. COSTO MANO DE OBRA,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-7889395.00,0.11
151,P-1723,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,-53000000.00,0.11
152,P-1723,1. INGRESO,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,68515939.00,0.11
326,P-1723,4. COSTO SUPERVISIÓN,0.00,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,0.00,0.11


In [277]:
# TESTING VISUAL 2
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121.00,0.29
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,461955807.36,0.29
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-212300000.00,0.29
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468.00,0.29


In [278]:
# Respaldo de Dataframe mes en curso antes de abrirrlo
mes_en_curso_proy_2M_5.to_csv('archivos_respaldo/mes_en_curso_proy_2M_5.csv', mode='a')

In [279]:
mes_en_curso_proy_2M_5[mes_en_curso_proy_2M_5["CODIGO_PROYECTO"]=="P-1726"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
159,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-2654140.00,0.50
160,P-1726,2. COSTO MATERIALES,0.00,NaN,NaN,-10000000.00,0.50
161,P-1726,1. INGRESO,0.00,NaN,NaN,25429159.00,0.50
329,P-1726,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.50


#### 4.4.8 - **CREACIÓN DATAFRAME PARA -> MES EN CURSO (PERSPECTIVA MES ANTERIOR)

##### 4.4.8.1 - TRATAMIENTO mes_en_curso2

In [280]:
# Se crea una copia del dataframe original mes_en_curso2

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       : ", mes_en_curso_2M.shape)

mes_en_curso_2M_1 = mes_en_curso_2M.copy()
mes_en_curso_2M_1 = mes_en_curso_2M_1[mes_en_curso_2M_1['DESTINO'].str.contains('REAL')]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     : ", mes_en_curso_2M_1.shape)

mes_en_curso_real_2M = pd.pivot_table(mes_en_curso_2M_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
mes_en_curso_real_2M = pd.DataFrame(mes_en_curso_real_2M.to_records())

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(10)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M       :  (1395, 10)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_2M_1     :  (979, 10)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M  :  (364, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
143,P-1671,2. COSTO MATERIALES,0
171,P-1679,2. COSTO MATERIALES,0
221,P-1691,4. COSTO SUPERVISIÓN,0
323,P-1719,4. COSTO SUPERVISIÓN,0
292,P-1712,1. INGRESO,15000000
106,P-1660,1. INGRESO,0
124,P-1666,3. COSTO MANO DE OBRA,0
261,P-1701,4. COSTO SUPERVISIÓN,0
334,P-1722,3. COSTO MANO DE OBRA,0
156,P-1675,1. INGRESO,0


In [281]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
292,P-1712,1. INGRESO,15000000
293,P-1712,2. COSTO MATERIALES,-5124213
294,P-1712,3. COSTO MANO DE OBRA,-2791400
295,P-1712,4. COSTO SUPERVISIÓN,0


In [282]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
2,1. TOTAL,1. INGRESO,460798513
3,1. TOTAL,2. COSTO MATERIALES,-169779486
4,1. TOTAL,3. COSTO MANO DE OBRA,-111885468
5,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121


##### 4.4.8.2 - MERGE CON DATAFRAME LIBRO MAYOR 2023 BASE 

In [283]:
# Merge Dataframe BASE 

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    : ", mes_en_curso_real_2M.shape)
print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   : ", df_libro_mayor5_base2.shape)

mes_en_curso_real_2M = df_libro_mayor5_base2.merge(mes_en_curso_real_2M, on=['CODIGO_PROYECTO', 'TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   : ", mes_en_curso_real_2M.shape)

mes_en_curso_real_2M.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M    :  (364, 3)
DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME: df_libro_mayor5_base2   :  (340, 5)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_real_2M   :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
168,P-1557,1. INGRESO,0,ROSARIO ROSALES,EBCO S.A.,0
88,P-1696,1. INGRESO,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,0
184,P-1612,3. COSTO MANO DE OBRA,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0
69,P-1690,3. COSTO MANO DE OBRA,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0
201,P-1641,1. INGRESO,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0


In [284]:
# TESTING VISUAL 1

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="P-1712"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
124,P-1712,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-2791400
125,P-1712,2. COSTO MATERIALES,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5124213
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,15000000
308,P-1712,4. COSTO SUPERVISIÓN,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,0


In [285]:
# TESTING VISUAL 2

mes_en_curso_real_2M[mes_en_curso_real_2M["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-169779486
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.3 - SE RECALCULAN LOS TOTALES

In [286]:
# Calcular la suma de los valores totales para cada tipo de cuenta

# Se eliminan totales excepto supervisión, para re hacer el calculo de totales

mes_en_curso_real_2M = mes_en_curso_real_2M.drop(mes_en_curso_real_2M[(mes_en_curso_real_2M['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M['TIPO_CUENTA'] != '4. COSTO SUPERVISIÓN')].index)

print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M : ", mes_en_curso_real_2M.shape)

df_total = mes_en_curso_real_2M.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
mes_en_curso_real_2M_3 = pd.concat([mes_en_curso_real_2M, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
mes_en_curso_real_2M_3.loc[mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3.sample(10)  

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M :  (337, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
13,P-1643,2. COSTO MATERIALES,0.00,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-330968
291,P-1693,4. COSTO SUPERVISIÓN,0.00,VOLTA V0800,ECOBIO SPA,0
170,P-1557,4. COSTO SUPERVISIÓN,0.00,ROSARIO ROSALES,EBCO S.A.,0
168,P-1557,1. INGRESO,0.00,ROSARIO ROSALES,EBCO S.A.,0
227,P-1658,1. INGRESO,0.00,VIVIPRA,VICHERAT Y PRADENAS LTDA.,0
287,P-1690,4. COSTO SUPERVISIÓN,0.00,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0
215,P-1648,1. INGRESO,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,0
315,P-1717,4. COSTO SUPERVISIÓN,0.00,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0
48,P-1680,2. COSTO MATERIALES,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,-11122532
278,P-1685,1. INGRESO,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0


In [287]:
# TESTING VISUAL 1

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169779486
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-5096121


In [288]:
# SE ELIMINA COSTO SUPERVISIÓN DUPLICADO.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3 = mes_en_curso_real_2M_3.drop(mes_en_curso_real_2M_3[(mes_en_curso_real_2M_3['CODIGO_PROYECTO'] == '1. TOTAL') & (mes_en_curso_real_2M_3['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (mes_en_curso_real_2M_3['TOTAL_LB'].isnull())].index)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 : ", mes_en_curso_real_2M_3.shape)

mes_en_curso_real_2M_3[mes_en_curso_real_2M_3["CODIGO_PROYECTO"]=="1. TOTAL"]

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (341, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_3 :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-5096121
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,460798513
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-169779486
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-111885468


##### 4.4.8.4 - CREACIÓN DE MARGEN

In [289]:
#CREACIÓN DE MARGENES

mes_en_curso_real_2M_4 = mes_en_curso_real_2M_3.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = mes_en_curso_real_2M_4.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)

,CODIGO_PROYECTO,MARGEN
36,P-1677,0.00
8,P-1626,0.00
51,P-1693,0.00
18,P-1649,0.00
14,P-1643,0.00
30,P-1670,0.00
21,P-1658,0.00
52,P-1694,0.32
32,P-1673,0.00
78,P-1723,0.39


In [290]:
print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 : ", mes_en_curso_real_2M_4.shape)

mes_en_curso_real_2M_5 = pd.merge(mes_en_curso_real_2M_4, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 : ", mes_en_curso_real_2M_5.shape)
mes_en_curso_real_2M_5.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_4 :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: mes_en_curso_real_2M_5 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0,0.00
125,P-1712,2. COSTO MATERIALES,0.00,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,-5124213,0.47
190,P-1626,3. COSTO MANO DE OBRA,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0,0.00
11,P-1641,2. COSTO MATERIALES,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0,0.00
42,P-1677,3. COSTO MANO DE OBRA,0.00,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0,0.00
206,P-1642,4. COSTO SUPERVISIÓN,0.00,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0,0.00
319,P-1719,4. COSTO SUPERVISIÓN,0.00,CD METALPAR,EBCO S.A.,0,0.00
183,P-1612,1. INGRESO,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0,0.00
210,P-1644,1. INGRESO,0.00,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,0,0.00
243,P-1666,3. COSTO MANO DE OBRA,0.00,CD AXIONLONG,EBCO S.A.,0,0.00


In [291]:
# TESTING VISUAL 1
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1716"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
136,P-1716,3. COSTO MANO DE OBRA,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-895428,0.61
137,P-1716,2. COSTO MATERIALES,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-2986507,0.61
138,P-1716,1. INGRESO,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,10082518,0.61
312,P-1716,4. COSTO SUPERVISIÓN,0.00,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,0,0.61


In [292]:
# TESTING VISUAL 2
mes_en_curso_real_2M_5[mes_en_curso_real_2M_5["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-14242535,0.64
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-13967551,0.64
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,79059997,0.64
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.64


In [293]:
# Respaldo de Dataframe mes en curso real
mes_en_curso_real_2M_5.to_csv('archivos_respaldo/mes_en_curso_real_2M_5.csv', mode='a')

### 4.5 - **MERGE PARA -> YTD PROYECTADO
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.5.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [294]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado.shape)
df_ytd_cerrado.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
123,P-1670,1. INGRESO,0,MK II Seguridad y Redes,Comercial K SpA.
147,P-1677,1. INGRESO,2369790,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...
239,P-1701,1. INGRESO,1393986,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.
18,P-1603,4. COSTO SUPERVISIÓN,0,Farellones,Empresa Constructora DLP S.A.
153,P-1680,2. COSTO MATERIALES,-38487045,Nueva Uno,Constructora Tecsa S.A.
140,P-1676,3. COSTO MANO DE OBRA,-1798488,Lo Campino,EBCO S.A.
262,P-1708,4. COSTO SUPERVISIÓN,0,Eco España,Constructora ICF SPA
42,P-1636,4. COSTO SUPERVISIÓN,0,Santo Domingo 1410,DANACORP S.A
249,P-1705,2. COSTO MATERIALES,-15060169,Los Clarines,CONSTRUCTORA INARCO S.A.
274,P-1713,4. COSTO SUPERVISIÓN,0,Reserva San Francisco,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...


In [295]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso2.shape)
mes_en_curso2.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)
mes_en_curso2_1 = mes_en_curso2[mes_en_curso2['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso2_1 = mes_en_curso2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso2_1.sample(5)

(1352, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
819,P-1679,Juan Sabaj,Constructora Grevia,3. COSTO MANO DE OBRA,0
1233,P-1701,Edificio Zenteno,CONSTRUCTORA GREVIA SpA.,1. INGRESO,0
319,P-1704,CD Mercado Libre Mezzanina,EBCO S.A.,3. COSTO MANO DE OBRA,0
28,P-1697,Estocolmo,Constructora Fortaleza S.A,3. COSTO MANO DE OBRA,-4485402
371,P-1719,CD METALPAR,EBCO S.A.,3. COSTO MANO DE OBRA,0


#### 4.5.2 - Concatenación de Dataframes

In [296]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado   ", df_ytd_cerrado.shape)
print("Dataframes antes de concatenar   : mes_en_curso2_1  ", mes_en_curso2_1.shape)

df_ytd_proyectado = pd.concat([df_ytd_cerrado, mes_en_curso2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado  ", df_ytd_proyectado.shape)

df_ytd_proyectado.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado    (340, 5)
Dataframes antes de concatenar   : mes_en_curso2_1   (873, 5)
Dataframes después de concatenar : df_ytd_proyectado   (1213, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
264,P-1709,3. COSTO MANO DE OBRA,-10702189,Centro Comercial Paseo San Pedro,EBCO S.A.
733,P-1664,3. COSTO MANO DE OBRA,0,MK II PCI,Comercial K SpA.
22,P-1611,4. COSTO SUPERVISIÓN,0,Edificio Plaza Egaña,EBCO S.A.
635,P-1639,3. COSTO MANO DE OBRA,0,Lyon 2546,Moller y Perez - Cotapos Construcciones Indust...
1201,P-1672,1. INGRESO,0,Sta Clara (ccdd-telecom),Constructora UPC S.A.


In [297]:
df_ytd_proyectado["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [298]:
# TESTING VISUAL 1 

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-8169893,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-13655419,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
331,P-1727,1. INGRESO,28570952,NaN,NaN
72,P-1727,3. COSTO MANO DE OBRA,-3455213,NaN,NaN
403,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN
404,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
488,P-1727,1. INGRESO,26500000,NaN,NaN
561,P-1727,2. COSTO MATERIALES,-10000000,NaN,NaN
1132,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN


In [299]:
# TESTING VISUAL 2

df_ytd_proyectado[df_ytd_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-605662121,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL
75,1. TOTAL,3. COSTO MANO DE OBRA,-114373715,1. TOTAL,1. TOTAL
78,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261,1. TOTAL,1. TOTAL
415,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL
416,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL
491,1. TOTAL,1. INGRESO,423800000,1. TOTAL,1. TOTAL
564,1. TOTAL,2. COSTO MATERIALES,-176535806,1. TOTAL,1. TOTAL


#### 4.5.3 - AGREGACIÓN COLUMNA MARGEN

In [300]:
#CREACIÓN DE MARGENES

df_ytd_proyectado2 = df_ytd_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 : ", df_ytd_proyectado2.shape)

df_ytd_proyectado2 = pd.merge(df_ytd_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 : ", df_ytd_proyectado2.shape)
df_ytd_proyectado2.sample(10)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: mes_en_curso_real4 :  (1213, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado2 :  (1213, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
586,P-1696,1. INGRESO,0,Santa Victoria - Terracorp,Constructora Terracorp S.A.,0.02
902,P-1685,3. COSTO MANO DE OBRA,0,Mapa Edificios No Operacionales,Claro Vicuña Valenzuela S.A.,0.00
183,P-1687,1. INGRESO,19576547,Rojas Magallanes,E.MOLINA MOREL CONSTRUCTORA S.A.,0.37
1013,P-1708,4. COSTO SUPERVISIÓN,0,Eco España,Constructora ICF SPA,-14.84
182,P-1687,4. COSTO SUPERVISIÓN,0,Rojas Magallanes,E.MOLINA MOREL CONSTRUCTORA S.A.,0.37
692,P-1557,3. COSTO MANO DE OBRA,0,Rosario Rosales,EBCO S.A.,0.00
1044,P-1716,3. COSTO MANO DE OBRA,0,CD Astara,Astara Logística Chile SpA.,0.57
525,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00
181,P-1687,2. COSTO MATERIALES,-6360393,Rojas Magallanes,E.MOLINA MOREL CONSTRUCTORA S.A.,0.37
908,P-1686,4. COSTO SUPERVISIÓN,0,Macul Icalma,EMPRESA CONSTRUCTORA INGEVEC S.A.,0.05


In [301]:
# TESTING VISUAL 1 

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-8169893,NaN,NaN,0.36
329,P-1727,2. COSTO MATERIALES,-13655419,NaN,NaN,0.36
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.36
331,P-1727,1. INGRESO,28570952,NaN,NaN,0.36
373,P-1727,3. COSTO MANO DE OBRA,-3455213,NaN,NaN,0.36
538,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.36
539,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.36
615,P-1727,1. INGRESO,26500000,NaN,NaN,0.36
688,P-1727,2. COSTO MATERIALES,-10000000,NaN,NaN,0.36
1097,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.36


In [302]:
# TESTING VISUAL 2

df_ytd_proyectado2[df_ytd_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965,1. TOTAL,1. TOTAL,0.30
1,1. TOTAL,2. COSTO MATERIALES,-605662121,1. TOTAL,1. TOTAL,0.30
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666,1. TOTAL,1. TOTAL,0.30
3,1. TOTAL,1. INGRESO,1579362409,1. TOTAL,1. TOTAL,0.30
374,1. TOTAL,3. COSTO MANO DE OBRA,-114373715,1. TOTAL,1. TOTAL,0.30
375,1. TOTAL,4. COSTO SUPERVISIÓN,-4643261,1. TOTAL,1. TOTAL,0.30
544,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,0.30
545,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,0.30
618,1. TOTAL,1. INGRESO,423800000,1. TOTAL,1. TOTAL,0.30
691,1. TOTAL,2. COSTO MATERIALES,-176535806,1. TOTAL,1. TOTAL,0.30


In [303]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado2.to_csv('archivos_respaldo/df_ytd_proyectado2.csv', mode='a')

### 4.6 - **MERGE PARA -> YTD PROYECTADO (PERSPECTIVA MES ANTERIOR)
YTD Proyectado = YTD + el mes en curso del proyectado

#### 4.6.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES


In [304]:
# DATAFRAME PROVENIENTE DEL LIBRO MAYOR 2023 HASTA EL MES CERRADO.

print(df_ytd_cerrado_2M.shape)
df_ytd_cerrado_2M.sample(10)

(340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
37,P-1632,2. COSTO MATERIALES,0,Chorrillos Clorox,Constructora Suksa S.A.
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN
169,P-1684,2. COSTO MATERIALES,-31987,Edificio Goycolea,EBCO S.A.
48,P-1641,3. COSTO MANO DE OBRA,0,Inacap Temuco,Desarrollos Constructivos Axis Sociedad Anonima
195,P-1690,1. INGRESO,19059330,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.
297,P-1719,2. COSTO MATERIALES,0,CD METALPAR,EBCO S.A.
146,P-1677,4. COSTO SUPERVISIÓN,0,Galvarino Gallardo,Moller y Perez - Cotapos Construcciones Indust...
162,P-1682,4. COSTO SUPERVISIÓN,0,Plaza Ceppi,EMP. CONSTRUCTORA INGENIEROS S.A.
96,P-1663,3. COSTO MANO DE OBRA,0,Oficinas Equifax,Constructora VRK Ingenieria Limitada
176,P-1686,3. COSTO MANO DE OBRA,-12899988,Macul Icalma,EMPRESA CONSTRUCTORA INGEVEC S.A.


In [305]:
# DATAFRAME PROVENIENTE DEL LIBRO PERO SOO CONSIDERA MES EN CURSO.

print(mes_en_curso_2M.shape)
mes_en_curso_2M.head(5)

# Se filtra por la columna DESTINO != REAL (para considerar solo lo proyectado)#
mes_en_curso_proy_2M_2_1 = mes_en_curso_2M[mes_en_curso_2M['DESTINO'] != 'REAL']

# Eliminar las columnas ORIGEN y DESTINO
mes_en_curso_proy_2M_2_1 = mes_en_curso_proy_2M_2_1.drop(columns=['ORIGEN', 'DESTINO','MES','AÑO','FECHA'])

mes_en_curso_proy_2M_2_1.sample(5)

(1395, 10)


,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
525,P-1725,Lo Cañas,Constructora Brisas de Batuco,1. INGRESO,19181571.00
782,P-1664,MK II PCI,Comercial K SpA.,1. INGRESO,0.00
429,P-1724,NaN,NaN,4. COSTO SUPERVISIÓN,0.00
1212,P-1621,Renault - Cormecanica,INDUSTRIA DE CONJUNTOS MECANICOS ACONCAGUA S.A,1. INGRESO,0.00
1386,P-1719,CD METALPAR,EBCO S.A.,2. COSTO MATERIALES,0.00


In [306]:
# TESTING VISUAL 1 

mes_en_curso_proy_2M_2_1[mes_en_curso_proy_2M_2_1["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,NOMBRE_OBRA,NOMBRE_CLIENTE,TIPO_CUENTA,TOTAL
112,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00
115,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00
452,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
453,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
529,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,461955807.36
603,1. TOTAL,1. TOTAL,1. TOTAL,2. COSTO MATERIALES,-212300000.00
1198,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1199,1. TOTAL,1. TOTAL,1. TOTAL,4. COSTO SUPERVISIÓN,0.00
1200,1. TOTAL,1. TOTAL,1. TOTAL,3. COSTO MANO DE OBRA,0.00
1202,1. TOTAL,1. TOTAL,1. TOTAL,1. INGRESO,0.00


#### 4.6.2 - Concatenación de Dataframes

In [307]:
# Se concatenan los dataframes.

print("Dataframes antes de concatenar   : df_ytd_cerrado_2M       :", df_ytd_cerrado_2M.shape)
print("Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1:", mes_en_curso_proy_2M_2_1.shape)

df_ytd_proyectado_2M = pd.concat([df_ytd_cerrado_2M, mes_en_curso_proy_2M_2_1], axis=0)
print("Dataframes después de concatenar : df_ytd_proyectado_2M  ", df_ytd_proyectado_2M.shape)

df_ytd_proyectado_2M.sample(5)

Dataframes antes de concatenar   : df_ytd_cerrado_2M       : (340, 5)
Dataframes antes de concatenar   : mes_en_curso_proy_2M_2_1: (878, 5)
Dataframes después de concatenar : df_ytd_proyectado_2M   (1218, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
258,P-1707,4. COSTO SUPERVISIÓN,0.00,Las Tranqueras,Constructora Santolaya Ltda.
365,P-1706,4. COSTO SUPERVISIÓN,0.00,California,CONSTRUCTORA E INVERSIONES VITAL SPA.
1376,P-1708,2. COSTO MATERIALES,0.00,Eco España,Constructora ICF SPA
602,P-1728,2. COSTO MATERIALES,0.00,NaN,NaN
815,P-1670,3. COSTO MANO DE OBRA,0.00,MK II Seguridad y Redes,Comercial K SpA.


In [308]:
df_ytd_proyectado_2M["CODIGO_PROYECTO"].unique()

array(['1. TOTAL', 'P-1557', 'P-1591', 'P-1592', 'P-1603', 'P-1611',
       'P-1612', 'P-1620', 'P-1626', 'P-1632', 'P-1636', 'P-1639',
       'P-1641', 'P-1642', 'P-1643', 'P-1644', 'P-1645', 'P-1648',
       'P-1649', 'P-1653', 'P-1656', 'P-1658', 'P-1660', 'P-1661',
       'P-1663', 'P-1664', 'P-1666', 'P-1667', 'P-1668', 'P-1669',
       'P-1670', 'P-1672', 'P-1673', 'P-1674', 'P-1675', 'P-1676',
       'P-1677', 'P-1679', 'P-1680', 'P-1681', 'P-1682', 'P-1683',
       'P-1684', 'P-1685', 'P-1686', 'P-1687', 'P-1688', 'P-1689',
       'P-1690', 'P-1691', 'P-1692', 'P-1693', 'P-1694', 'P-1695',
       'P-1696', 'P-1697', 'P-1698', 'P-1699', 'P-1700', 'P-1701',
       'P-1703', 'P-1704', 'P-1705', 'P-1706', 'P-1707', 'P-1708',
       'P-1709', 'P-1712', 'P-1713', 'P-1714', 'P-1715', 'P-1716',
       'P-1717', 'P-1718', 'P-1719', 'P-1720', 'P-1721', 'P-1722',
       'P-1723', 'P-1724', 'P-1725', 'P-1726', 'P-1727', 'P-1728',
       'P-1729', 'P-1600', 'P-1601', 'P-1615', 'P-1621', 'P-

In [309]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
331,P-1727,1. INGRESO,11005000.00,NaN,NaN
108,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN
440,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN
441,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN
527,P-1727,1. INGRESO,17255460.00,NaN,NaN
601,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN
1177,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN


In [310]:
# TESTING VISUAL 2

df_ytd_proyectado_2M[df_ytd_proyectado_2M["CODIGO_PROYECTO"] == "1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE
0,1. TOTAL,3. COSTO MANO DE OBRA,-363774250.00,1. TOTAL,1. TOTAL
1,1. TOTAL,2. COSTO MATERIALES,-435882635.00,1. TOTAL,1. TOTAL
2,1. TOTAL,4. COSTO SUPERVISIÓN,-17923405.00,1. TOTAL,1. TOTAL
3,1. TOTAL,1. INGRESO,1118563896.00,1. TOTAL,1. TOTAL
112,1. TOTAL,3. COSTO MANO DE OBRA,-111885468.00,1. TOTAL,1. TOTAL
115,1. TOTAL,4. COSTO SUPERVISIÓN,-5096121.00,1. TOTAL,1. TOTAL
452,1. TOTAL,3. COSTO MANO DE OBRA,0.00,1. TOTAL,1. TOTAL
453,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL
529,1. TOTAL,1. INGRESO,461955807.36,1. TOTAL,1. TOTAL
603,1. TOTAL,2. COSTO MATERIALES,-212300000.00,1. TOTAL,1. TOTAL


#### 4.6.3 - AGREGACIÓN COLUMNA MARGEN

In [311]:
#CREACIÓN DE MARGENES

df_ytd_proyectado_2M_2 = df_ytd_proyectado_2M.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ytd_proyectado_2M_2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)

df_ytd_proyectado_2M_2 = pd.merge(df_ytd_proyectado_2M_2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 : ", df_ytd_proyectado_2M_2.shape)
df_ytd_proyectado_2M_2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_proyectado_2M_2 :  (1218, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
666,P-1699,2. COSTO MATERIALES,-18000000.00,Villa Panamericana Lote B,Empresa Constructora DLP S.A.,0.29
879,P-1679,4. COSTO SUPERVISIÓN,0.00,Juan Sabaj,Constructora Grevia,0.00
193,P-1690,2. COSTO MATERIALES,-7192915.00,Ongolmo - Home Condominio,CANTAUCO CONSTRUCCION LTDA.,0.27
356,P-1703,3. COSTO MANO DE OBRA,-1057164.00,Santa Victoria Danacorp,DANACORP S.A,0.37
100,P-1664,3. COSTO MANO DE OBRA,0.00,MK II PCI,Comercial K SpA.,0.00
333,P-1728,2. COSTO MATERIALES,-1822779.00,NaN,NaN,0.53
537,P-1726,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.36
1109,P-1728,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.53
634,P-1661,2. COSTO MATERIALES,0.00,San Vicente de Paul,DANACORP S.A,0.00
150,P-1679,4. COSTO SUPERVISIÓN,0.00,Juan Sabaj,Constructora Grevia,0.00


In [312]:
# TESTING VISUAL 1 

df_ytd_proyectado_2M_2[df_ytd_proyectado_2M_2["CODIGO_PROYECTO"] == "P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,NOMBRE_OBRA,NOMBRE_CLIENTE,MARGEN
328,P-1727,3. COSTO MANO DE OBRA,-4714680.00,NaN,NaN,0.25
329,P-1727,2. COSTO MATERIALES,-3753865.00,NaN,NaN,0.25
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
331,P-1727,1. INGRESO,11005000.00,NaN,NaN,0.25
373,P-1727,3. COSTO MANO DE OBRA,-3205300.00,NaN,NaN,0.25
539,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25
540,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.25
618,P-1727,1. INGRESO,17255460.00,NaN,NaN,0.25
692,P-1727,2. COSTO MATERIALES,-9500000.00,NaN,NaN,0.25
1103,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.25


In [313]:
# Respaldo de Dataframe mes en curso real

df_ytd_proyectado_2M_2.to_csv('archivos_respaldo/df_ytd_proyectado_2M_2.csv', mode='a')

### 4.7 - **MERGE PARA -> PTD CERRADO

Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.7.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [331]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.2 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 : ", df_ytd_cerrado2.shape)

df_ytd_cerrado_1 = df_ytd_cerrado2.copy()
df_ytd_cerrado_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
338,P-1729,4. COSTO SUPERVISIÓN,0
115,P-1668,1. INGRESO,0
314,P-1723,4. COSTO SUPERVISIÓN,0
25,P-1612,2. COSTO MATERIALES,30258
80,P-1656,3. COSTO MANO DE OBRA,0


In [332]:
# TESTING VISUAL 1

print(df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_1[df_ytd_cerrado_1["CODIGO_PROYECTO"] == "1. TOTAL"]

472985657


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL
0,1. TOTAL,3. COSTO MANO DE OBRA,-478147965
1,1. TOTAL,2. COSTO MATERIALES,-605662121
2,1. TOTAL,4. COSTO SUPERVISIÓN,-22566666
3,1. TOTAL,1. INGRESO,1579362409


In [333]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 : ", df_ytd_cerrado_1.shape)
df_ytd_cerrado_2 = pd.pivot_table(df_ytd_cerrado_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 : ", df_ytd_cerrado_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2 = df_ytd_cerrado_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2 = pd.DataFrame(df_ytd_cerrado_2.to_records())
df_ytd_cerrado_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
16,P-1603,1. INGRESO,0
65,P-1645,2. COSTO MATERIALES,19980
310,P-1722,3. COSTO MANO DE OBRA,0
209,P-1694,2. COSTO MATERIALES,-22032871
167,P-1683,4. COSTO SUPERVISIÓN,0
42,P-1636,3. COSTO MANO DE OBRA,0
164,P-1683,1. INGRESO,0
127,P-1672,4. COSTO SUPERVISIÓN,0
90,P-1660,3. COSTO MANO DE OBRA,0
173,P-1685,2. COSTO MATERIALES,88900


In [335]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3

# Se crea una copia del dataframe 
print("Dimensiones del dataframe df_libro_mayor_old3 : ", df_libro_mayor_old3.shape)
df_libro_mayor_old4 = df_libro_mayor_old3.copy()
print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old3 :  (1741, 9)
Dimensiones del dataframe df_libro_mayor_old4 :  (1741, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1193,P-1686,1. INGRESO,2022-04-01 00:00:00,2022.00,4.00,8878586,3805108,5073478,0.00
1542,P-1704,3. COSTO MANO DE OBRA,2022-11-01 00:00:00,2022.00,11.00,0,13659190,-13659190,0.00
64,P-1611,3. COSTO MANO DE OBRA,2021-02-01 00:00:00,2021.00,2.00,0,0,0,0.00
1048,P-1677,3. COSTO MANO DE OBRA,2022-06-01 00:00:00,2022.00,6.00,0,1483640,-1483640,0.00
1606,P-1716,2. COSTO MATERIALES,2022-12-01 00:00:00,2022.00,12.00,0,4855555,-4855555,0.00
535,P-1660,2. COSTO MATERIALES,2021-04-01 00:00:00,2021.00,4.00,0,2552404,-2552404,0.00
810,P-1670,2. COSTO MATERIALES,2021-08-01 00:00:00,2021.00,8.00,67428,0,67428,0.00
15,,OTRA,2021-01-01 00:00:00,2021.00,1.00,0,10873783,-10873783,0.00
1131,P-1683,2. COSTO MATERIALES,2021-11-01 00:00:00,2021.00,11.00,0,3616116,-3616116,0.00
1315,P-1691,3. COSTO MANO DE OBRA,2022-03-01 00:00:00,2022.00,3.00,0,1721780,-1721780,0.00


In [336]:
# Se realiza limpieza del dataframe para obtener solo las columnas utiles.

df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['CODIGO_PROYECTO'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != 'OTRA']
df_libro_mayor_old4 = df_libro_mayor_old4[df_libro_mayor_old4['TIPO_CUENTA'] != '']
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['TIPO_CUENTA'])
df_libro_mayor_old4 = df_libro_mayor_old4.dropna(subset=['CODIGO_PROYECTO'])

print("Dimensiones del dataframe df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old4.sample(10)

Dimensiones del dataframe df_libro_mayor_old4 :  (1680, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1406,P-1696,2. COSTO MATERIALES,2022-11-01 00:00:00,2022.00,11.00,0,1280342,-1280342,0.00
1739,1. TOTAL,4. COSTO SUPERVISIÓN,2022-11-01 00:00:00,2022.00,11.00,0,5089343,-5089343,0.00
911,P-1673,1. INGRESO,2022-01-01 00:00:00,2022.00,1.00,2801791,2801791,0,0.00
1293,P-1691,1. INGRESO,2022-02-01 00:00:00,2022.00,2.00,6315236,0,6315236,0.00
720,P-1667,3. COSTO MANO DE OBRA,2022-04-01 00:00:00,2022.00,4.00,0,232327,-232327,0.00
679,P-1667,1. INGRESO,2022-01-01 00:00:00,2022.00,1.00,4718999,0,4718999,0.00
1656,1. TOTAL,2. COSTO MATERIALES,2021-01-01 00:00:00,2021.00,1.00,1101953,60946682,-59844729,0.00
464,P-1655,2. COSTO MATERIALES,2021-11-01 00:00:00,2021.00,11.00,91645,0,91645,0.00
558,P-1660,3. COSTO MANO DE OBRA,2022-02-01 00:00:00,2022.00,2.00,0,71885,-71885,0.00
1499,P-1702,2. COSTO MATERIALES,2022-05-01 00:00:00,2022.00,5.00,0,70180,-70180,0.00


In [337]:
# TESTING VISUAL 1

print(df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_libro_mayor_old4[df_libro_mayor_old4["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,FECHA,AÑO,MES,CREDITO,DEBITO,TOTAL,MARGEN
1632,1. TOTAL,1. INGRESO,2021-01-01 00:00:00,2021.00,1.00,153082726,51479131,101603595,0.00
1633,1. TOTAL,1. INGRESO,2021-02-01 00:00:00,2021.00,2.00,202772310,71750206,131022104,0.00
1634,1. TOTAL,1. INGRESO,2021-03-01 00:00:00,2021.00,3.00,227194679,39876627,187318052,0.00
1635,1. TOTAL,1. INGRESO,2021-04-01 00:00:00,2021.00,4.00,165417555,36206601,129210954,0.00
1636,1. TOTAL,1. INGRESO,2021-05-01 00:00:00,2021.00,5.00,113310222,10911630,102398592,0.00


In [338]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 : ", df_libro_mayor_old4.shape)
df_libro_mayor_old5 = pd.pivot_table(df_libro_mayor_old4, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

# Se renombra campo "TOTAL"
df_libro_mayor_old5 = df_libro_mayor_old5.rename(columns={"TOTAL": "TOTAL LB OLD"})

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)

df_libro_mayor_old5 = pd.DataFrame(df_libro_mayor_old5.to_records())
df_libro_mayor_old5.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_libro_mayor_old4 :  (1680, 9)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_libro_mayor_old5 :  (253, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
17,P-1616,2. COSTO MATERIALES,83720
16,P-1615,3. COSTO MANO DE OBRA,-1902829
239,P-1714,2. COSTO MATERIALES,-31104108
73,P-1652,2. COSTO MATERIALES,-13839565
226,P-1707,2. COSTO MATERIALES,-7486796
232,P-1712,1. INGRESO,132349558
126,P-1672,3. COSTO MANO DE OBRA,-46046794
165,P-1686,1. INGRESO,67737824
150,P-1681,1. INGRESO,10720132
173,P-1688,3. COSTO MANO DE OBRA,-3626251


In [339]:
# TESTING VISUAL 1

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "1. TOTAL"]

1369937537


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
0,1. TOTAL,1. INGRESO,3185711545
1,1. TOTAL,2. COSTO MATERIALES,-1152612025
2,1. TOTAL,3. COSTO MANO DE OBRA,-611212228
3,1. TOTAL,4. COSTO SUPERVISIÓN,-51949755


In [340]:
# TESTING VISUAL 2

print(df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"]["TOTAL LB OLD"].sum())
df_libro_mayor_old5[df_libro_mayor_old5["CODIGO_PROYECTO"] == "P-1615"].head(5)

31342023


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
14,P-1615,1. INGRESO,56248814
15,P-1615,2. COSTO MATERIALES,-23003962
16,P-1615,3. COSTO MANO DE OBRA,-1902829


#### 4.7.2 - Concatenación de Dataframes

In [341]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)
df_ptd_cerrado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
167,P-1729,2. COSTO MATERIALES,0,NaN,NaN
185,P-1612,4. COSTO SUPERVISIÓN,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A
98,P-1700,2. COSTO MATERIALES,0,CD ORIZON,EBCO S.A.
189,P-1626,1. INGRESO,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.
290,P-1693,1. INGRESO,0,VOLTA V0800,ECOBIO SPA


In [342]:
# MERGE 1 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_ytd_cerrado_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_pendiente2 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,0
144,P-1720,3. COSTO MANO DE OBRA,0,NaN,NaN,-65665
116,P-1707,2. COSTO MATERIALES,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-8495556
258,P-1673,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0
0,P-1557,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,EBCO S.A.,-1247014
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0
1,P-1591,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EBCO S.A.,-4593960
129,P-1713,1. INGRESO,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,4823540
33,P-1670,1. INGRESO,0,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,0


In [343]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado = pd.merge(df_ptd_cerrado, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
68,P-1689,2. COSTO MATERIALES,0,CASINO BUENAVENTURA,EBCO S.A.,26522,-8884093.00
332,P-1728,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,NaN
49,P-1680,1. INGRESO,0,NUEVA UNO,CONSTRUCTORA TECSA S.A.,76988974,111775261.00
34,P-1672,2. COSTO MATERIALES,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,-229982,-45941535.00
43,P-1677,2. COSTO MATERIALES,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-1379557,-30518645.00
153,P-1724,3. COSTO MANO DE OBRA,0,NaN,NaN,-506066,NaN
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,NaN
113,P-1706,2. COSTO MATERIALES,0,CALIFORNIA,CONSTRUCTORA E INVERSIONES VITAL SPA.,-13077846,NaN
12,P-1642,2. COSTO MATERIALES,0,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,8880,-3100254.00
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,28719770,14973674.00


In [344]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL YTD"] = df_ptd_cerrado["TOTAL YTD"].fillna(0)
df_ptd_cerrado["TOTAL LB OLD"] = df_ptd_cerrado["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado["TOTAL"] = df_ptd_cerrado["TOTAL YTD"] + df_ptd_cerrado["TOTAL LB OLD"]
df_ptd_cerrado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,0,0.00,0.00
192,P-1632,1. INGRESO,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0,0.00,0.00
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0,0.00,0.00
86,P-1696,3. COSTO MANO DE OBRA,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-15747680,-19361193.00,-35108873.00
322,P-1721,4. COSTO SUPERVISIÓN,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,0,0.00,0.00


In [345]:
# TESTING VISUAL 1

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL"]

472985657
1369937537.0
1842923194.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1579362409,3185711545.00,4765073954.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-605662121,-1152612025.00,-1758274146.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-478147965,-611212228.00,-1089360193.00


In [347]:
# TESTING VISUAL 2

print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL YTD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())

df_ptd_cerrado[df_ptd_cerrado["CODIGO_PROYECTO"] == "P-1727"]

6745640
0.0
6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,-8169893,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-13655419,0.00,-13655419.00
164,P-1727,1. INGRESO,0,NaN,NaN,28570952,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,0.00,0.00


#### 4.7.3 - SE AGREGAN FILAS DE 1.TOTAL

In [348]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado : ", df_ptd_cerrado.shape)

df_ptd_cerrado2 = df_ptd_cerrado[~((df_ptd_cerrado["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (337, 8)


In [349]:
# TESTING VISUAL 2


print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]

-22566666
-51949755.0
-74516421.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-22566666,-51949755.00,-74516421.00


In [350]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_total = df_ptd_cerrado2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado2 = pd.concat([df_ptd_cerrado2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado2.loc[df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 : ", df_ptd_cerrado2.shape)

df_ptd_cerrado2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
297,P-1698,4. COSTO SUPERVISIÓN,0.00,CD ANDES,EBCO S.A.,0.00,0.00,0.00
231,P-1660,3. COSTO MANO DE OBRA,0.00,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0.00,-6132285.00,-6132285.00
58,P-1685,2. COSTO MATERIALES,0.00,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,88900.00,-16551905.00,-16463005.00
95,P-1699,2. COSTO MATERIALES,0.00,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,-31245254.00,-22960.00,-31268214.00
110,P-1705,2. COSTO MATERIALES,0.00,LOS CLARINES,CONSTRUCTORA INARCO S.A.,-15060169.00,-43782.00,-15103951.00


In [352]:
# TESTING VISUAL 1

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1590900355.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686850217.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-74516421.00


In [353]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado2 = df_ptd_cerrado2[~((df_ptd_cerrado2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1665416776.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686850217.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00


In [354]:
# TESTING VISUAL 2

print(df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado2[df_ptd_cerrado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00


#### 4.7.4 - AGREGACIÓN COLUMNA MARGEN

In [355]:
#CREACIÓN DE MARGENES

df_ptd_cerrado3 = df_ptd_cerrado2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3 = pd.merge(df_ptd_cerrado3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)
df_ptd_cerrado3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
240,P-1664,3. COSTO MANO DE OBRA,0.00,MK II PCI,COMERCIAL K SPA.,0.00,0.00,0.00,0.64
218,P-1649,1. INGRESO,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,0.00,54564902.00,54564902.00,0.56
145,P-1720,1. INGRESO,0.00,NaN,NaN,0.00,0.00,0.00,0.00
174,P-1592,1. INGRESO,0.00,NUEVA COSTANERA 3300,EBCO S.A.,0.00,17821101.00,17821101.00,1.05
91,P-1697,1. INGRESO,0.00,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,53923452.00,35778806.00,89702258.00,0.46


In [356]:
# TESTING VISUAL 1

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1665416776.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686850217.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37


In [357]:
# TESTING VISUAL 2

print(df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado3[df_ptd_cerrado3["CODIGO_PROYECTO"] == "P-1727"].head(5)

6745640.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24


In [358]:
df_ptd_cerrado3.to_csv('archivos_respaldo/df_ptd_cerrado3.csv', index = False)

### 4.8 - **MERGE PARA -> PTD CERRADO (PERSPECTIVA MES ANTERIOR)
Cruce entre YTD_Cerrado + Libros mayores de años anteriores.

#### 4.8.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [372]:
# Exploración Dataframe para el dataframe que proviene del Libro Mayor 2023.
# Dataframe : df_ytd_cerrado2 proveniente del punto 4.3 - MERGE PARA -> YTD AÑO EN CURSO HASTA MES CERRADO PERSPECTIVA MES ANTERIOR

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 : ", df_ytd_cerrado_2M_3.shape)

df_ytd_cerrado_2M_3_1 = df_ytd_cerrado_2M_3.copy()
df_ytd_cerrado_2M_3_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3 :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
143,P-1719,1. INGRESO,0,CD METALPAR,EBCO S.A.,131348,1.00
184,P-1612,3. COSTO MANO DE OBRA,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0,0.00
222,P-1653,3. COSTO MANO DE OBRA,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0,0.00
192,P-1632,1. INGRESO,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0,0.00
279,P-1685,3. COSTO MANO DE OBRA,0,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,0,0.00


In [373]:
# TESTING VISUAL 1

print(df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ytd_cerrado_2M_3_1[df_ytd_cerrado_2M_3_1["CODIGO_PROYECTO"] == "1. TOTAL"]

300983606


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,0.27
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,0.27
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435882635,0.27
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,0.27


In [374]:
# Se crea Pivote para reducir las dimensiones

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   : ", df_ytd_cerrado_2M_3_1.shape)
df_ytd_cerrado_2M_3_2 = pd.pivot_table(df_ytd_cerrado_2M_3_1, values=['TOTAL'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

# Se renombra campo "TOTAL"
df_ytd_cerrado_2M_3_2 = df_ytd_cerrado_2M_3_2.rename(columns={"TOTAL": "TOTAL YTD"})

df_ytd_cerrado_2M_3_2 = pd.DataFrame(df_ytd_cerrado_2M_3_2.to_records())
df_ytd_cerrado_2M_3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_1   :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 1)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL YTD
134,P-1674,3. COSTO MANO DE OBRA,0
37,P-1632,2. COSTO MATERIALES,0
12,P-1592,1. INGRESO,0
19,P-1603,4. COSTO SUPERVISIÓN,0
157,P-1681,2. COSTO MATERIALES,0
192,P-1690,1. INGRESO,19059330
218,P-1696,3. COSTO MANO DE OBRA,-15437322
222,P-1697,3. COSTO MANO DE OBRA,-6902710
334,P-1728,3. COSTO MANO DE OBRA,-630869
71,P-1648,4. COSTO SUPERVISIÓN,0


In [375]:
# Exploración y tratamiento de dataframe: df_ytd_cerrado df_libro_mayor_old3
# Se utiliza el dataframe creado previamente en el punto: 4.7.1 : df_libro_mayor_old5 (NO ES NECESARIO TRATAR NUEVAMENTE LOS DATOS DEL LIBRO MAYOR DE AÑOS ANTRIORES)

print("Dimensiones del dataframe df_libro_mayor_old5 : ", df_libro_mayor_old5.shape)
df_libro_mayor_old5.sample(10)

Dimensiones del dataframe df_libro_mayor_old5 :  (253, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL LB OLD
227,P-1707,3. COSTO MANO DE OBRA,-8068564
210,P-1701,1. INGRESO,78135106
248,P-1718,3. COSTO MANO DE OBRA,-2971700
123,P-1671,3. COSTO MANO DE OBRA,-224796
203,P-1698,3. COSTO MANO DE OBRA,-19007691
27,P-1626,2. COSTO MATERIALES,678564
171,P-1688,1. INGRESO,16367077
105,P-1665,3. COSTO MANO DE OBRA,-329352
55,P-1643,1. INGRESO,0
12,P-1612,2. COSTO MATERIALES,280644


#### 4.8.2 - Concatenación de Dataframes

In [376]:
# Se considera df_libro_mayor5_base2 como base para construir el dataframe

df_ptd_cerrado_anterior = df_libro_mayor5_base2.copy()

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)
df_ptd_cerrado_anterior.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
262,P-1675,3. COSTO MANO DE OBRA,0,PALENA II,CONSTRUCTORA MYA SPA.
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.
323,P-1722,2. COSTO MATERIALES,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA
51,P-1682,3. COSTO MANO DE OBRA,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.
109,P-1705,3. COSTO MANO DE OBRA,0,LOS CLARINES,CONSTRUCTORA INARCO S.A.


In [377]:
# MERGE 1 - df_ytd_cerrado_2M_3_2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 : ", df_ytd_cerrado_2M_3_2.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_ytd_cerrado_2M_3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ytd_cerrado_2M_3_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD
20,P-1656,2. COSTO MATERIALES,0,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,0
84,P-1695,2. COSTO MATERIALES,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-10303848
251,P-1669,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0
129,P-1713,1. INGRESO,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,4823540
250,P-1669,1. INGRESO,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0
144,P-1720,3. COSTO MANO DE OBRA,0,NaN,NaN,0
89,P-1697,3. COSTO MANO DE OBRA,0,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-6902710
35,P-1672,1. INGRESO,0,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,9435519
303,P-1705,4. COSTO SUPERVISIÓN,0,LOS CLARINES,CONSTRUCTORA INARCO S.A.,0
16,P-1645,1. INGRESO,0,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0


In [378]:
# MERGE 2 - df_ytd_cerrado2

# Se hace merge con los codigos de proyecto.

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior = pd.merge(df_ptd_cerrado_anterior, df_libro_mayor_old5, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 6)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD
258,P-1673,4. COSTO SUPERVISIÓN,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0,NaN
100,P-1701,3. COSTO MANO DE OBRA,0,EDIFICIO ZENTENO,CONSTRUCTORA GREVIA SPA.,-891528,-24664670.00
197,P-1636,4. COSTO SUPERVISIÓN,0,SANTO DOMINGO 1410,DANACORP S.A,0,NaN
236,P-1663,1. INGRESO,0,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,0,NaN
126,P-1712,1. INGRESO,0,CENTRO COMERCIAL VIÑA OUTLET PARK,EBCO S.A.,17769320,132349558.00
75,P-1692,3. COSTO MANO DE OBRA,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,-21813347,-18580010.00
72,P-1691,3. COSTO MANO DE OBRA,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-252060,-16971354.00
191,P-1626,4. COSTO SUPERVISIÓN,0,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,0,NaN
180,P-1611,1. INGRESO,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0,4173520.00
321,P-1720,4. COSTO SUPERVISIÓN,0,NaN,NaN,0,NaN


In [379]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL YTD"] = df_ptd_cerrado_anterior["TOTAL YTD"].fillna(0)
df_ptd_cerrado_anterior["TOTAL LB OLD"] = df_ptd_cerrado_anterior["TOTAL LB OLD"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_anterior["TOTAL"] = df_ptd_cerrado_anterior["TOTAL YTD"] + df_ptd_cerrado_anterior["TOTAL LB OLD"]
df_ptd_cerrado_anterior.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
11,P-1641,2. COSTO MATERIALES,0,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,-369720,-669050.00,-1038770.00
142,P-1718,1. INGRESO,0,PATIO COMIDAS,EBCO S.A.,88900450,15594306.00,104494756.00
177,P-1603,1. INGRESO,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0,0.00,0.00
26,P-1666,2. COSTO MATERIALES,0,CD AXIONLONG,EBCO S.A.,0,-25105908.00,-25105908.00
282,P-1687,4. COSTO SUPERVISIÓN,0,ROJAS MAGALLANES,E.MOLINA MOREL CONSTRUCTORA S.A.,0,0.00,0.00


In [380]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL"]

300983606
1369937537.0
1670921143.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,1118563896,3185711545.00,4304275441.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-435882635,-1152612025.00,-1588494660.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-363774250,-611212228.00,-974986478.00


In [381]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())

df_ptd_cerrado_anterior[df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "P-1704"]

38892837
252119092.0
291011929.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0,0.00,0.00


#### 4.8.3 - SE AGREGAN FILAS DE 1.TOTAL

In [382]:
# Se eliminan Totales, excepto cuando se trata de COSTO_SUPERVISIÓN

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior : ", df_ptd_cerrado_anterior.shape)

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior[~((df_ptd_cerrado_anterior["CODIGO_PROYECTO"] == "1. TOTAL") & (df_ptd_cerrado_anterior["TIPO_CUENTA"] != "4. COSTO SUPERVISIÓN"))]

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior :  (340, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (337, 8)


In [383]:
# TESTING VISUAL 2


print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL YTD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL LB OLD"].sum())
print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())

df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]

-17923405
-51949755.0
-69873160.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-17923405,-51949755.00,-69873160.00


In [384]:
# Calcular la suma de los valores totales para cada tipo de cuenta

print("DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_total = df_ptd_cerrado_anterior2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_ptd_cerrado_anterior2 = pd.concat([df_ptd_cerrado_anterior2, df_total], ignore_index=True)

# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_ptd_cerrado_anterior2.loc[df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

print("DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 : ", df_ptd_cerrado_anterior2.shape)

df_ptd_cerrado_anterior2.sample(5)  

DIMENSIONES ANTES DE TRATAMIENTO DEL DATAFRAME: df_ptd_cerrado_anterior2 :  (337, 8)
DIMENSIONES DESPUÉS DE TRATAMIENTO DEL DATAFRAME df_ptd_cerrado_anterior2 :  (341, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
276,P-1683,4. COSTO SUPERVISIÓN,0.00,CD BP-A05,EBCO S.A.,0.00,0.00,0.00
186,P-1620,1. INGRESO,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,0.00,0.00,0.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00
66,P-1688,2. COSTO MATERIALES,0.00,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-14620172.00,-7648784.00,-22268956.00
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00


In [385]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1423541565.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1517070731.00
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00
340,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-69873160.00


In [387]:
# Se eliminan COSTO SUPERVISION INCORRECTO

df_ptd_cerrado_anterior2 = df_ptd_cerrado_anterior2[~((df_ptd_cerrado_anterior2['CODIGO_PROYECTO'] == '1. TOTAL') & (df_ptd_cerrado_anterior2['TIPO_CUENTA'] == '4. COSTO SUPERVISIÓN') & (df_ptd_cerrado_anterior2['TOTAL_LB'].isna()))]
 
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior2[df_ptd_cerrado_anterior2["CODIGO_PROYECTO"] == "P-1704"].head(5)

1493414725.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


#### 4.8.4 - AGREGACIÓN COLUMNA MARGEN

In [388]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_anterior3 = df_ptd_cerrado_anterior2.copy()


# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_anterior3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado_anterior3 = pd.merge(df_ptd_cerrado_anterior3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)
df_ptd_cerrado_anterior3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
269,P-1680,4. COSTO SUPERVISIÓN,0.00,NUEVA UNO,CONSTRUCTORA TECSA S.A.,0.00,0.00,0.00,0.30
305,P-1707,4. COSTO SUPERVISIÓN,0.00,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,0.00,0.00,0.00,0.26
135,P-1715,1. INGRESO,0.00,CD LO ECHEVERS,QUEZADA Y BOETSCH,40610937.00,0.00,40610937.00,0.14
121,P-1709,3. COSTO MANO DE OBRA,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,-9642970.00,-2009577.00,-11652547.00,0.41
283,P-1688,4. COSTO SUPERVISIÓN,0.00,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0.00,0.00,0.00,0.16


In [389]:
# TESTING VISUAL 1

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1493414725.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-17923405.00,-51949755.00,-69873160.00,0.37
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4042394062.00,0.37
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1517070731.00,0.37
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-962035446.00,0.37


In [390]:
# TESTING VISUAL 2

print(df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_anterior3[df_ptd_cerrado_anterior3["CODIGO_PROYECTO"] == "P-1704"].head(5)

2536455.0


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48


In [391]:
df_ptd_cerrado_anterior3.to_csv('archivos_respaldo/df_ptd_cerrado_anterior3.csv', index = False)

### 4.9 - MERGE PTD + MES EN CURSO (PROYECTADO)
Cruce entre df_ptd_cerrado3 + mes_en_curso_proy5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.9.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [394]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado3_1 = df_ptd_cerrado3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado3_1 = df_ptd_cerrado3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   : ", df_ptd_cerrado3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado3_1 = df_ptd_cerrado3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
259,P-1674,1. INGRESO,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,23646019.00
283,P-1688,4. COSTO SUPERVISIÓN,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0.00
59,P-1686,3. COSTO MANO DE OBRA,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,-38451306.00
35,P-1672,1. INGRESO,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,134527241.00
281,P-1686,4. COSTO SUPERVISIÓN,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,0.00


In [395]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
mes_en_curso_proy5_1 = mes_en_curso_proy5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy5_1 = mes_en_curso_proy5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  : ", mes_en_curso_proy5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 : ", mes_en_curso_proy5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy5_1 = mes_en_curso_proy5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy5  :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
33,P-1670,1. INGRESO,0
282,P-1687,4. COSTO SUPERVISIÓN,0
281,P-1686,4. COSTO SUPERVISIÓN,0
231,P-1660,3. COSTO MANO DE OBRA,0
206,P-1642,4. COSTO SUPERVISIÓN,0


#### 4.9.2 - MERGE DE DATAFRAMES 

In [396]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)

df_ptd_cerrado_y_proyectado = pd.merge(df_ptd_cerrado3_1, mes_en_curso_proy5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado : ", df_ptd_cerrado_y_proyectado.shape)

df_ptd_cerrado_y_proyectado.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
71,P-1690,1. INGRESO,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,52317842.00,5000000
319,P-1719,4. COSTO SUPERVISIÓN,CD METALPAR,EBCO S.A.,0.00,0
290,P-1693,1. INGRESO,VOLTA V0800,ECOBIO SPA,138699361.00,0
25,P-1664,2. COSTO MATERIALES,MK II PCI,COMERCIAL K SPA.,-50055425.00,0
206,P-1642,4. COSTO SUPERVISIÓN,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0


In [397]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL PTD"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado["TOTAL"] = df_ptd_cerrado_y_proyectado["TOTAL PTD"] + df_ptd_cerrado_y_proyectado["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
293,P-1695,4. COSTO SUPERVISIÓN,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,0.00,0,0.00
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000,-224683461.00
84,P-1695,2. COSTO MATERIALES,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-25647031.00,-3000000,-28647031.00
52,P-1682,2. COSTO MATERIALES,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-25232525.00,0,-25232525.00
327,P-1724,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00


In [398]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1793663994.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261,-79159682.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,423800000,4926992575.00
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686850217.00,-176535806,-1863386023.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715,-1190782876.00


In [399]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado[df_ptd_cerrado_y_proyectado["CODIGO_PROYECTO"] == "P-1727"].head(5)

19790427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213,-11625106.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000,-23655419.00
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,26500000,55070952.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00


#### 4.9.3 - AGREGACIÓN COLUMNA MARGEN

In [400]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado2 = df_ptd_cerrado_y_proyectado.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2 = pd.merge(df_ptd_cerrado_y_proyectado2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)
df_ptd_cerrado_y_proyectado2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
232,P-1660,4. COSTO SUPERVISIÓN,CORONEL ALVARADO,LARRAIN PRIETO RISOPATRON S.A.,0.00,0,0.00,0.31
200,P-1639,4. COSTO SUPERVISIÓN,LYON 2546,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0.00,0,0.00,0.48
219,P-1649,3. COSTO MANO DE OBRA,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,-220289.00,0,-220289.00,0.56
225,P-1656,3. COSTO MANO DE OBRA,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,-929633.00,0,-929633.00,0.47
182,P-1611,4. COSTO SUPERVISIÓN,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,0,0.00,1.14
196,P-1636,3. COSTO MANO DE OBRA,SANTO DOMINGO 1410,DANACORP S.A,-410529.00,0,-410529.00,0.77
215,P-1648,1. INGRESO,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,672750.00,0,672750.00,2.17
28,P-1667,2. COSTO MATERIALES,ECO PLACILLA,CONSTRUCTORA ICF SPA,-45657105.00,0,-45657105.00,0.01
169,P-1557,2. COSTO MATERIALES,ROSARIO ROSALES,EBCO S.A.,0.00,0,0.00,0.00
37,P-1674,2. COSTO MATERIALES,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,-8601126.00,0,-8601126.00,0.63


In [401]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1793663994.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-74516421.00,-4643261,-79159682.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4503192575.00,423800000,4926992575.00,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1686850217.00,-176535806,-1863386023.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-1076409161.00,-114373715,-1190782876.00,0.36


In [402]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado2[df_ptd_cerrado_y_proyectado2["CODIGO_PROYECTO"] == "P-1727"].head(5)

19790427.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-8169893.00,-3455213,-11625106.00,0.36
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-13655419.00,-10000000,-23655419.00,0.36
164,P-1727,1. INGRESO,NaN,NaN,28570952.00,26500000,55070952.00,0.36
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00,0.36


In [403]:
df_ptd_cerrado_y_proyectado2.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado2.csv', index = False)

### 4.10 - MERGE PTD + MES EN CURSO (PROYECTADO) (PERSPECTIVA MES ANTERIOR)
Cruce entre df_ptd_cerrado_anterior3 + mes_en_curso_proy_2M_5

NOTA IMPORTANTE:
Ambos dataframes vienen depurados de los MERGES anteriores y no es necesario trabajar con el df_libro_mayor5_base2, por lo tanto solo se realiza concatenación y se crean las filas 1. TOTAL y el MARGEN. 

#### 4.10.1 - EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [408]:
# TRATAMIENTO DATAFRAME df_ptd_cerrado3_1

# Se realiza una copia del dataframe 
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3.copy()

# Se Seleccionan columnas utiles:

df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "NOMBRE_OBRA", "NOMBRE_CLIENTE", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   : ", df_ptd_cerrado_anterior3.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

# Se renombra campo "TOTAL"
df_ptd_cerrado_anterior3_1 = df_ptd_cerrado_anterior3_1.rename(columns={"TOTAL": "TOTAL PTD"})

df_ptd_cerrado_anterior3_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : df_ptd_cerrado_anterior3   :  (340, 9)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD
258,P-1673,4. COSTO SUPERVISIÓN,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00
315,P-1717,4. COSTO SUPERVISIÓN,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00
243,P-1666,3. COSTO MANO DE OBRA,CD AXIONLONG,EBCO S.A.,-1957926.00
157,P-1725,2. COSTO MATERIALES,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4180127.00
35,P-1672,1. INGRESO,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,134527241.00


In [409]:
# TRATAMIENTO DATAFRAME mes en curso proyectado

# Se realiza una copia del dataframe 
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5.copy()

# Se Seleccionan columnas utiles:

mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1[["CODIGO_PROYECTO", "TIPO_CUENTA", "TOTAL"]]

print("DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   : ", mes_en_curso_proy_2M_5.shape)
print("DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 : ", mes_en_curso_proy_2M_5_1.shape)

# Se renombra campo "TOTAL"
mes_en_curso_proy_2M_5_1 = mes_en_curso_proy_2M_5_1.rename(columns={"TOTAL": "TOTAL MEC PROY"})

mes_en_curso_proy_2M_5_1.sample(5)

DIMENSIONES ANTES DE TRANSFORMACIÓN DATAFRAME  : mes_en_curso_proy_2M_5   :  (340, 7)
DIMENSIONES DESPUÉS DE TRANSFORMACIÓN DATAFRAME: mes_en_curso_proy_2M_5_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL MEC PROY
204,P-1642,1. INGRESO,0.00
281,P-1686,4. COSTO SUPERVISIÓN,0.00
115,P-1707,3. COSTO MANO DE OBRA,-1568341.00
257,P-1673,3. COSTO MANO DE OBRA,0.00
40,P-1676,3. COSTO MANO DE OBRA,0.00


#### 4.10.2 - MERGE DE DATAFRAMES 

In [410]:
# Se hace merge manteniendo como base df_ptd_cerrado3_1 que viene del total de proyectos existentes en el libro mayor 2023

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 : ", df_ptd_cerrado_anterior3_1.shape)

df_ptd_cerrado_y_proyectado_ant = pd.merge(df_ptd_cerrado_anterior3_1, mes_en_curso_proy_2M_5_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant : ", df_ptd_cerrado_y_proyectado_ant.shape)

df_ptd_cerrado_y_proyectado_ant.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_anterior3_1 :  (340, 5)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY
321,P-1720,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00
115,P-1707,3. COSTO MANO DE OBRA,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-11882405.00,-1568341.00
193,P-1632,3. COSTO MANO DE OBRA,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,0.00
86,P-1696,3. COSTO MANO DE OBRA,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-34798515.00,-1750141.00
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0.00


In [411]:
# Se estandarizan columnas de Totales y se agrega columna TOTAL 

# Reemplazar NaN por cero en las columnas "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"].fillna(0)
df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"] = df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"].fillna(0)

# Crear la nueva columna "TOTAL" que suma "TOTAL YTD" y "TOTAL LB OLD"
df_ptd_cerrado_y_proyectado_ant["TOTAL"] = df_ptd_cerrado_y_proyectado_ant["TOTAL PTD"] + df_ptd_cerrado_y_proyectado_ant["TOTAL MEC PROY"]
df_ptd_cerrado_y_proyectado_ant.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,461955807.36,4504349869.36
154,P-1724,2. COSTO MATERIALES,NaN,NaN,-7634256.00,0.00,-7634256.00
169,P-1557,2. COSTO MATERIALES,ROSARIO ROSALES,EBCO S.A.,0.00,0.00,0.00
53,P-1682,1. INGRESO,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,55359285.00,18808581.00,74167866.00
118,P-1708,3. COSTO MANO DE OBRA,ECO ESPAÑA,CONSTRUCTORA ICF SPA,-1471220.00,-921206.00,-2392426.00


In [412]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1626088943.3564873


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,461955807.36,4504349869.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1517070731.00,-212300000.00,-1729370731.00
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00


In [413]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1727"].head(5)

7086615.0


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
162,P-1727,3. COSTO MANO DE OBRA,NaN,NaN,-4714680.00,-3205300.00,-7919980.00
163,P-1727,2. COSTO MATERIALES,NaN,NaN,-3753865.00,-9500000.00,-13253865.00
164,P-1727,1. INGRESO,NaN,NaN,11005000.00,17255460.00,28260460.00
330,P-1727,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0.00,0.00


#### 4.10.3 - AGREGACIÓN COLUMNA MARGEN

In [414]:
#CREACIÓN DE MARGENES

df_ptd_cerrado_y_proyectado_ant2 = df_ptd_cerrado_y_proyectado_ant.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_ptd_cerrado_y_proyectado_ant2.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2 = pd.merge(df_ptd_cerrado_y_proyectado_ant2, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)
df_ptd_cerrado_y_proyectado_ant2.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
169,P-1557,2. COSTO MATERIALES,ROSARIO ROSALES,EBCO S.A.,0.00,0.00,0.00,0.00
84,P-1695,2. COSTO MATERIALES,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,-17009933.00,-10500000.00,-27509933.00,0.07
246,P-1667,4. COSTO SUPERVISIÓN,ECO PLACILLA,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.01
207,P-1643,1. INGRESO,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.00
72,P-1691,3. COSTO MANO DE OBRA,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-17223414.00,0.00,-17223414.00,0.16
140,P-1718,3. COSTO MANO DE OBRA,PATIO COMIDAS,EBCO S.A.,-29315555.00,-8486339.00,-37801894.00,0.46
116,P-1707,2. COSTO MATERIALES,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-10347749.00,-7000000.00,-17347749.00,0.17
135,P-1715,1. INGRESO,CD LO ECHEVERS,QUEZADA Y BOETSCH,40610937.00,18230799.58,58841736.58,0.24
241,P-1664,4. COSTO SUPERVISIÓN,MK II PCI,COMERCIAL K SPA.,0.00,0.00,0.00,0.64
35,P-1672,1. INGRESO,STA CLARA (CCDD-TELECOM),CONSTRUCTORA UPC S.A.,134527241.00,0.00,134527241.00,0.32


In [415]:
# TESTING VISUAL 1

print(df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant2[df_ptd_cerrado_y_proyectado_ant2["CODIGO_PROYECTO"] == "1. TOTAL"].head(5)

1626088943.3564873


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,1. TOTAL,1. TOTAL,-69873160.00,-5096121.00,-74969281.00,0.36
337,1. TOTAL,1. INGRESO,1. TOTAL,1. TOTAL,4042394062.00,461955807.36,4504349869.36,0.36
338,1. TOTAL,2. COSTO MATERIALES,1. TOTAL,1. TOTAL,-1517070731.00,-212300000.00,-1729370731.00,0.36
339,1. TOTAL,3. COSTO MANO DE OBRA,1. TOTAL,1. TOTAL,-962035446.00,-111885468.00,-1073920914.00,0.36


In [416]:
# TESTING VISUAL 2

print(df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_ptd_cerrado_y_proyectado_ant[df_ptd_cerrado_y_proyectado_ant["CODIGO_PROYECTO"] == "P-1704"].head(5)

321376053.27884996


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00


In [417]:
df_ptd_cerrado_y_proyectado_ant.to_csv('archivos_respaldo/df_ptd_cerrado_y_proyectado_ant.csv', index = False)

### 4.11 MERGE PRESUPUESTO
Se realiza mergen entre los datos que provienen de la planilla de presupuesto y tabla de presupuestos.

#### 4.11.1 CREACIÓN Y TRATAMIENTO INICIAL DE DATAFRAMES A CONCANTENAR.

In [421]:
print("DIMENSIONES DATAFRAME *df_tabla_presupuesto3* :", df_tabla_presupuesto3.shape)
df_tabla_presupuesto4 = df_tabla_presupuesto3.copy()
print("DIMENSIONES DATAFRAME *df_presupuesto2* :", df_presupuesto2.shape)
df_presupuesto3 = df_presupuesto2.copy()

DIMENSIONES DATAFRAME *df_tabla_presupuesto3* : (388, 5)
DIMENSIONES DATAFRAME *df_presupuesto2* : (70, 4)


In [422]:
df_tabla_presupuesto4 = df_tabla_presupuesto4[['CODIGO_PROYECTO', 'TIPO_COSTO', 'TOTAL', 'ORIGEN']]
df_tabla_presupuesto4.rename(columns={'TIPO_COSTO': 'TIPO_CUENTA'}, inplace=True)

df_tabla_presupuesto4.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
303,P-1705,2. COSTO MATERIALES,-21364144.00,TABLA_PRESUPUESTO
348,P-1721,3. COSTO MANO DE OBRA,0.00,TABLA_PRESUPUESTO
132,P-1646,2. COSTO MATERIALES,-13088304.00,TABLA_PRESUPUESTO
108,P-1638,2. COSTO MATERIALES,-16268360.00,TABLA_PRESUPUESTO
142,P-1649,3. COSTO MANO DE OBRA,-38928950.00,TABLA_PRESUPUESTO
80,P-1628,4. COSTO SUPERVISIÓN,-509390.00,TABLA_PRESUPUESTO
38,P-1614,4. COSTO SUPERVISIÓN,-2114140.00,TABLA_PRESUPUESTO
346,P-1720,3. COSTO MANO DE OBRA,0.00,TABLA_PRESUPUESTO
45,P-1617,2. COSTO MATERIALES,-14244564.00,TABLA_PRESUPUESTO
238,P-1681,3. COSTO MANO DE OBRA,-2136024.00,TABLA_PRESUPUESTO


In [423]:
df_presupuesto3.rename(columns={'VALOR': 'TOTAL'}, inplace=True)
df_presupuesto3.sample(10)

,CODIGO_PROYECTO,TOTAL,TIPO_CUENTA,ORIGEN
23,P-1679,169118040.00,1. INGRESO,PLANILLA PRESUPUESTO
41,P-1697,97580171.14,1. INGRESO,PLANILLA PRESUPUESTO
35,P-1691,5751334.03,1. INGRESO,PLANILLA PRESUPUESTO
20,P-1676,153113097.14,1. INGRESO,PLANILLA PRESUPUESTO
34,P-1690,64872885.78,1. INGRESO,PLANILLA PRESUPUESTO
24,P-1680,206761360.91,1. INGRESO,PLANILLA PRESUPUESTO
2,P-1638,43206282.00,1. INGRESO,PLANILLA PRESUPUESTO
60,P-1718,184014731.01,1. INGRESO,PLANILLA PRESUPUESTO
5,P-1654,23880352.49,1. INGRESO,PLANILLA PRESUPUESTO
27,P-1683,57098333.68,1. INGRESO,PLANILLA PRESUPUESTO


#### 4.11.2 SE CONCATENA LOS DATAFRAMES

In [424]:
# Imprimir las dimensiones de los DataFrames
print("Dimensiones de df_tabla_presupuesto4:", df_tabla_presupuesto4.shape)
print("Dimensiones de df_presupuesto3:", df_presupuesto3.shape)

# Concatenar los DataFrames
df_presupuesto_final = pd.concat([df_tabla_presupuesto4, df_presupuesto3], ignore_index=True)
print("Dimensiones de df_presupuesto_final:", df_presupuesto_final.shape)

df_presupuesto_final.sample(10)

Dimensiones de df_tabla_presupuesto4: (388, 4)
Dimensiones de df_presupuesto3: (70, 4)
Dimensiones de df_presupuesto_final: (458, 4)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL,ORIGEN
414,P-1683,1. INGRESO,57098333.68,PLANILLA PRESUPUESTO
194,P-1666,4. COSTO SUPERVISIÓN,-1955309.00,TABLA_PRESUPUESTO
387,S-1601,4. COSTO SUPERVISIÓN,-819548.00,TABLA_PRESUPUESTO
63,P-1623,2. COSTO MATERIALES,-15611718.00,TABLA_PRESUPUESTO
135,P-1647,2. COSTO MATERIALES,-13523550.00,TABLA_PRESUPUESTO
280,P-1696,2. COSTO MATERIALES,-30348318.00,TABLA_PRESUPUESTO
200,P-1668,4. COSTO SUPERVISIÓN,-1557545.00,TABLA_PRESUPUESTO
271,P-1692,4. COSTO SUPERVISIÓN,-5768030.00,TABLA_PRESUPUESTO
26,P-1610,4. COSTO SUPERVISIÓN,-2263170.00,TABLA_PRESUPUESTO
179,P-1661,4. COSTO SUPERVISIÓN,-2015626.00,TABLA_PRESUPUESTO


#### 4.11.3 SE REALIZA MERGE CON CODIGOS DE PROYECTO

In [434]:
# Se hace merge con los codigos de proyecto.

print("Dimensiones de df_libro_mayor5_base2", df_libro_mayor5_base2.shape)

df_presupuesto_final2 = pd.merge(df_libro_mayor5_base2, df_presupuesto_final, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("Dimensiones de df_presupuesto_final2", df_presupuesto_final2.shape)
df_presupuesto_final2.sample(10)

Dimensiones de df_libro_mayor5_base2 (340, 5)
Dimensiones de df_presupuesto_final2 (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
94,P-1699,3. COSTO MANO DE OBRA,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.,0.00,TABLA_PRESUPUESTO
251,P-1669,4. COSTO SUPERVISIÓN,0,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,-2037555.00,TABLA_PRESUPUESTO
37,P-1674,2. COSTO MATERIALES,0,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,-12233776.00,TABLA_PRESUPUESTO
79,P-1693,2. COSTO MATERIALES,0,VOLTA V0800,ECOBIO SPA,-84368545.00,TABLA_PRESUPUESTO
85,P-1695,1. INGRESO,0,EDIFICIO ESTORIL,EMPRESA CONSTRUCTORA BRAVO E IZQUIERDO,94918400.00,PLANILLA PRESUPUESTO
21,P-1658,2. COSTO MATERIALES,0,VIVIPRA,VICHERAT Y PRADENAS LTDA.,-38335303.00,TABLA_PRESUPUESTO
320,P-1720,2. COSTO MATERIALES,0,NaN,NaN,-21939079.00,TABLA_PRESUPUESTO
143,P-1719,1. INGRESO,0,CD METALPAR,EBCO S.A.,194809188.00,PLANILLA PRESUPUESTO
184,P-1612,3. COSTO MANO DE OBRA,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,-11192188.00,TABLA_PRESUPUESTO
23,P-1661,2. COSTO MATERIALES,0,SAN VICENTE DE PAUL,DANACORP S.A,-15916097.00,TABLA_PRESUPUESTO


#### 4.11.4 SE AGREGAN FILAS DE "1.TOTAL"

In [436]:
# Calcular la suma de los valores totales para cada tipo de cuenta


df_total = df_presupuesto_final2.groupby('TIPO_CUENTA')['TOTAL'].sum().reset_index()
df_total['CODIGO_PROYECTO'] = '1. TOTAL'

# Agregar las filas adicionales al DataFrame
df_presupuesto_final2 = pd.concat([df_presupuesto_final2, df_total], ignore_index=True)

df_presupuesto_final2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
319,P-1719,4. COSTO SUPERVISIÓN,0.00,CD METALPAR,EBCO S.A.,NaN,NaN
180,P-1611,1. INGRESO,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,NaN,NaN
307,P-1709,4. COSTO SUPERVISIÓN,0.00,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,-1433251.00,TABLA_PRESUPUESTO
24,P-1663,2. COSTO MATERIALES,0.00,OFICINAS EQUIFAX,CONSTRUCTORA VRK INGENIERIA LIMITADA,-28692183.00,TABLA_PRESUPUESTO
164,P-1727,1. INGRESO,0.00,NaN,NaN,166727122.73,PLANILLA PRESUPUESTO
144,P-1720,3. COSTO MANO DE OBRA,0.00,NaN,NaN,0.00,TABLA_PRESUPUESTO
328,P-1725,4. COSTO SUPERVISIÓN,0.00,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4468043.00,TABLA_PRESUPUESTO
132,P-1714,1. INGRESO,0.00,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,61454685.57,PLANILLA PRESUPUESTO
234,P-1661,3. COSTO MANO DE OBRA,0.00,SAN VICENTE DE PAUL,DANACORP S.A,-8083499.00,TABLA_PRESUPUESTO
92,P-1698,2. COSTO MATERIALES,0.00,CD ANDES,EBCO S.A.,-27960522.00,TABLA_PRESUPUESTO


In [437]:
# Modificar las columnas "NOMBRE_OBRA" y "NOMBRE_CLIENTE" para "1. TOTAL"
df_presupuesto_final2.loc[df_presupuesto_final2['CODIGO_PROYECTO'] == '1. TOTAL', ['NOMBRE_OBRA', 'NOMBRE_CLIENTE']] = '1. TOTAL'

df_presupuesto_final2.sample(10)      

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN
18,P-1649,2. COSTO MATERIALES,0.00,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.,-192491263.00,TABLA_PRESUPUESTO
174,P-1592,1. INGRESO,0.00,NUEVA COSTANERA 3300,EBCO S.A.,NaN,NaN
20,P-1656,2. COSTO MATERIALES,0.00,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,-12184301.00,TABLA_PRESUPUESTO
324,P-1722,3. COSTO MANO DE OBRA,0.00,ECO CERRILLOS,CONSTRUCTORA ICF SPA,0.00,TABLA_PRESUPUESTO
65,P-1688,3. COSTO MANO DE OBRA,0.00,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-7417508.00,TABLA_PRESUPUESTO
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-92338978.00,TABLA_PRESUPUESTO
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-13744197.00,TABLA_PRESUPUESTO
340,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,7806687863.43,NaN
210,P-1644,1. INGRESO,0.00,MIRADOR DE SAN CRISTOBAL,EBCO S.A.,NaN,NaN
268,P-1679,4. COSTO SUPERVISIÓN,0.00,JUAN SABAJ,CONSTRUCTORA GREVIA,-6052803.00,TABLA_PRESUPUESTO


#### 4.11.5 AGREGACIÓN COLUMNA MARGEN

In [438]:
#CREACIÓN DE MARGENES

df_presupuesto_final3 = df_presupuesto_final2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_presupuesto_final3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 : ", df_presupuesto_final3.shape)

df_presupuesto_final4 = pd.merge(df_presupuesto_final3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 : ", df_presupuesto_final4.shape)
df_presupuesto_final4.sample(10)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_presupuesto_final3 :  (344, 7)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4 :  (344, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
325,P-1722,4. COSTO SUPERVISIÓN,0.00,ECO CERRILLOS,CONSTRUCTORA ICF SPA,NaN,NaN,0.56
97,P-1700,3. COSTO MANO DE OBRA,0.00,CD ORIZON,EBCO S.A.,-22319794.00,TABLA_PRESUPUESTO,0.36
38,P-1675,2. COSTO MATERIALES,0.00,PALENA II,CONSTRUCTORA MYA SPA.,-7361719.00,TABLA_PRESUPUESTO,0.41
226,P-1656,4. COSTO SUPERVISIÓN,0.00,MIRADOR CLUB HÍPICO,CONSTRUCTORA VITAL EDIFICACION SPA,-1293524.00,TABLA_PRESUPUESTO,0.49
221,P-1653,1. INGRESO,0.00,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,NaN,NaN,0.00
276,P-1683,4. COSTO SUPERVISIÓN,0.00,CD BP-A05,EBCO S.A.,-1865714.00,TABLA_PRESUPUESTO,0.31
68,P-1689,2. COSTO MATERIALES,0.00,CASINO BUENAVENTURA,EBCO S.A.,-10722527.00,TABLA_PRESUPUESTO,0.36
265,P-1676,4. COSTO SUPERVISIÓN,0.00,LO CAMPINO,EBCO S.A.,-8951416.00,TABLA_PRESUPUESTO,0.39
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,NaN,NaN,0.00
15,P-1645,2. COSTO MATERIALES,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,-8503715.00,TABLA_PRESUPUESTO,0.35


In [443]:
# TESTING VISUAL 1

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[~((df_presupuesto_final4['CODIGO_PROYECTO'] == '1. TOTAL') & df_presupuesto_final4['TOTAL'].isna())]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "1. TOTAL"]

2049707761.133747


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
340,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,7806687863.43,NaN,0.26
341,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,-4375161031.30,NaN,0.26
342,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,-1139100986.00,NaN,0.26
343,1. TOTAL,4. COSTO SUPERVISIÓN,NaN,1. TOTAL,1. TOTAL,-242718085.00,NaN,0.26


In [444]:
# TESTING VISUAL 2

print(df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]["TOTAL"].sum())
df_presupuesto_final4 = df_presupuesto_final4[(df_presupuesto_final4['CODIGO_PROYECTO'] != '1. TOTAL') | (df_presupuesto_final4['ORIGEN'] != 'AGREGACIÓN TIPO_CUENTA VALOR O')]
df_presupuesto_final4[df_presupuesto_final4["CODIGO_PROYECTO"] == "P-1704"]

483525857.8694887


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL,ORIGEN,MARGEN
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,TABLA_PRESUPUESTO,0.46
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,TABLA_PRESUPUESTO,0.46
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,PLANILLA PRESUPUESTO,0.46
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,TABLA_PRESUPUESTO,0.46


In [445]:
df_presupuesto_final4.to_csv('archivos_respaldo/df_presupuesto_final4.csv', index = False)

### 4.12 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO)

#### 4.12.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [449]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
305,P-1707,4. COSTO SUPERVISIÓN,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.
51,P-1682,3. COSTO MANO DE OBRA,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.
277,P-1684,4. COSTO SUPERVISIÓN,0,EDIFICIO GOYCOLEA,EBCO S.A.
171,P-1591,1. INGRESO,0,EDIFICIO ESTORIL,EBCO S.A.
244,P-1666,4. COSTO SUPERVISIÓN,0,CD AXIONLONG,EBCO S.A.


In [451]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1 = df_presupuesto_final4_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1 = pd.DataFrame(df_presupuesto_final4_1.to_records())

df_presupuesto_final4_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 : ", df_presupuesto_final4_1.shape)

df_presupuesto_final4_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
86,P-1658,3. COSTO MANO DE OBRA,-8470198.00
206,P-1693,3. COSTO MANO DE OBRA,-11371251.00
106,P-1666,3. COSTO MANO DE OBRA,-8652090.00
220,P-1697,1. INGRESO,97580171.14
18,P-1603,3. COSTO MANO DE OBRA,-4345810.00
319,P-1724,4. COSTO SUPERVISIÓN,0.00
269,P-1712,2. COSTO MATERIALES,-104292394.00
261,P-1708,2. COSTO MATERIALES,-83658973.00
35,P-1626,4. COSTO SUPERVISIÓN,-8577318.00
207,P-1693,4. COSTO SUPERVISIÓN,-2506554.00


In [452]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_1= df_ptd_cerrado3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_2 = df_ptd_cerrado3_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_2 = pd.DataFrame(df_ptd_cerrado3_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 : ", df_ptd_cerrado3_1.shape)
df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 : ", df_ptd_cerrado3_2.shape)

df_ptd_cerrado3_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
333,P-1728,2. COSTO MATERIALES,-1822779.00
144,P-1677,1. INGRESO,90155711.00
149,P-1679,2. COSTO MATERIALES,-96622.00
213,P-1695,2. COSTO MATERIALES,-25647031.00
152,P-1680,1. INGRESO,188764235.00
86,P-1658,3. COSTO MANO DE OBRA,-319859.00
62,P-1644,3. COSTO MANO DE OBRA,-341190.00
56,P-1643,1. INGRESO,0.00
45,P-1639,2. COSTO MATERIALES,-11381381.00
221,P-1697,2. COSTO MATERIALES,-28478810.00


#### 4.12.2 MERGE ENTRE DATAFRAMES 

In [453]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    : ", df_presupuesto_final4_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente = df_libro_mayor5_base2.merge(df_presupuesto_final4_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)

df_pendiente.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1    :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
235,P-1661,4. COSTO SUPERVISIÓN,0,SAN VICENTE DE PAUL,DANACORP S.A,-2015626.00
61,P-1686,1. INGRESO,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,105081270.13
44,P-1677,1. INGRESO,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,83882285.07
323,P-1722,2. COSTO MATERIALES,0,ECO CERRILLOS,CONSTRUCTORA ICF SPA,-124836610.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00


In [454]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               : ", df_pendiente.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          : ", df_ptd_cerrado3_2.shape)

df_pendiente2 = df_pendiente.merge(df_ptd_cerrado3_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              : ", df_pendiente2.shape)
df_pendiente2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente               :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
322,P-1721,4. COSTO SUPERVISIÓN,0,STRIP CENTER LUIS PASTEUR,INMOBILIARIA LUIS PASTEUR S.A.,0.00,0.00
301,P-1703,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA DANACORP,DANACORP S.A,-2728704.00,0.00
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92338978.00,-13655419.00
265,P-1676,4. COSTO SUPERVISIÓN,0,LO CAMPINO,EBCO S.A.,-8951416.00,0.00
180,P-1611,1. INGRESO,0,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,4173520.00


In [455]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-433653.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


In [456]:
# TESTING VISUAL 2
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4503192575.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375161031.30,-1686850217.00
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00


#### 4.12.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [458]:
# Modificar los valores negativos a positivos
df_pendiente2['TOTAL_PREP2'] = np.where(df_pendiente2['TOTAL_PREP'] < 0, df_pendiente2['TOTAL_PREP'] * -1, df_pendiente2['TOTAL_PREP'])

df_pendiente2['TOTAL_PDT2'] = np.where(df_pendiente2['TOTAL_PDT'] < 0, df_pendiente2['TOTAL_PDT'] * -1, df_pendiente2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente2['TOTAL_1'] = df_pendiente2['TOTAL_PREP2'] - df_pendiente2['TOTAL_PDT2']

df_pendiente2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
177,P-1603,1. INGRESO,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,0.00,0.00,0.00,0.00,0.00
71,P-1690,1. INGRESO,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,64872885.78,52317842.00,64872885.78,52317842.00,12555043.78
36,P-1673,2. COSTO MATERIALES,0,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,-10704866.00,-11666275.00,10704866.00,11666275.00,-961409.00
221,P-1653,1. INGRESO,0,EDUARDO DE LA BARRA,CONSTRUCTORA E INVERSIONES VITAL SPA.,0.00,13787411.00,0.00,13787411.00,-13787411.00
77,P-1692,1. INGRESO,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,91720000.00,85099786.00,91720000.00,85099786.00,6620214.00


In [459]:
# TESTING VISUAL 1
 
df_pendiente2[df_pendiente2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-145534854.00,156095149.00,145534854.00,10560295.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-197683461.00,387816472.00,197683461.00,190133011.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,686431807.00,1058718831.87,686431807.00,372287024.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [460]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente2['TOTAL'] = df_pendiente2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente2['TOTAL_1'] != 0)
df_pendiente2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente2.loc[df_pendiente2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
75,P-1692,3. COSTO MANO DE OBRA,0,JOSÉ PEREZ COTAPOS,EBCO S.A.,-23173688.00,-45696730.00,23173688.00,45696730.00,-22523042.00,-22523042.00
266,P-1677,4. COSTO SUPERVISIÓN,0,GALVARINO GALLARDO,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-4657579.00,0.00,4657579.00,0.00,4657579.00,-4657579.00
152,P-1723,1. INGRESO,0,VILLA PANAMERICANA LOTE C,CONSTRUCTORA MENA Y OVALLE S.A.,328203511.36,86650911.00,328203511.36,86650911.00,241552600.36,241552600.36
71,P-1690,1. INGRESO,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,64872885.78,52317842.00,64872885.78,52317842.00,12555043.78,12555043.78
88,P-1696,1. INGRESO,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,75985768.29,75515364.00,75985768.29,75515364.00,470404.29,470404.29
288,P-1691,4. COSTO SUPERVISIÓN,0,ROSARIO ROSALES,CONSTRUCTORA E INMOBILIARIA PRADOVERDE LTDA.,-2099697.00,0.00,2099697.00,0.00,2099697.00,-2099697.00
179,P-1603,4. COSTO SUPERVISIÓN,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-869162.00,0.00,869162.00,0.00,869162.00,-869162.00
328,P-1725,4. COSTO SUPERVISIÓN,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-4468043.00,0.00,4468043.00,0.00,4468043.00,-4468043.00
167,P-1729,2. COSTO MATERIALES,0,NaN,NaN,-51223238.00,0.00,51223238.00,0.00,51223238.00,-51223238.00
87,P-1696,2. COSTO MATERIALES,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-30348318.00,-30852143.00,30348318.00,30852143.00,-503825.00,-503825.00


In [461]:
df_pendiente3 = df_pendiente2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)

df_pendiente3 = pd.merge(df_pendiente3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 : ", df_pendiente3.shape)
df_pendiente3.sample(5)


DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
157,P-1725,2. COSTO MATERIALES,0,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-46312307.00,-10888672.00,46312307.00,10888672.00,35423635.00,-35423635.00,0.31
273,P-1682,4. COSTO SUPERVISIÓN,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.00,0.00,0.00,-5.72
61,P-1686,1. INGRESO,0,MACUL ICALMA,EMPRESA CONSTRUCTORA INGEVEC S.A.,105081270.13,99626406.00,105081270.13,99626406.00,5454864.13,5454864.13,-3.45
171,P-1591,1. INGRESO,0,EDIFICIO ESTORIL,EBCO S.A.,0.00,0.00,0.00,0.00,0.00,0.00,0.00
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.00,0.00,0.00,0.59


In [462]:
# TESTING VISUAL 1
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-8169893.00,0.00,8169893.00,-8169893.00,-8169893.00,0.37
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92338978.00,-13655419.00,92338978.00,13655419.00,78683559.00,-78683559.00,0.37
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,28570952.00,166727122.73,28570952.00,138156170.73,138156170.73,0.37
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.37


In [463]:
# TESTING VISUAL 2
 
df_pendiente3[df_pendiente3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-74516421.00,242718085.00,74516421.00,168201664.00,-168201664.00,0.12
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4503192575.00,7806687863.43,4503192575.00,3303495288.43,3303495288.43,0.12
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375161031.30,-1686850217.00,4375161031.30,1686850217.00,2688310814.30,-2688310814.30,0.12
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-1076409161.00,1139100986.00,1076409161.00,62691825.00,-62691825.00,0.12


In [464]:
df_pendiente3.to_csv('archivos_respaldo/df_pendiente3.csv', index = False)

### 4.13 MERGE PENDIENTE (PRESUPUESTO -PDT CERRADO) (PERSPECTIVA MES ANTERIOR)

#### 4.13.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [467]:
# Exploración de dataframe base "df_libro_mayor5_base2"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 : ", df_libro_mayor5_base2.shape)
df_libro_mayor5_base2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_libro_mayor5_base2 :  (340, 5)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE
95,P-1699,2. COSTO MATERIALES,0,VILLA PANAMERICANA LOTE B,EMPRESA CONSTRUCTORA DLP S.A.
70,P-1690,2. COSTO MATERIALES,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.
169,P-1557,2. COSTO MATERIALES,0,ROSARIO ROSALES,EBCO S.A.
130,P-1714,3. COSTO MANO DE OBRA,0,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA
218,P-1649,1. INGRESO,0,INACAP VITACURA E2,CONSTRUCTORA CUEVAS Y PURCELL S.A.


In [468]:
# Exploración y tratamiento de dataframe de presupuesto "df_presupuesto_final4"

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 : ", df_presupuesto_final4.shape)

df_presupuesto_final4_1_1 = df_presupuesto_final4.copy()

df_presupuesto_final4_1_1 = df_presupuesto_final4_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_presupuesto_final4_1_1 = pd.DataFrame(df_presupuesto_final4_1_1.to_records())

df_presupuesto_final4_1_1.rename(columns={'TOTAL': 'TOTAL_PREP'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 : ", df_presupuesto_final4_1_1.shape)

df_presupuesto_final4_1_1.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_presupuesto_final4 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_presupuesto_final4_1_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PREP
327,P-1726,4. COSTO SUPERVISIÓN,0.00
290,P-1717,3. COSTO MANO DE OBRA,0.00
73,P-1649,2. COSTO MATERIALES,-192491263.00
188,P-1689,1. INGRESO,20986779.50
69,P-1648,2. COSTO MATERIALES,-220215219.50
120,P-1670,1. INGRESO,202553946.00
326,P-1726,3. COSTO MANO DE OBRA,0.00
215,P-1695,4. COSTO SUPERVISIÓN,-4879045.00
103,P-1664,4. COSTO SUPERVISIÓN,-76676.00
118,P-1669,3. COSTO MANO DE OBRA,-9423692.00


In [480]:
# Exploración y tratamiento de dataframe de presupuesto "df_ptd_cerrado3"

# Se Crea una copia del dataframe df_ptd_cerrado3 para trabajarlo 

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 : ", df_ptd_cerrado_anterior3.shape)

df_ptd_cerrado3_1_1 = df_ptd_cerrado_anterior3.drop(["NOMBRE_OBRA","NOMBRE_CLIENTE","TOTAL_LB","TOTAL YTD","TOTAL LB OLD","MARGEN"], axis=1)

df_ptd_cerrado3_1_2 = df_ptd_cerrado3_1_1.pivot_table(values="TOTAL", index=['CODIGO_PROYECTO', 'TIPO_CUENTA'], aggfunc='sum')
df_ptd_cerrado3_1_2 = pd.DataFrame(df_ptd_cerrado3_1_2.to_records())

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)
df_ptd_cerrado3_1_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 : ", df_ptd_cerrado3_1_2.shape)

df_ptd_cerrado3_1_2.sample(10)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME  : df_ptd_cerrado_anterior3 :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME  : df_ptd_cerrado3_1_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT
75,P-1649,4. COSTO SUPERVISIÓN,0.00
148,P-1679,1. INGRESO,0.00
235,P-1700,4. COSTO SUPERVISIÓN,0.00
228,P-1699,1. INGRESO,73159443.00
38,P-1632,3. COSTO MANO DE OBRA,0.00
196,P-1691,1. INGRESO,41603116.00
265,P-1709,2. COSTO MATERIALES,-31036355.00
103,P-1664,4. COSTO SUPERVISIÓN,0.00
221,P-1697,2. COSTO MATERIALES,-25685194.00
98,P-1663,3. COSTO MANO DE OBRA,0.00


#### 4.13.2 MERGE ENTRE DATAFRAMES 

In [481]:
# MERGE df_libro_mayor5_2 y df_presupuesto_final5_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      : ", df_libro_mayor5_base2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  : ", df_presupuesto_final4_1_1.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_libro_mayor5_base2.merge(df_presupuesto_final4_1_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)

df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_libro_mayor5_base2      :  (340, 5)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_1_1  :  (340, 3)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP
137,P-1716,2. COSTO MATERIALES,0,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-56794940.00
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375161031.30
116,P-1707,2. COSTO MATERIALES,0,LAS TRANQUERAS,CONSTRUCTORA SANTOLAYA LTDA.,-23066986.00
313,P-1717,1. INGRESO,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,170727176.42
314,P-1717,3. COSTO MANO DE OBRA,0,ECO QUILÍN I,CONSTRUCTORA ICF SPA,0.00


In [482]:
# MERGE df_pendiente y df_ptd_cerrado3_2

print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             : ", df_pendiente_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        : ", df_ptd_cerrado3_1_2.shape)

df_pendiente_2 = df_pendiente_2.merge(df_ptd_cerrado3_1_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              : ", df_pendiente_2.shape)
df_pendiente_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_pendiente_2             :  (340, 6)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_1_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pendiente_2              :  (340, 7)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
301,P-1703,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA DANACORP,DANACORP S.A,-2728704.00,0.00
193,P-1632,3. COSTO MANO DE OBRA,0,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,-43457804.00,0.00
121,P-1709,3. COSTO MANO DE OBRA,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,-8592182.00,-11652547.00
123,P-1709,1. INGRESO,0,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,105853124.47,72858203.00
1,P-1591,3. COSTO MANO DE OBRA,0,EDIFICIO ESTORIL,EBCO S.A.,0.00,-2483998.00


In [483]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1643"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT
13,P-1643,2. COSTO MATERIALES,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,-49797008.30,-102685.00
207,P-1643,1. INGRESO,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
208,P-1643,3. COSTO MANO DE OBRA,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00
209,P-1643,4. COSTO SUPERVISIÓN,0,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00


#### 4.13.3 CALCULO DE COLUMNA TOTAL

El objetivo de este Dataframe es lograr identificar la diferencia entre el el presupuesto y el PDT Cerrado.

In [484]:
# Modificar los valores negativos a positivos
df_pendiente_2['TOTAL_PREP2'] = np.where(df_pendiente_2['TOTAL_PREP'] < 0, df_pendiente_2['TOTAL_PREP'] * -1, df_pendiente_2['TOTAL_PREP'])

df_pendiente_2['TOTAL_PDT2'] = np.where(df_pendiente_2['TOTAL_PDT'] < 0, df_pendiente_2['TOTAL_PDT'] * -1, df_pendiente_2['TOTAL_PDT'])

# Calcular la columna "TOTAL" con los valores positivos
df_pendiente_2['TOTAL_1'] = df_pendiente_2['TOTAL_PREP2'] - df_pendiente_2['TOTAL_PDT2']

df_pendiente_2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
66,P-1688,2. COSTO MATERIALES,0,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,-20308610.00,-22268956.00,20308610.00,22268956.00,-1960346.00
28,P-1667,2. COSTO MATERIALES,0,ECO PLACILLA,CONSTRUCTORA ICF SPA,-36944607.00,-45566253.00,36944607.00,45566253.00,-8621646.00
333,P-1729,1. INGRESO,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00
165,P-1728,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-630869.00,0.00,630869.00,-630869.00
287,P-1690,4. COSTO SUPERVISIÓN,0,ONGOLMO - HOME CONDOMINIO,CANTAUCO CONSTRUCCION LTDA.,0.00,0.00,0.00,0.00,0.00


In [485]:
# TESTING VISUAL 1
 
df_pendiente_2[df_pendiente_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-132643971.00,156095149.00,132643971.00,23451178.00
107,P-1704,2. COSTO MATERIALES,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-387816472.00,-183715910.00,387816472.00,183715910.00,204100562.00
108,P-1704,1. INGRESO,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,1058718831.87,607371810.00,1058718831.87,607371810.00,451347021.87
302,P-1704,4. COSTO SUPERVISIÓN,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-31281353.00,0.00,31281353.00,0.00,31281353.00


In [486]:
# Se realiza estandarización de columna TOTAL considerando que los Tipos de cuenta de Costos deben estar en valor negativo.

# Crear nueva columna 'TOTAL' con el valor de 'TOTAL_PREV'
df_pendiente_2['TOTAL'] = df_pendiente_2['TOTAL_1']

# Aplicar condiciones para cambiar el valor a negativo
mask = (df_pendiente_2['TIPO_CUENTA'].isin(['2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN'])) & (df_pendiente_2['TOTAL_1'] != 0)
df_pendiente_2.loc[mask, 'TOTAL'] *= -1

# Mantener el valor de 'TOTAL_PREV' como negativo si ya es negativo
df_pendiente_2.loc[df_pendiente_2['TOTAL_1'] < 0, 'TOTAL'] *= -1

df_pendiente_2.sample(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL
178,P-1603,3. COSTO MANO DE OBRA,0,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-4345810.00,0.00,4345810.00,0.00,4345810.00,-4345810.00
106,P-1704,3. COSTO MANO DE OBRA,0,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-156095149.00,-132643971.00,156095149.00,132643971.00,23451178.00,-23451178.00
33,P-1670,1. INGRESO,0,MK II SEGURIDAD Y REDES,COMERCIAL K SPA.,202553946.00,164021639.00,202553946.00,164021639.00,38532307.00,38532307.00
276,P-1683,4. COSTO SUPERVISIÓN,0,CD BP-A05,EBCO S.A.,-1865714.00,0.00,1865714.00,0.00,1865714.00,-1865714.00
98,P-1700,2. COSTO MATERIALES,0,CD ORIZON,EBCO S.A.,-72938784.00,-63386553.00,72938784.00,63386553.00,9552231.00,-9552231.00
88,P-1696,1. INGRESO,0,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,75985768.29,75515364.00,75985768.29,75515364.00,470404.29,470404.29
128,P-1713,2. COSTO MATERIALES,0,RESERVA SAN FRANCISCO,EMPRESA NACIONAL DE CONSTRUCCIÓN LAS VERTIENTE...,-55849894.00,-1865409.00,55849894.00,1865409.00,53984485.00,-53984485.00
296,P-1698,3. COSTO MANO DE OBRA,0,CD ANDES,EBCO S.A.,-6815995.00,-19007691.00,6815995.00,19007691.00,-12191696.00,-12191696.00
52,P-1682,2. COSTO MATERIALES,0,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,-23788513.00,-26234374.00,23788513.00,26234374.00,-2445861.00,-2445861.00
17,P-1648,2. COSTO MATERIALES,0,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,-220215219.50,786912.00,220215219.50,786912.00,219428307.50,-219428307.50


In [487]:
df_pendiente_3 = df_pendiente_2.copy()

# Esta línea de código es para evitar ver el error.
with np.errstate(divide='ignore', invalid='ignore'):

    df_margen = df_pendiente_3.groupby(['CODIGO_PROYECTO']).apply(lambda x: pd.Series({
        'MARGEN': x.loc[x['TIPO_CUENTA'].isin(['1. INGRESO','2. COSTO MATERIALES', '3. COSTO MANO DE OBRA', '4. COSTO SUPERVISIÓN']), 'TOTAL'].sum() / x.loc[x['TIPO_CUENTA'] == '1. INGRESO', 'TOTAL'].sum()
    }))

df_margen = pd.DataFrame(df_margen.to_records())
df_margen['MARGEN'] = df_margen['MARGEN'].fillna(0).replace(-np.inf, 0)
df_margen.replace([np.inf, -np.inf, np.nan], 0, inplace=True)


# mostrar el DataFrame resultante
df_margen.sample(10)


print("DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)

df_pendiente_3 = pd.merge(df_pendiente_3, df_margen, on=['CODIGO_PROYECTO'], how='left')

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 : ", df_pendiente_3.shape)
df_pendiente_3.sample(5)

DIMENSIONES ANTES   DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 11)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_pendiente_3 :  (340, 12)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
185,P-1612,4. COSTO SUPERVISIÓN,0,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,-2259746.00,0.00,2259746.00,0.00,2259746.00,-2259746.00,0.00
301,P-1703,4. COSTO SUPERVISIÓN,0,SANTA VICTORIA DANACORP,DANACORP S.A,-2728704.00,0.00,2728704.00,0.00,2728704.00,-2728704.00,-1.40
46,P-1679,2. COSTO MATERIALES,0,JUAN SABAJ,CONSTRUCTORA GREVIA,-61846896.00,-96622.00,61846896.00,96622.00,61750274.00,-61750274.00,0.43
249,P-1668,4. COSTO SUPERVISIÓN,0,QUILLAYES PLAZA,E.MOLINA MOREL CONSTRUCTORA S.A.,-1557545.00,0.00,1557545.00,0.00,1557545.00,-1557545.00,-0.77
263,P-1675,4. COSTO SUPERVISIÓN,0,PALENA II,CONSTRUCTORA MYA SPA.,-466503.00,0.00,466503.00,0.00,466503.00,-466503.00,-13.08


In [488]:
# TESTING VISUAL 1
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
162,P-1727,3. COSTO MANO DE OBRA,0,NaN,NaN,0.00,-4714680.00,0.00,4714680.00,-4714680.00,-4714680.00,0.40
163,P-1727,2. COSTO MATERIALES,0,NaN,NaN,-92338978.00,-3753865.00,92338978.00,3753865.00,88585113.00,-88585113.00,0.40
164,P-1727,1. INGRESO,0,NaN,NaN,166727122.73,11005000.00,166727122.73,11005000.00,155722122.73,155722122.73,0.40
330,P-1727,4. COSTO SUPERVISIÓN,0,NaN,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0.40


In [489]:
# TESTING VISUAL 2
 
df_pendiente_3[df_pendiente_3["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL_PREP,TOTAL_PDT,TOTAL_PREP2,TOTAL_PDT2,TOTAL_1,TOTAL,MARGEN
336,1. TOTAL,4. COSTO SUPERVISIÓN,0,1. TOTAL,1. TOTAL,-242718085.00,-69873160.00,242718085.00,69873160.00,172844925.00,-172844925.00,0.15
337,1. TOTAL,1. INGRESO,0,1. TOTAL,1. TOTAL,7806687863.43,4042394062.00,7806687863.43,4042394062.00,3764293801.43,3764293801.43,0.15
338,1. TOTAL,2. COSTO MATERIALES,0,1. TOTAL,1. TOTAL,-4375161031.30,-1517070731.00,4375161031.30,1517070731.00,2858090300.30,-2858090300.30,0.15
339,1. TOTAL,3. COSTO MANO DE OBRA,0,1. TOTAL,1. TOTAL,-1139100986.00,-962035446.00,1139100986.00,962035446.00,177065540.00,-177065540.00,0.15


In [490]:
df_pendiente_3.to_csv('archivos_respaldo/df_pendiente_3.csv', index = False)

### 4.14 MERGE PDT CERRADO VS (PDT + PROYECTADO)

#### 4.14.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [493]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2 = df_ptd_cerrado3.copy()

df_ptd_cerrado3_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
207,P-1643,1. INGRESO,0.00,CHORRILLOS M4,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.00
153,P-1724,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-506066.00,0.00,-506066.00,0.72
87,P-1696,2. COSTO MATERIALES,0.00,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,-20459616.00,-10392527.00,-30852143.00,0.13
201,P-1641,1. INGRESO,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0.00,0.00,0.00,0.00
79,P-1693,2. COSTO MATERIALES,0.00,VOLTA V0800,ECOBIO SPA,103024.00,-89715133.00,-89612109.00,0.16


In [495]:
df_ptd_cerrado_y_proyectado2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
329,P-1726,4. COSTO SUPERVISIÓN,NaN,NaN,0.00,0,0.00,0.31
278,P-1685,1. INGRESO,MAPA EDIFICIOS NO OPERACIONALES,CLARO VICUÑA VALENZUELA S.A.,48965677.00,0,48965677.00,0.04
40,P-1676,3. COSTO MANO DE OBRA,LO CAMPINO,EBCO S.A.,-29320930.00,0,-29320930.00,0.59
25,P-1664,2. COSTO MATERIALES,MK II PCI,COMERCIAL K SPA.,-50055425.00,0,-50055425.00,0.64
284,P-1689,1. INGRESO,CASINO BUENAVENTURA,EBCO S.A.,22795459.00,0,22795459.00,0.51


In [496]:
# Exploración de dataframe presupuesto

df_ptd_cerrado_y_proyectado2_1 = df_ptd_cerrado_y_proyectado2.copy()
df_ptd_cerrado_y_proyectado2_1.rename(columns={'TOTAL': 'TOTAL_PDT_PROY'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_ptd_cerrado_y_proyectado2_1 = pd.pivot_table(df_ptd_cerrado_y_proyectado2_1, values=['TOTAL_PDT_PROY'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_ptd_cerrado_y_proyectado2_1 = pd.DataFrame(df_ptd_cerrado_y_proyectado2_1.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_ptd_cerrado_y_proyectado2_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PROY
95,P-1661,4. COSTO SUPERVISIÓN,0
143,P-1676,4. COSTO SUPERVISIÓN,0
88,P-1660,1. INGRESO,26532592
250,P-1705,3. COSTO MANO DE OBRA,-18870981
251,P-1705,4. COSTO SUPERVISIÓN,0


#### 4.14.2 MERGE ENTRE DATAFRAMES

In [497]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  : ", df_ptd_cerrado3_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1     : ", df_ptd_cerrado_y_proyectado2_1.shape)

df_pdt_vs_proy = df_ptd_cerrado3_2.merge(df_ptd_cerrado_y_proyectado2_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy           : ", df_pdt_vs_proy.shape)

df_pdt_vs_proy.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2                  :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_1     :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY
256,P-1673,1. INGRESO,0.00,INACAP TEMUCO - ETAPA A,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,0.00,26810752.00,26810752.00,0.56,26810752
169,P-1557,2. COSTO MATERIALES,0.00,ROSARIO ROSALES,EBCO S.A.,0.00,0.00,0.00,0.00,0
265,P-1676,4. COSTO SUPERVISIÓN,0.00,LO CAMPINO,EBCO S.A.,0.00,0.00,0.00,0.59,0
244,P-1666,4. COSTO SUPERVISIÓN,0.00,CD AXIONLONG,EBCO S.A.,0.00,0.00,0.00,0.63,0
331,P-1728,1. INGRESO,0.00,NaN,NaN,0.00,0.00,0.00,0.00,0


#### 4.14.3 CREACIÓN DE COLUMNA % DE AVANCE

In [498]:
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['TOTAL_PDT'] / df_pdt_vs_proy['TOTAL_PDT_PROY']

# Reemplazar NaN por cero
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_proy['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_proy['% AVANCE'] = df_pdt_vs_proy['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_proy.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-1247014.00,0.00,-1247014.00,0.00,-1548079,0.81
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,0.00,-4593960.00,0.00,-6703922,0.69
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,41716.00,856004.00,897720.00,1.05,897720,1.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-30000,1.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,602825,1.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,310902,1.00
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-16400,1.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,682024,1.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,75965.00,236593.00,312558.00,0.00,312558,1.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-455187,1.00


In [499]:
# TESTING VISUAL 1
 
df_pdt_vs_proy[df_pdt_vs_proy["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-8169893.00,0.00,-8169893.00,0.24,-11625106,0.70
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-13655419.00,0.00,-13655419.00,0.24,-23655419,0.58
164,P-1727,1. INGRESO,0.00,NaN,NaN,28570952.00,0.00,28570952.00,0.24,55070952,0.52
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.24,0,0.00


In [500]:
# TESTING VISUAL 2
 
df_pdt_vs_proy[df_pdt_vs_proy["CODIGO_PROYECTO"]=="1. TOTAL"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
336,1. TOTAL,4. COSTO SUPERVISIÓN,0.00,1. TOTAL,1. TOTAL,-22566666.00,-51949755.00,-74516421.00,0.37,-79159682,0.94
337,1. TOTAL,1. INGRESO,NaN,1. TOTAL,1. TOTAL,NaN,NaN,4503192575.00,0.37,4926992575,0.91
338,1. TOTAL,2. COSTO MATERIALES,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1686850217.00,0.37,-1863386023,0.91
339,1. TOTAL,3. COSTO MANO DE OBRA,NaN,1. TOTAL,1. TOTAL,NaN,NaN,-1076409161.00,0.37,-1190782876,0.90


In [502]:
df_pdt_vs_proy.to_csv('archivos_respaldo/df_pdt_vs_proy.csv', index = False)

### 4.15 MERGE PDT CERRADO VS (PDT + PROYECTADO) PERSPECTIVA MES ANTERIOR

#### 4.15.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [504]:
# Exploración de dataframe df_pdt_cerrado

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 : ", df_ptd_cerrado3.shape)

df_ptd_cerrado3_2_2 = df_ptd_cerrado_anterior3.copy()

df_ptd_cerrado3_2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado3_2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado3 :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN
17,P-1648,2. COSTO MATERIALES,0.00,ETAPA 2 CD PERALILLO TW,INMOBILIARIA Y RENTAS LOGÍSTICA S.A.,200959.00,585953.00,786912.00,2.17
244,P-1666,4. COSTO SUPERVISIÓN,0.00,CD AXIONLONG,EBCO S.A.,0.00,0.00,0.00,0.63
182,P-1611,4. COSTO SUPERVISIÓN,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,0.00,0.00,0.00,1.14
30,P-1669,3. COSTO MANO DE OBRA,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,-156861.00,-3156860.00,-3313721.00,0.59
251,P-1669,4. COSTO SUPERVISIÓN,0.00,DEPARTAMENTAL,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0.00,0.00,0.59


In [505]:
df_ptd_cerrado_y_proyectado_ant2.sample(5)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL,MARGEN
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28,0.48
53,P-1682,1. INGRESO,PLAZA CEPPI,EMP. CONSTRUCTORA INGENIEROS S.A.,55359285.00,18808581.00,74167866.00,0.35
265,P-1676,4. COSTO SUPERVISIÓN,LO CAMPINO,EBCO S.A.,0.00,0.00,0.00,0.59
310,P-1714,4. COSTO SUPERVISIÓN,DEPARTAMENTAL II,CONSTRUCTORA FUERZA SUR DOS SPA,0.00,0.00,0.00,0.01
78,P-1693,3. COSTO MANO DE OBRA,VOLTA V0800,ECOBIO SPA,-25236917.00,-2407425.00,-27644342.00,0.15


In [506]:
# Exploración de dataframe presupuesto

df_ptd_cerrado_y_proyectado_ant2_2 = df_ptd_cerrado_y_proyectado_ant2.copy()
df_ptd_cerrado_y_proyectado_ant2_2.rename(columns={'TOTAL': 'TOTAL_PDT_PROY'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_ptd_cerrado_y_proyectado_ant2_2 = pd.pivot_table(df_ptd_cerrado_y_proyectado_ant2_2, values=['TOTAL_PDT_PROY'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_ptd_cerrado_y_proyectado_ant2_2 = pd.DataFrame(df_ptd_cerrado_y_proyectado_ant2_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_ptd_cerrado_y_proyectado_ant2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PROY
131,P-1673,4. COSTO SUPERVISIÓN,0.00
35,P-1626,4. COSTO SUPERVISIÓN,0.00
71,P-1648,4. COSTO SUPERVISIÓN,0.00
93,P-1661,2. COSTO MATERIALES,-14270769.00
85,P-1658,2. COSTO MATERIALES,-11357166.00


#### 4.15.2 MERGE ENTRE DATAFRAMES

In [507]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_2                  : ", df_ptd_cerrado3_2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2   : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)

df_pdt_vs_proy_2 = df_ptd_cerrado3_2_2.merge(df_ptd_cerrado_y_proyectado_ant2_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy_2           : ", df_pdt_vs_proy_2.shape)

df_pdt_vs_proy_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado3_2_2                  :  (340, 9)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2   :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_vs_proy_2           :  (340, 10)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY
201,P-1641,1. INGRESO,0.00,INACAP TEMUCO,DESARROLLOS CONSTRUCTIVOS AXIS SOCIEDAD ANONIMA,0.00,0.00,0.00,0.00,0.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,602825.00
156,P-1725,3. COSTO MANO DE OBRA,0.00,LO CAÑAS,CONSTRUCTORA BRISAS DE BATUCO,-2705788.00,0.00,-2705788.00,0.23,-4825931.00
295,P-1697,4. COSTO SUPERVISIÓN,0.00,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,0.00,0.00,0.00,0.44,0.00
16,P-1645,1. INGRESO,0.00,LOS TRES ANTONIOS,SOCIEDAD CONSTRUCTORA ECHAVARRI HERMANOS LIMITADA,0.00,17327418.00,17327418.00,0.34,17327418.00


#### 4.15.3 CREACIÓN DE COLUMNA % DE AVANCE

In [508]:
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['TOTAL_PDT'] / df_pdt_vs_proy_2['TOTAL_PDT_PROY']

# Reemplazar NaN por cero
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_vs_proy_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_vs_proy_2['% AVANCE'] = df_pdt_vs_proy_2['% AVANCE'].replace(np.inf, 0)

df_pdt_vs_proy_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,0.00,ROSARIO ROSALES,EBCO S.A.,-945949.00,0.00,-945949.00,0.00,-1891898.00,0.50
1,P-1591,3. COSTO MANO DE OBRA,0.00,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,0.00,-2483998.00,0.00,-4967996.00,0.50
2,P-1592,2. COSTO MATERIALES,0.00,NUEVA COSTANERA 3300,EBCO S.A.,-9821.00,856004.00,846183.00,1.05,846183.00,1.00
3,P-1603,2. COSTO MATERIALES,0.00,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-30000.00,1.00
4,P-1611,2. COSTO MATERIALES,0.00,EDIFICIO PLAZA EGAÑA,EBCO S.A.,6648.00,596177.00,602825.00,1.14,602825.00,1.00
5,P-1612,2. COSTO MATERIALES,0.00,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,30258.00,280644.00,310902.00,0.00,310902.00,1.00
6,P-1620,2. COSTO MATERIALES,0.00,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-16400.00,1.00
7,P-1626,2. COSTO MATERIALES,0.00,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,3460.00,678564.00,682024.00,10.03,682024.00,1.00
8,P-1632,2. COSTO MATERIALES,0.00,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,0.00,236593.00,236593.00,0.00,236593.00,1.00
9,P-1636,2. COSTO MATERIALES,0.00,SANTO DOMINGO 1410,DANACORP S.A,0.00,-455187.00,-455187.00,0.77,-455187.00,1.00


In [509]:
# TESTING VISUAL 1
 
df_pdt_vs_proy_2[df_pdt_vs_proy_2["CODIGO_PROYECTO"]=="P-1727"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
162,P-1727,3. COSTO MANO DE OBRA,0.00,NaN,NaN,-4714680.00,0.00,-4714680.00,0.23,-7919980.00,0.60
163,P-1727,2. COSTO MATERIALES,0.00,NaN,NaN,-3753865.00,0.00,-3753865.00,0.23,-13253865.00,0.28
164,P-1727,1. INGRESO,0.00,NaN,NaN,11005000.00,0.00,11005000.00,0.23,28260460.00,0.39
330,P-1727,4. COSTO SUPERVISIÓN,0.00,NaN,NaN,0.00,0.00,0.00,0.23,0.00,0.00


In [510]:
# TESTING VISUAL 2
 
df_pdt_vs_proy_2[df_pdt_vs_proy_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_LB,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL YTD,TOTAL LB OLD,TOTAL_PDT,MARGEN,TOTAL_PDT_PROY,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-44701591.00,-87942380.00,-132643971.00,0.48,-146886506.00,0.90
107,P-1704,2. COSTO MATERIALES,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-58121771.00,-125594139.00,-183715910.00,0.48,-207715910.00,0.88
108,P-1704,1. INGRESO,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,141716199.00,465655611.00,607371810.00,0.48,675978469.28,0.90
302,P-1704,4. COSTO SUPERVISIÓN,0.00,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,0.00,0.00


In [511]:
df_pdt_vs_proy_2.to_csv('archivos_respaldo/df_pdt_vs_proy_2.csv', index = False)

### 4.16 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO

#### 4.16.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [520]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 : ", df_ptd_cerrado_y_proyectado2.shape)

df_ptd_cerrado_y_proyectado2_2 = df_ptd_cerrado_y_proyectado2.copy()

df_ptd_cerrado_y_proyectado2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
307,P-1709,4. COSTO SUPERVISIÓN,CENTRO COMERCIAL PASEO SAN PEDRO,EBCO S.A.,0.00,0,0.00,0.43
205,P-1642,3. COSTO MANO DE OBRA,CORINTO,EMP. CONSTRUCTORA INGENIEROS S.A.,0.00,0,0.00,0.57
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0,0.00,0.49
136,P-1716,3. COSTO MANO DE OBRA,CD ASTARA,ASTARA LOGÍSTICA CHILE SPA.,-26321855.00,-1666000,-27987855.00,0.53
27,P-1667,3. COSTO MANO DE OBRA,ECO PLACILLA,CONSTRUCTORA ICF SPA,-37605510.00,-382370,-37987880.00,0.01


In [518]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2 = df_presupuesto_final4.copy()
df_presupuesto_final4_2.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2 = pd.pivot_table(df_presupuesto_final4_2, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2 = pd.DataFrame(df_presupuesto_final4_2.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 8)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
104,P-1666,1. INGRESO,0.00
193,P-1690,2. COSTO MATERIALES,-22724089.00
24,P-1612,1. INGRESO,0.00
51,P-1641,4. COSTO SUPERVISIÓN,-657160.00
71,P-1648,4. COSTO SUPERVISIÓN,-8171095.00


#### 4.16.2 MERGE ENTRE DATAFRAMES

In [519]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 : ", df_ptd_cerrado_y_proyectado2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        : ", df_presupuesto_final4_2.shape)

df_pdt_proy_vs_prep = df_ptd_cerrado_y_proyectado2_2.merge(df_presupuesto_final4_2, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           : ", df_pdt_proy_vs_prep.shape)

df_pdt_proy_vs_prep.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2        :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
40,P-1676,3. COSTO MANO DE OBRA,LO CAMPINO,EBCO S.A.,-29320930.00,0,-29320930.00,0.59,-41959762.00
88,P-1696,1. INGRESO,SANTA VICTORIA - TERRACORP,CONSTRUCTORA TERRACORP S.A.,75515364.00,0,75515364.00,0.12,75985768.29
185,P-1612,4. COSTO SUPERVISIÓN,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0,0.00,0.00,-2259746.00
143,P-1719,1. INGRESO,CD METALPAR,EBCO S.A.,131348.00,0,131348.00,1.00,194809188.00
168,P-1557,1. INGRESO,ROSARIO ROSALES,EBCO S.A.,0.00,0,0.00,0.00,0.00


#### 4.16.3 CREACIÓN DE COLUMNA % DE AVANCE

In [521]:
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['TOTAL_PDT'] / df_pdt_proy_vs_prep['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep['% AVANCE'] = df_pdt_proy_vs_prep['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-1247014.00,-301065,-1548079.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-4593960.00,-2109962,-6703922.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,897720.00,0,897720.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,312558.00,0,312558.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0,-455187.00,0.77,-11381553.00,0.04


In [522]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,62000000,748431807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0,0.00,0.49,-31281353.00,-0.00


In [523]:
# TESTING VISUAL 2
 
df_pdt_proy_vs_prep[df_pdt_proy_vs_prep["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-145534854.00,-12890883,-158425737.00,0.49,-156095149.00,1.01
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-197683461.00,-27000000,-224683461.00,0.49,-387816472.00,0.58
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,686431807.00,62000000,748431807.00,0.49,1058718831.87,0.71
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0,0.00,0.49,-31281353.00,-0.00


In [524]:
df_pdt_proy_vs_prep.to_csv('archivos_respaldo/df_pdt_proy_vs_prep.csv', index = False)

### 4.17 MERGE PTD CERRADO + PROYECTADO VS PRESUPUESTO (PERSPECTIVA MES ANTERIOR)

#### 4.17.1 EXPLORACIÓN Y PREPARACIÓN DE DATAFRAMES

In [527]:
# Exploración de dataframe df_ptd_cerrado_y_proyectado_ant2

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 : ", df_ptd_cerrado_y_proyectado_ant2.shape)

df_ptd_cerrado_y_proyectado_ant2_2 = df_ptd_cerrado_y_proyectado_ant2.copy()

df_ptd_cerrado_y_proyectado_ant2_2.rename(columns={'TOTAL': 'TOTAL_PDT'}, inplace=True)
df_ptd_cerrado_y_proyectado_ant2_2.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2 :  (340, 8)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN
49,P-1680,1. INGRESO,NUEVA UNO,CONSTRUCTORA TECSA S.A.,174843492.00,12372127.00,187215619.00,0.24
292,P-1694,4. COSTO SUPERVISIÓN,ECO HUEMUL,CONSTRUCTORA ICF SPA,0.00,0.00,0.00,0.23
184,P-1612,3. COSTO MANO DE OBRA,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,0.00,0.00,0.00,0.00
142,P-1718,1. INGRESO,PATIO COMIDAS,EBCO S.A.,104494756.00,18676629.00,123171385.00,0.46
174,P-1592,1. INGRESO,NUEVA COSTANERA 3300,EBCO S.A.,17821101.00,0.00,17821101.00,1.05


In [528]:
# Exploración de dataframe presupuesto

df_presupuesto_final4_2_1 = df_presupuesto_final4.copy()
df_presupuesto_final4_2_1.rename(columns={'TOTAL': 'TOTAL_PDT_PREP'}, inplace=True)

print("DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 : ", df_presupuesto_final4_2.shape)

df_presupuesto_final4_2_1 = pd.pivot_table(df_presupuesto_final4_2_1, values=['TOTAL_PDT_PREP'], index=['CODIGO_PROYECTO','TIPO_CUENTA'], aggfunc='sum', fill_value=0)
df_presupuesto_final4_2_1 = pd.DataFrame(df_presupuesto_final4_2_1.to_records())

print("DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 : ", df_presupuesto_final4_2_1.shape)

df_presupuesto_final4_2_1.sample(5)

DIMENSIONES ANTES DE CAMBIOS DATAFRAME: df_presupuesto_final4_2 :  (340, 3)
DIMENSIONES DESPUÉS DE CAMBIOS DATAFRAME: df_presupuesto_final4_2_1 :  (340, 3)


,CODIGO_PROYECTO,TIPO_CUENTA,TOTAL_PDT_PREP
176,P-1686,1. INGRESO,105081270.13
15,P-1592,4. COSTO SUPERVISIÓN,0.00
142,P-1676,3. COSTO MANO DE OBRA,-41959762.00
38,P-1632,3. COSTO MANO DE OBRA,-43457804.00
78,P-1653,3. COSTO MANO DE OBRA,-3230898.00


#### 4.17.2 MERGE ENTRE DATAFRAMES

In [529]:
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 : ", df_ptd_cerrado_y_proyectado_ant2_2.shape)
print("DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          : ", df_presupuesto_final4_2_1.shape)

df_pdt_proy_vs_prep_2 = df_ptd_cerrado_y_proyectado_ant2_2.merge(df_presupuesto_final4_2_1, on=['CODIGO_PROYECTO','TIPO_CUENTA'], how='left')

print("DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           : ", df_pdt_proy_vs_prep_2.shape)

df_pdt_proy_vs_prep_2.sample(5)

DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_ptd_cerrado_y_proyectado_ant2_2 :  (340, 8)
DIMENSIONES ANTERIOR A CAMBIOS DATAFRAME: df_presupuesto_final4_2_1          :  (340, 3)
DIMENSIONES DESPUÉS De CAMBIOS DATAFRAME: df_pdt_proy_vs_prep_2           :  (340, 9)


,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP
283,P-1688,4. COSTO SUPERVISIÓN,SUIZA,MOLLER Y PEREZ - COTAPOS CONSTRUCCIONES INDUST...,0.00,0.00,0.00,0.17,-1849560.00
176,P-1592,4. COSTO SUPERVISIÓN,NUEVA COSTANERA 3300,EBCO S.A.,0.00,0.00,0.00,1.05,0.00
261,P-1674,4. COSTO SUPERVISIÓN,OFICINAS BEIERSDORF CHORRILLOS,CONSTRUCTORA SUKSA S.A.,0.00,0.00,0.00,0.63,-736437.00
89,P-1697,3. COSTO MANO DE OBRA,ESTOCOLMO,CONSTRUCTORA FORTALEZA S.A,-15721639.00,-4075065.00,-19796704.00,0.47,0.00
331,P-1728,1. INGRESO,NaN,NaN,0.00,6372127.00,6372127.00,0.53,7874868.00


#### 4.17.3 CREACIÓN DE COLUMNA % DE AVANCE

In [530]:
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['TOTAL_PDT'] / df_pdt_proy_vs_prep_2['TOTAL_PDT_PREP']

# Reemplazar NaN por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].fillna(0)

# Reemplazar -inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'].replace(-np.inf, 0, inplace=True)

# Reemplazar inf por cero
df_pdt_proy_vs_prep_2['% AVANCE'] = df_pdt_proy_vs_prep_2['% AVANCE'].replace(np.inf, 0)

df_pdt_proy_vs_prep_2.head(10)

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
0,P-1557,3. COSTO MANO DE OBRA,ROSARIO ROSALES,EBCO S.A.,-945949.00,-945949.00,-1891898.00,0.00,0.00,0.00
1,P-1591,3. COSTO MANO DE OBRA,EDIFICIO ESTORIL,EBCO S.A.,-2483998.00,-2483998.00,-4967996.00,0.00,0.00,0.00
2,P-1592,2. COSTO MATERIALES,NUEVA COSTANERA 3300,EBCO S.A.,846183.00,0.00,846183.00,1.05,0.00,0.00
3,P-1603,2. COSTO MATERIALES,FARELLONES,EMPRESA CONSTRUCTORA DLP S.A.,-30000.00,0.00,-30000.00,0.00,-13744197.00,0.00
4,P-1611,2. COSTO MATERIALES,EDIFICIO PLAZA EGAÑA,EBCO S.A.,602825.00,0.00,602825.00,1.14,-24208141.00,-0.02
5,P-1612,2. COSTO MATERIALES,BARRIO PARQUE SANTIAGO 7,CONSTRUCTORA ALTURAS DEL SUR S.A,310902.00,0.00,310902.00,0.00,-30609079.00,-0.01
6,P-1620,2. COSTO MATERIALES,"OF APOQUINDO- HABILITACIONES, INTEGRACION Y AD...",CONSTRUCTORA INARCO S.A.,-16400.00,0.00,-16400.00,0.00,-11746595.00,0.00
7,P-1626,2. COSTO MATERIALES,CLINICA BIO BIO,CONSTRUCTORA LYD S.A.,682024.00,0.00,682024.00,10.03,-168705405.50,-0.00
8,P-1632,2. COSTO MATERIALES,CHORRILLOS CLOROX,CONSTRUCTORA SUKSA S.A.,236593.00,0.00,236593.00,0.00,-219387084.00,-0.00
9,P-1636,2. COSTO MATERIALES,SANTO DOMINGO 1410,DANACORP S.A,-455187.00,0.00,-455187.00,0.77,-11381553.00,0.04


In [531]:
# TESTING VISUAL 1
 
df_pdt_proy_vs_prep_2[df_pdt_proy_vs_prep_2["CODIGO_PROYECTO"]=="P-1704"]

,CODIGO_PROYECTO,TIPO_CUENTA,NOMBRE_OBRA,NOMBRE_CLIENTE,TOTAL PTD,TOTAL MEC PROY,TOTAL_PDT,MARGEN,TOTAL_PDT_PREP,% AVANCE
106,P-1704,3. COSTO MANO DE OBRA,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-132643971.00,-14242535.00,-146886506.00,0.48,-156095149.00,0.94
107,P-1704,2. COSTO MATERIALES,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,-183715910.00,-24000000.00,-207715910.00,0.48,-387816472.00,0.54
108,P-1704,1. INGRESO,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,607371810.00,68606659.28,675978469.28,0.48,1058718831.87,0.64
302,P-1704,4. COSTO SUPERVISIÓN,CD MERCADO LIBRE MEZZANINA,EBCO S.A.,0.00,0.00,0.00,0.48,-31281353.00,-0.00


In [532]:
df_pdt_proy_vs_prep_2.to_csv('archivos_respaldo/df_pdt_proy_vs_prep_2.csv', index = False)

# AQUÍ ESTOY

## 5. - EXPORTACIÓN DE ARCHIVOS

In [314]:
# Conexión a google drive

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

# Credenciales de GOOGLE API, en archivo .json
credentials = ServiceAccountCredentials.from_json_keyfile_name('keen-extension-358919-9214486a06be.json', scope) # Your json file here

gc = gspread.authorize(credentials)

## SIEMPRE DAR ACCESO A ESTE USUARIO: test-321@keen-extension-358919.iam.gserviceaccount.com

### 5.1 EXPORTACIÓN LIBRO MAYOR PARA CUADRO FINANCIERO DE PROYECTOS

In [315]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1COX0_p3dTPNLZMwoh__0uQdOYfP_AA1mw7lfKx4SMMQ')

# select a work sheet from its name
worksheet1 = gs.worksheet('Hoja 1')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe=df_libro_mayor6, include_index=False,
include_column_header=True, resize=True)

### 5.2 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO
Dataframe: df_ytd_cerrado3

In [316]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.2.1 EXPORTACIÓN LIBRO MAYOR YTD HASTA MES CERRADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: df_ytd_cerrado_2M_3

In [317]:
gs = gc.open_by_key('1_53Vj0j4xnqsWDdHDdetp3JmoUHRjLlvsUyi7tQXCY4')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_cerrado_2M_3, include_index=False,
include_column_header=True, resize=True)

### 5.3 EXPORTACIÓN MES EN CURSO PROYECTADO
Dataframe: mes_en_curso2

In [318]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy5, include_index=False,
include_column_header=True, resize=True)

### 5.3.1 EXPORTACIÓN MES EN CURSO PROYECTADO (MES ANTERIOR) PERSPECTIVA MES ANTERIOR
Dataframe: mes_en_curso_2M

In [319]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1xyeMVO6VL0yBCX6aZRcFxNKPH5yum2o2gF_HM2NSLIs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_proy_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.4 EXPORTACIÓN MES EN CURSO

In [320]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real5, include_index=False,
include_column_header=True, resize=True)

### 5.4.1 EXPORTACIÓN MES EN CURSO PERSPECTIVA MES ANTERIOR

In [321]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1k4Yf_ycqV1TjqPcS0eDGyUU_oKu9JVzYUHw9shUBGTM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= mes_en_curso_real_2M_5, include_index=False,
include_column_header=True, resize=True)

### 5.5 EXPORTACIÓN YTD PROYECTADO

In [359]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.5.1 EXPORTACIÓN YTD PROYECTADO PERSPECTIVA MES ANTERIOR

In [360]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1_39cxdPhNrDcz2WJ-t3oLQRpgQPhWwFsopAGzcmxuuk')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ytd_proyectado_2M_2, include_index=False,
include_column_header=True, resize=True)

### 5.6 EXPORTACIÓN PTD CERRADO

In [404]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado3, include_index=False,
include_column_header=True, resize=True)

### 5.6.1 EXPORTACIÓN PTD CERRADO (PERSPECTIVA MES ANTERIOR)

In [405]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1bMRHlz-oMTZ7ccBXbEvjQ3es7mBcWxZqpVrPBXGAVjc')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_anterior3, include_index=False,
include_column_header=True, resize=True)

### 5.7 EXPORTACIÓN PTD CERRADO + PROYECTADO

In [406]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado2, include_index=False,
include_column_header=True, resize=True)

### 5.7.1 EXPORTACIÓN PTD CERRADO + PROYECTADO (PERSPECTIVA MES ANTERIOR)

In [420]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('14dw0WLmoa8gDJLcOkDJor17BCy7pozNQK7L1bSvJQqs')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_ptd_cerrado_y_proyectado_ant2, include_index=False,
include_column_header=True, resize=True)

### 5.8 EXPORTACIÓN PRESUPUESTO

In [447]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yTtFi7x0rc_FU6xb-FrdQGKndttaf0dg3ENDyOO52eM')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_presupuesto_final4, include_index=False,
include_column_header=True, resize=True)

### 5.9 EXPORTACIÓN PENDIENTE

In [466]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente3, include_index=False,
include_column_header=True, resize=True)

### 5.10 EXPORTACIÓN PENDIENTE (PERSPECTIVA MES ANTERIOR)

In [491]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1qLDVshnNP-whjDaMYi1N4QSeeD6AK4EVSQtQx6Y438o')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pendiente_3, include_index=False,
include_column_header=True, resize=True)

### 5.11 EXPORTACIÓN PTD Cerrado vs PTD Cerrado + Proyectado

In [503]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_proy, include_index=False,
include_column_header=True, resize=True)

### 5.12 EXPORTACIÓN PTD Cerrado vs PTD Cerrado + Proyectado (PERSPECTIVA MES ANTERIOR)

In [513]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1ZsZGZ5uzyEK7rOqHXER0MYjKbrbiLK9cdeYlXk4t8oE')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_vs_proy_2, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto

In [526]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_actual')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep, include_index=False,
include_column_header=True, resize=True)

### 5.13 EXPORTACIÓN  PTD Cerrado + Proyectado vs Presupuesto (PERSPECTIVA ANTERIOR)

In [533]:
# write to dataframe
# open a google sheet
gs = gc.open_by_key('1yqyEJ4mkcaS_kI7rrNZ8reRxqFhWdbhBtvDcSeWxkPo')

# select a work sheet from its name
worksheet1 = gs.worksheet('vista_anterior')
worksheet1.clear()
set_with_dataframe(worksheet=worksheet1, dataframe= df_pdt_proy_vs_prep_2, include_index=False,
include_column_header=True, resize=True)